In [ ]:
%matplotlib inline
import sys
from IPython.display import display
from pathlib import Path as path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.transforms as transforms
import matplotlib.gridspec as gridspec
from matplotlib import ticker
from matplotlib.colors import ListedColormap
from matplotlib.colorbar import Colorbar
import xarray as xr
import spectrochempy as scp
from spectrochempy import ur
import hyperspy.api as hs
import scipy

In [ ]:
# 画图的初始设置
plt.style.use(r'C:\Users\chengliu\OneDrive - UAB\ICMAB-python\Figure\liuchzzyy.mplstyle')
# display(plt.style.available)
plt.style.use('default')

# 颜色设定
sys.path.append(r'C:\Users\chengliu\OneDrive - UAB\ICMAB-Python\Figure')
from colors import tol_cmap, tol_cset
colors = list(tol_cset('vibrant'))
if r'sunset' not in plt.colormaps():
    plt.colormaps.register(tol_cmap('sunset'))
if r'rainbow_PuRd' not in plt.colormaps():
    plt.colormaps.register(tol_cmap('rainbow_PuRd')) # 备用 plasma

# 输出的文件夹
path_out = path(r"C:\Users\chengliu\Desktop\Figure")

# V4

### Operando Data,除去 Mn2+ 的部分, pre-edge 数据获取

In [ ]:
# 读取数据
path_file = path(r'C:\Users\chengliu\OneDrive - UAB\ICMAB-Data\Zn-Mn\Uno\Result\XAS\Operando\αMnO2\XAS\CLAESS\2022-10\Results\cell3\PaperUno\Oct2022_cell3_P2a\XANES\Mn\TrialsOne_αMnO2_Pristine\MCR')
Mn_raw = xr.open_dataset(path.joinpath(path_file, r'Raw_data.NETCDF4'), engine='h5netcdf')
Mn_mcr_2_hm = xr.open_dataset(path.joinpath(path_file, r'PcaMcr_onereaction_2e_data.NETCDF4'), engine='h5netcdf')

In [ ]:
MnO2_rest = Mn_raw['Raw_spectrum'].sel(Energy=slice(Mn_mcr_2_hm['Energy'].data[0], Mn_mcr_2_hm['Energy'].data[-1])).data - (Mn_mcr_2_hm['MCR_HM_C'][:,1]*Mn_mcr_2_hm['MCR_HM_St'][1,:]).data.T

MnO2_Rest = xr.DataArray(
    data=MnO2_rest,
    coords={
        'Spectrum': range(MnO2_rest.shape[1]),
        'Energy': Mn_raw['Raw_spectrum'].sel(Energy=slice(Mn_mcr_2_hm['Energy'].data[0], Mn_mcr_2_hm['Energy'].data[-1]))['Energy'],
    },
    dims=['Energy', 'Spectrum'],
    name = r'MnO2_Rest',
)

#### PCA 降噪

In [ ]:
MnO2_rest = scp.NDDataset(data=MnO2_Rest.data.T,
                     name="opPre_Edge_Mn_Oct2022",
                     author="Cheng Liu, and Dino Tonti",
                     description="XANES obtained from CLAESS, alba, July2023",
                     history="creation",
                     title='absorption',
                     units=ur.absorbance)

MnO2_rest.y = scp.Coord.arange(MnO2_Rest['Spectrum'].shape[0], title='sperctum number')
MnO2_rest.x = scp.Coord(MnO2_Rest.coords['Energy'], title='Energy', units=ur.eV)

MnO2_rest_denoise = scp.denoise(MnO2_rest)
MnO2_Rest_Denoise = MnO2_rest_denoise.to_xarray().rename('MnO2_Rest_Denoise')
MnO2_Rest_Denoise.attrs['units'] = r'a.u.'
MnO2_rest_denoise.plot()

In [ ]:
# 保存数据
xr.merge([Mn_raw['Raw_spectrum'], Mn_raw['Raw_ref'], Mn_mcr_2_hm['MCR_HM_C'], Mn_mcr_2_hm['MCR_HM_St'], MnO2_Rest, MnO2_Rest_Denoise],).to_netcdf(path.joinpath(path_out, r'opPreEdge_Mn.NETCDF4'), engine="h5netcdf")

MnO2_Rest1 = MnO2_Rest.to_pandas()
MnO2_Rest1.columns = Mn_raw['Spectrum'].data
MnO2_Rest1.reset_index().to_csv(path.joinpath(path_out, r'opPreEdge_Mn_No_Mn_2.csv'), header=True, index=False)

MnO2_Rest_Denoise1 = MnO2_Rest_Denoise.to_pandas().T
MnO2_Rest_Denoise1.columns = Mn_raw['Spectrum'].data
MnO2_Rest_Denoise1.reset_index().to_csv(path.joinpath(path_out, r'opPreEdge_Mn_No_Mn_2_denoise.csv'), header=True, index=False)

In [ ]:
%matplotlib inline
plt.close('all')

fig = plt.figure(figsize=(7.0, 2.5))
gs = gridspec.GridSpec(1, 3, width_ratios=[1,1,1], height_ratios=None, wspace=0, hspace=0, figure=fig)

xascolors = [colors[0], colors[4]]
labels = [r'0.2M Mn$\mathrm{^{2\!+}}$', r'${\alpha}$MnO$\mathrm{_2}$']
Mn_cmap = ListedColormap(mpl.colormaps['sunset'](np.linspace(0.5, 1.0, MnO2_Rest.shape[1])), name='Mn_cmap', )

# 图 A
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_subplot()
ax.set_position((0, 0, 1.0, 1.0))
ax.set_box_aspect(0.8)

for i in range(2):
    ax.plot(Mn_raw['Energy'].sel(Energy=slice(6537.5, 6550.5)), Mn_raw['Raw_ref'].sel(Energy=slice(6537.5, 6550.5))[:, i], c=xascolors[i], ls='--', lw=1.0, label=labels[i], zorder=5)
ax.legend(loc='upper left', bbox_to_anchor=(0.0, 1.03), ncols=1, frameon=False, labelcolor='linecolor', fontsize=10)

for i in range(Mn_raw['Raw_spectrum'].shape[1]):
    ax.plot(Mn_raw['Energy'].sel(Energy=slice(6537.5, 6550.5)), Mn_raw['Raw_spectrum'].sel(Energy=slice(6537.5, 6550.5))[:, i], c=Mn_cmap.colors[i], ls='-', lw=1.0)
    
ax.set_xlim(6537.5, 6550.5)
ax.set_xlabel(r'Energy (eV)', fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=2, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=1, offset=0))
ax.set_ylim(0, 0.3)
ax.set_ylabel(r"Absorption (arb.u.)", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.06))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.03))
ax.tick_params(axis='both', labelsize=9) 

colorbar = Colorbar(ax=ax.inset_axes((0.65, 0.05, 0.3, 0.06)), location='bottom', orientation='horizontal', cmap=Mn_cmap, ticklocation='top', spacing = 'proportional', drawedges=False)
colorbar.set_ticks([])

ax.text(0.65, 0.18, 'OCV', transform=ax.transAxes, fontsize=9, va='top', ha='left', fontfamily='Arial', )
ax.text(0.86, 0.18, '0.9 V', transform=ax.transAxes, fontsize=9, va='top', ha='left', fontfamily='Arial', )

# 图 B
subfig = fig.add_subfigure(gs[0, 1])
ax = subfig.add_subplot()
ax.set_position((0.3, 0, 1.0, 1.0))
ax.set_box_aspect(0.8)

for i in range(2):
    ax.plot(Mn_raw['Energy'].sel(Energy=slice(6537.5, 6550.5)), Mn_raw['Raw_ref'].sel(Energy=slice(6537.5, 6550.5))[:, i], c=xascolors[i], ls='--', lw=1.0, label=labels[i], zorder=1)
ax.legend(loc='upper left', bbox_to_anchor=(0.0, 1.03), ncols=1, frameon=False, labelcolor='linecolor', fontsize=10)

for i in range(MnO2_Rest.shape[1]):
    ax.plot(MnO2_Rest['Energy'].sel(Energy=slice(6537.5, 6550.5)), MnO2_Rest.sel(Energy=slice(6537.5, 6550.5))[:, i], c=Mn_cmap.colors[i], ls='-', lw=1.0)
    
ax.set_xlim(6537.5, 6550.5)
ax.set_xlabel(r'Energy (eV)', fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=2, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=1, offset=0))
ax.set_ylim(0, 0.12)
ax.set_ylabel(r"Absorption (arb.u.)", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.03))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.015))
ax.tick_params(axis='both', labelsize=9) 

colorbar = Colorbar(ax=ax.inset_axes((0.65, 0.05, 0.3, 0.06)), location='bottom', orientation='horizontal', cmap=Mn_cmap, ticklocation='top', spacing = 'proportional', drawedges=False)
colorbar.set_ticks([])

ax.text(0.65, 0.18, 'OCV', transform=ax.transAxes, fontsize=9, va='top', ha='left', fontfamily='Arial', )
ax.text(0.86, 0.18, '0.9 V', transform=ax.transAxes, fontsize=9, va='top', ha='left', fontfamily='Arial', )

# 图 C
subfig = fig.add_subfigure(gs[0, 2])
ax = subfig.add_subplot()
ax.set_position((0.6, 0, 1.0, 1.0))
ax.set_box_aspect(0.8)

for i in range(2):
    ax.plot(Mn_raw['Energy'].sel(Energy=slice(6537.5, 6550.5)), Mn_raw['Raw_ref'].sel(Energy=slice(6537.5, 6550.5))[:, i], c=xascolors[i], ls='--', lw=1.0, label=labels[i], zorder=5)
ax.legend(loc='upper left', bbox_to_anchor=(0.0, 1.03), ncols=1, frameon=False, labelcolor='linecolor', fontsize=10)

for i in range(MnO2_Rest_Denoise.shape[0]):
    ax.plot(MnO2_Rest_Denoise['Energy'].sel(Energy=slice(6537.5, 6550.5)), MnO2_Rest_Denoise.sel(Energy=slice(6537.5, 6550.5))[i, :], c=Mn_cmap.colors[i], ls='-', lw=1.0)

ax.set_xlim(6537.5, 6550.5)
ax.set_xlabel(r'Energy (eV)', fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=2, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=1, offset=0))
ax.set_ylim(-0.01, 0.12)
ax.set_ylabel(r"Absorption (arb.u.)", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.03))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.015))
ax.tick_params(axis='both', labelsize=9) 

colorbar = Colorbar(ax=ax.inset_axes((0.65, 0.05, 0.3, 0.06)), location='bottom', orientation='horizontal', cmap=Mn_cmap, ticklocation='top', spacing = 'proportional', drawedges=False)
colorbar.set_ticks([])

ax.text(0.65, 0.18, 'OCV', transform=ax.transAxes, fontsize=9, va='top', ha='left', fontfamily='Arial', )
ax.text(0.86, 0.18, '0.9 V', transform=ax.transAxes, fontsize=9, va='top', ha='left', fontfamily='Arial', )

plt.savefig(path.joinpath(path_out, r'PreEdge_1_300_V3_0.tif'), transparent=False, pad_inches=0.05, bbox_inches='tight', dpi=300, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'PreEdge_1_600_V3_0.tif'), transparent=False, pad_inches=0.05, bbox_inches='tight', dpi=600, pil_kwargs={"compression": "tiff_lzw"})
plt.show()

#### 找到所有的 data, 并除去背景

In [ ]:
def index(energy, value):
    return np.argmin(abs(energy-value))

In [ ]:
root_dir = path(r'C:\Users\chengliu\OneDrive - UAB\ICMAB-Data\Zn-Mn\Uno\Result\XAS\Operando\αMnO2\XAS\CLAESS\2022-10\Results\cell3\PaperUno\Oct2022_cell3_P2a\PreEdge\PreEdge\V3\2-bg_Remove')
dat_files = list(root_dir.rglob('*.dat'))

df=[]
for file in dat_files:
    data = pd.read_csv(file, sep='\s+', skiprows=0, comment='#', header=0, index_col=None, usecols=[0, 1, 2, 3])
    data_baseline = data.iloc[:, 2:4].copy().dropna(axis=0, how='any', ignore_index=True, inplace=False)
    data = data.iloc[:, 0:2].copy().dropna(axis=0, how='any', ignore_index=True, inplace=False)
    data = data.truncate(before=index(data.iloc[:, 0], data_baseline.iloc[0, 0]), after=index(data.iloc[:, 0], data_baseline.iloc[-1, 0]), axis=0, copy=True).reset_index(drop=True)
    data2 = data.iloc[:,1] - data_baseline.iloc[:,1]
    df.append(data)
    df.append(data_baseline.iloc[:,1])
    df.append(data2)  
data = pd.concat(df, ignore_index=True, axis=1)
data.to_csv(path.joinpath(path_out, r'PreEdge.csv'), index=None, header=True)

dat_files

#### 读取数据

In [ ]:
path_file = path(r'C:\Users\chengliu\OneDrive - UAB\ICMAB-Data\Zn-Mn\Uno\Result\XAS\Operando\αMnO2\XAS\CLAESS\2022-10\Results\cell3\PaperUno\Oct2022_cell3_P2a\PreEdge\PreEdge\V3\PreEdge.csv')
data = pd.read_csv(path_file, sep=',', comment='#', header=0, index_col=None)
data

In [ ]:
# 画图
%matplotlib inline
fig = plt.figure(figsize=(7, 2.5))
gs = gridspec.GridSpec(1, 2, width_ratios=[1, 1], height_ratios=None, wspace=0.0, hspace=0.0, figure=fig)

xas_colors_Mn = [colors[0], colors[4]]
xas_cmap_Mn = ListedColormap(mpl.colormaps['coolwarm'](np.linspace(0.5, 1.0, data.shape[1]//4-6)), name='xas_cmap_Mn', )
labels_Mn = [r'${\alpha}$-MnO$\mathrm{_2}$', r'0.2M Mn$\mathrm{^{2\!+}}$']

# 图 B
subfig = fig.add_subfigure(gs[0, 0], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0, 0, 1.0, 1.0))
ax.set_box_aspect(0.8)

ax.plot(data.iloc[:, 4], data.iloc[:, 7]+0.06, ls='-', color=colors[4], label=r'ref.MnO')
ax.plot(data.iloc[:, 0], data.iloc[:, 3]+0.06, ls='-', color=colors[5], label=r'ref.Mn$\mathrm{_2O_3}$')
ax.plot(data.iloc[:, 8], data.iloc[:, 11]+0.06, ls='-', color=colors[3], label=r'ref.Mn$\mathrm{O_2}$')
ax.plot(data.iloc[:, 16], data.iloc[:, 19]+0.06, ls='-', color=colors[4], label=r'ref.Mn$\mathrm{^{2+}}$')
ax.plot(data.iloc[:, 12], data.iloc[:, 15]+0.06, ls='-', color=colors[3], label=r'ref.$\mathrm{\alpha}$Mn$\mathrm{O_2}$')

ax.legend(loc='upper left', bbox_to_anchor=(0.0, 1.02), ncols=1, frameon=False, fontsize=9, labelcolor='linecolor', columnspacing=0.0)

for i in range(data.shape[1]//4-6):
    ax.plot(data.iloc[:, i*4+24], data.iloc[:, i*4+27], c=xas_cmap_Mn.colors[i], ls='-', label=None)

ax.set_xlim(6530.0, 6550.0)
ax.set_xlabel(r'Energy (eV)', fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=5, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=2.5, offset=0))
ax.set_ylim(-0.01, 0.20)
ax.set_ylabel(r'Relative Intensity (arb.u.)', fontsize=11,)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.05))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.025))

# Add colorbar and adjust ticks afterwards
colorbar = Colorbar(ax=ax.inset_axes((0.5, 0.8, 0.4, 0.07)), location='bottom', orientation='horizontal', 
                      cmap=xas_cmap_Mn, ticklocation='top', spacing = 'proportional', drawedges=False)
colorbar.set_ticks([])
colorbar.outline.set_visible(False)
ax.text(0.5, 0.94, 'OCV', transform=ax.transAxes, fontsize=9,
        va='top', ha='left', fontfamily='Arial', )
ax.text(0.65, 0.94, r'Discharge', transform=ax.transAxes, fontsize=9,
        va='top', ha='left', fontfamily='Arial', )

# ax.text(-0.25, 1.0, r'b', transform=ax.transAxes, fontsize=14, va='center', ha='right', fontfamily='Arial', fontweight='bold')

ax.axvline(x=6540.3, ymax=0.75, ls='--')
ax.axvline(x=6543.0, ymax=0.75, ls='--')

plt.savefig(path.joinpath(path_out, r'PreEdge_2_300_V3_0.tif'), transparent=False, pad_inches=0.05, bbox_inches='tight', dpi=300, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'PreEdge_2_600_V3_0.tif'), transparent=False, pad_inches=0.05, bbox_inches='tight', dpi=600, pil_kwargs={"compression": "tiff_lzw"})
plt.show()

### Mn2+ Solution

#### 拟合峰形, pseudo-Voigt 峰 - 两个

In [ ]:
# 通过对比 和 Mn2O3 的拟合值, 确定拟合参数
pv1 = hs.model.components1D.Voigt(centre=6540.3, sigma=0.62343, gamma=0.55, module='scipy')
pv1.name = r'Mn_II_1'
# pv1.lwidth = 1.1
# pv1.gwidth = 1.3
pv1.set_parameters_not_free(parameter_name_list=['sigma', 'gamma',])
# pv1.set_parameters_not_free(parameter_name_list=['gamma',])
pv1.sigma.bmin = 0.55
pv1.sigma.bmax = 0.65
pv1.centre.bmin = 6537.5
pv1.centre.bmax = 6542.5
pv1.area.bmin = 0

pv2 = hs.model.components1D.Voigt(centre=6540.0, module='scipy')
pv2.name = r'Mn_II_2'
pv2.centre.bmin = 6537.5
pv2.centre.bmax = 6542.5
pv2.area.bmin = 0
pv2.sigma.twin = pv1.sigma
pv2.gamma.twin = pv1.gamma

In [ ]:
Mn_Mn_2 = data.iloc[:, [16, 19]].copy().dropna(axis=0, how='all')

In [ ]:
from hyperspy.axes import DataAxis
energy_Mn_2 = Mn_Mn_2.iloc[:, 0].to_numpy(dtype='float64')
energy_Mn_2 = DataAxis(axis=energy_Mn_2, index_in_array=None, name='Energy', units='eV',)
Mn_2 = hs.signals.Signal1D(Mn_Mn_2.iloc[:, 1], axes=[energy_Mn_2,])
Mn_2 = Mn_2.isig[6530.0:].deepcopy()
Mn_2.plot()

In [ ]:
Mn_2_model = Mn_2.create_model()
Mn_2_model.extend((pv1, pv2))
Mn_2_model.set_signal_range(6538.0, 6544.0)

In [ ]:
%matplotlib inline
plt.close('all')
Mn_2_model.fit(optimizer="lm", bounded=True, return_info=False) 
Mn_2_model.plot(plot_components=True, plot_residual=True)
plt.savefig(path.joinpath(path_out, r'PreEdge_3_300_V3_0.tif'), pad_inches=0.05, bbox_inches='tight', dpi=300, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'PreEdge_3_600_V3_0.tif'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
Mn_2_model.print_current_values()

In [ ]:
# 转化成 DataFrame
fit_Mn_II_1 = pd.DataFrame([Mn_2_model.axes_manager[0].axis, Mn_2_model.as_signal(component_list=['Mn_II_1']).data])
fit_Mn_II_1 = fit_Mn_II_1.T
fit_Mn_II_1.dropna(how='any', inplace=True)

fit_Mn_II_2 = pd.DataFrame([Mn_2_model.axes_manager[0].axis, Mn_2_model.as_signal(component_list=['Mn_II_2']).data])
fit_Mn_II_2 = fit_Mn_II_2.T
fit_Mn_II_2.dropna(how='any', inplace=True)

In [ ]:
Pre_Edge_Mn_2 = xr.Dataset(
    data_vars=dict(
        Mn_2 = (['Energy_Mn_2'], Mn_2.data),
        fit = (['Energy_fit'], Mn_2_model.as_signal().data),
        fit_Mn_II_1 = (['Energy_fit'], Mn_2_model.as_signal(component_list=['Mn_II_1']).data),
        fit_Mn_II_2 = (['Energy_fit'], Mn_2_model.as_signal(component_list=['Mn_II_2']).data),
        ),
    coords=dict(
        Energy_Mn_2 = Mn_2.axes_manager[0].axis,
        Energy_fit= Mn_2_model.axes_manager[0].axis,
    ),
    attrs=dict(
        chisq = Mn_2_model.chisq.data[0],
        red_chisq = Mn_2_model.red_chisq.data[0],
    ),
)

Mn_2_model.save_parameters2file(path.joinpath(path_out, r'Mn_2_model_parameters_voigt_2'))
Mn_2_model.save(path.joinpath(path_out, r'Mn_2_model_voigt_2'), 'model', overwrite=True)
Pre_Edge_Mn_2.to_netcdf(path.joinpath(path_out, r'Mn_2_result_voigt_2.NETCDF4'), engine="h5netcdf")

In [ ]:
# 画图
%matplotlib inline
fig = plt.figure(figsize=(7, 2.5))
gs = gridspec.GridSpec(1, 2, width_ratios=[1, 1], height_ratios=None, wspace=0.0, hspace=0.0, figure=fig)

xas_colors_Mn = [colors[0], colors[4]]
xas_cmap_Mn = ListedColormap(mpl.colormaps['coolwarm'](np.linspace(0.5, 1.0, data.shape[1]//4-5)), name='xas_cmap_Mn', )
labels_Mn = [r'${\alpha}$-MnO$\mathrm{_2}$', r'0.2M Mn$\mathrm{^{2\!+}}$']

# 图
subfig = fig.add_subfigure(gs[0, 0], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0, 0, 1.0, 1.0))
ax.set_box_aspect(0.8)

ax.plot(data.iloc[:, 4], data.iloc[:, 7]+0.08, ls='-', color=colors[4], label=r'ref.MnO')
ax.plot(data.iloc[:, 0], data.iloc[:, 3]+0.08, ls='-', color=colors[5], label=r'ref.Mn$\mathrm{_2O_3}$')
ax.plot(data.iloc[:, 8], data.iloc[:, 11]+0.08, ls='-', color=colors[3], label=r'ref.Mn$\mathrm{O_2}$')
ax.legend(loc='upper left', bbox_to_anchor=(0.0, 1.02), ncols=1, frameon=False, fontsize=9, labelcolor='linecolor', columnspacing=0.0)

ax.plot(data.iloc[:, 16], data.iloc[:, 19], ls='-', color='k', label=None)
# ax.plot(fit_Mn_II_2.iloc[:, 0], fit_Mn_II_2.iloc[:, 1], ls='--', color=colors[4], label=None)
ax.plot(fit_Mn_II_1.iloc[:, 0], fit_Mn_II_1.iloc[:, 1], ls='--', color=colors[4], label=None)
# ax.plot(Mn_2_model.axes_manager[0].axis, Mn_2_model.as_signal().data, ls='--', color=colors[3], label=None)

ax.set_xlim(6535.0, 6550.0)
ax.set_xlabel(r'Energy (eV)', fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=5, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=2.5, offset=0))
ax.set_ylim(-0.01, 0.20)
ax.set_ylabel(r'Relative Intensity (arb.u.)', fontsize=11,)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.05))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.025))

# ax.text(-0.25, 1.0, r'a', transform=ax.transAxes, fontsize=14, va='center', ha='right', fontfamily='Arial', fontweight='bold')

ax.axvline(x=6540.3, ymax=0.65, ls='--')
ax.axvline(x=6543.0, ymax=0.7, ls='--')

plt.savefig(path.joinpath(path_out, r'PreEdge_4_300_V3_0.tif'), transparent=False, pad_inches=0.05, bbox_inches='tight', dpi=300, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'PreEdge_4_600_V3_0.tif'), transparent=False, pad_inches=0.05, bbox_inches='tight', dpi=600, pil_kwargs={"compression": "tiff_lzw"})
plt.show()

### 拟合 Mn(III) Reference

#### 拟合峰形, pseudo-Voigt 峰 - 三个

In [ ]:
pv1 = hs.model.components1D.Voigt(centre=6540.7, sigma=0.62343, gamma=0.55, module='scipy')
pv1.name = r'Mn_III_1'
# pv1.lwidth = 1.1
# pv1.gwidth = 1.3
# pv1.set_parameters_not_free(parameter_name_list=['sigma', 'gamma',])
pv1.set_parameters_not_free(parameter_name_list=['gamma',])
pv1.sigma.bmin = 0.55
pv1.sigma.bmax = 0.65
pv1.centre.bmin = 6538.0
pv1.centre.bmax = 6542.5
pv1.area.bmin = 0

pv2 = hs.model.components1D.Voigt(centre=6543.5, module='scipy')
pv2.name = r'Mn_III_2'
pv2.centre.bmin = 6543.0
pv2.centre.bmax = 6545.0
pv2.area.bmin = 0
pv2.sigma.twin = pv1.sigma
pv2.gamma.twin = pv1.gamma

pv3 = hs.model.components1D.Voigt(centre=6542.3, module='scipy')
pv3.name = r'Mn_III_3'
pv3.centre.bmin = pv1.centre.bmax - 0.5
pv3.centre.bmax = pv2.centre.bmin + 0.5
pv3.area.bmin = 0
pv3.sigma.twin = pv1.sigma
pv3.gamma.twin = pv1.gamma

In [ ]:
Mn_Mn_III = data.iloc[:, [0, 3]].copy().dropna(axis=0, how='all')

In [ ]:
from hyperspy.axes import DataAxis
energy_Mn_Mn_III = Mn_Mn_III.iloc[:, 0].to_numpy(dtype='float64')
energy_Mn_Mn_III = DataAxis(axis=energy_Mn_Mn_III, index_in_array=None, name='Energy', units='eV',)
Mn_III = hs.signals.Signal1D(Mn_Mn_III.iloc[:, 1], axes=[energy_Mn_Mn_III,])
Mn_III = Mn_III.isig[6535.0:].deepcopy()
Mn_III.plot()

In [ ]:
Mn_III_model = Mn_III.create_model()
Mn_III_model.extend((pv1, pv2, pv3))
Mn_III_model.set_signal_range(6537.0, 6548.0)

In [ ]:
%matplotlib inline
plt.close('all')
Mn_III_model.fit(optimizer="lm", bounded=True, return_info=False) 
Mn_III_model.plot(plot_components=True, plot_residual=True)
plt.savefig(path.joinpath(path_out, r'PreEdge_5_300_V3_0.tif'), pad_inches=0.05, bbox_inches='tight', dpi=300, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'PreEdge_5_600_V3_0.tif'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
Mn_III_model.print_current_values()

In [ ]:
# 转化成 DataFrame
fit_Mn_III_1 = pd.DataFrame([Mn_III_model.axes_manager[0].axis, Mn_III_model.as_signal(component_list=['Mn_III_1']).data])
fit_Mn_III_1 = fit_Mn_III_1.T
fit_Mn_III_1.dropna(how='any', inplace=True)

fit_Mn_III_2 = pd.DataFrame([Mn_III_model.axes_manager[0].axis, Mn_III_model.as_signal(component_list=['Mn_III_2']).data])
fit_Mn_III_2 = fit_Mn_III_2.T
fit_Mn_III_2.dropna(how='any', inplace=True)

fit_Mn_III_3 = pd.DataFrame([Mn_III_model.axes_manager[0].axis, Mn_III_model.as_signal(component_list=['Mn_III_3']).data])
fit_Mn_III_3 = fit_Mn_III_3.T
fit_Mn_III_3.dropna(how='any', inplace=True)

In [ ]:
Pre_Edge_Mn_III = xr.Dataset(
    data_vars=dict(
        Mn_III = (['Energy_Mn_III'], Mn_III.data),
        fit = (['Energy_fit'], Mn_III_model.as_signal().data),
        fit_Mn_III_1 = (['Energy_fit'], Mn_III_model.as_signal(component_list=['Mn_III_1']).data),
        fit_Mn_III_2 = (['Energy_fit'], Mn_III_model.as_signal(component_list=['Mn_III_2']).data),
        fit_Mn_III_3 = (['Energy_fit'], Mn_III_model.as_signal(component_list=['Mn_III_3']).data),
        ),
    coords=dict(
        Energy_Mn_III = Mn_III.axes_manager[0].axis,
        Energy_fit= Mn_III_model.axes_manager[0].axis,
    ),
    attrs=dict(
        chisq = Mn_III_model.chisq.data[0],
        red_chisq = Mn_III_model.red_chisq.data[0],
    ),
)

Mn_III_model.save_parameters2file(path.joinpath(path_out, r'Ref_Mn_3_model_parameters_voigt_3'))
Mn_III_model.save(path.joinpath(path_out, r'Ref_Mn_3_model_voigt_3'), 'model', overwrite=True)
Pre_Edge_Mn_III.to_netcdf(path.joinpath(path_out, r'Ref_Mn_3_result_voigt_3.NETCDF4'), engine="h5netcdf")

In [ ]:
# 画图
%matplotlib inline
fig = plt.figure(figsize=(7, 2.5))
gs = gridspec.GridSpec(1, 2, width_ratios=[1, 1], height_ratios=None, wspace=0.0, hspace=0.0, figure=fig)

xas_colors_Mn = [colors[0], colors[4]]
xas_cmap_Mn = ListedColormap(mpl.colormaps['coolwarm'](np.linspace(0.5, 1.0, data.shape[1]//4-5)), name='xas_cmap_Mn', )
labels_Mn = [r'${\alpha}$-MnO$\mathrm{_2}$', r'0.2M Mn$\mathrm{^{2\!+}}$']

# 图 B
subfig = fig.add_subfigure(gs[0, 0], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0, 0, 1.0, 1.0))
ax.set_box_aspect(0.8)

ax.plot(data.iloc[:, 4], data.iloc[:, 7]+0.08, ls='-', color=colors[4], label=r'ref.MnO')
ax.plot(data.iloc[:, 0], data.iloc[:, 3]+0.08, ls='-', color=colors[5], label=r'ref.Mn$\mathrm{_2O_3}$')
ax.plot(data.iloc[:, 8], data.iloc[:, 11]+0.08, ls='-', color=colors[3], label=r'ref.Mn$\mathrm{O_2}$')
ax.legend(loc='upper left', bbox_to_anchor=(0.0, 1.02), ncols=1, frameon=False, fontsize=9, labelcolor='linecolor', columnspacing=0.0)

ax.plot(Mn_III.axes_manager[0].axis, Mn_III.data, ls='-', color='k', label=None)
# ax.plot(fit_Mn_III_3.iloc[:, 0], fit_Mn_III_3.iloc[:, 1], ls='--', color=colors[5], label=None)
# ax.plot(fit_Mn_III_2.iloc[:, 0], fit_Mn_III_2.iloc[:, 1], ls='--', color=colors[5], label=None)
# ax.plot(fit_Mn_III_1.iloc[:, 0], fit_Mn_III_1.iloc[:, 1], ls='--', color=colors[5], label=None)
ax.plot(Mn_III_model.axes_manager[0].axis, Mn_III_model.as_signal().data, ls='--', color=colors[3], label=None)

ax.set_xlim(6535.0, 6550.0)
ax.set_xlabel(r'Energy (eV)', fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=5, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=2.5, offset=0))
ax.set_ylim(-0.01, 0.20)
ax.set_ylabel(r'Relative Intensity (arb.u.)', fontsize=11,)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.05))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.025))

# ax.text(-0.25, 1.0, r'b', transform=ax.transAxes, fontsize=14, va='center', ha='right', fontfamily='Arial', fontweight='bold')

ax.axvline(x=6540.3, ymax=0.65, ls='--')
ax.axvline(x=6543.0, ymax=0.7, ls='--')

plt.savefig(path.joinpath(path_out, r'PreEdge_6_300_V3_0.tif'), transparent=False, pad_inches=0.05, bbox_inches='tight', dpi=300, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'PreEdge_6_600_V3_0.tif'), transparent=False, pad_inches=0.05, bbox_inches='tight', dpi=600, pil_kwargs={"compression": "tiff_lzw"})
plt.show()

### Prisitine alpha MnO2

#### 拟合峰形, IV + III + II (Voigt 峰 - 一个)

In [ ]:
# IV/III 的模型
Mn_IV = data.iloc[:, [8, 11]].copy().dropna(axis=0, how='all')
Mn_III = data.iloc[:, [0, 3]].copy().dropna(axis=0, how='all')

Mn_IV.iloc[115:, 1] = 0
Mn_III.iloc[97:, 1] = 0

# 将 refMn_I//III/IV 做成一个 scale model
from hyperspy.axes import DataAxis
energy_refMn_IV = Mn_IV.iloc[:, 0].to_numpy(dtype='float64')
energy_refMn_IV = DataAxis(axis=energy_refMn_IV, index_in_array=None, name='Energy', units='eV',)
refMn_IV = hs.signals.Signal1D(Mn_IV.iloc[:, 1], axes=[energy_refMn_IV,])
model_refMn_IV = hs.model.components1D.ScalableFixedPattern(refMn_IV) 

energy_refMn_III = Mn_III.iloc[:, 0].to_numpy(dtype='float64')
energy_refMn_III = DataAxis(axis=energy_refMn_III, index_in_array=None, name='Energy', units='eV',)
refMn_III = hs.signals.Signal1D(Mn_III.iloc[:, 1], axes=[energy_refMn_III,])
model_refMn_III = hs.model.components1D.ScalableFixedPattern(refMn_III) 

# 初始化上述的模型
model_refMn_IV.name= r'Mn_IV'
# model_refMn_IV.xscale.value=1.0
# model_refMn_IV.shift.value=0
# model_refMn_IV.set_parameters_not_free(parameter_name_list=['xscale', 'shift',])
model_refMn_IV.yscale.bmin=0.0

model_refMn_III.name= r'Mn_III'
# model_refMn_III.xscale.value=1.0
# model_refMn_III.shift.value=0
# model_refMn_III.set_parameters_not_free(parameter_name_list=['xscale', 'shift',])
model_refMn_III.yscale.bmin=0.0

# II 的模型
pv4 = hs.model.components1D.Voigt(centre=6540.77, sigma=0.62343, gamma=0.55)
pv4.name = r'Mn_II'
pv4.set_parameters_not_free(parameter_name_list=['centre', 'sigma', 'gamma',])
pv4.area.bmin = 0


In [ ]:
Mn_aMnO2 = data.iloc[:, [12, 15]].copy().dropna(axis=0, how='all')

In [ ]:
from hyperspy.axes import DataAxis
energy_aMnO2 = Mn_aMnO2.iloc[:, 0].to_numpy(dtype='float64')
energy_aMnO2 = DataAxis(axis=energy_aMnO2, index_in_array=None, name='Energy', units='eV',)
aMnO2 = hs.signals.Signal1D(Mn_aMnO2.iloc[:, 1], axes=[energy_aMnO2,])
aMnO2 = aMnO2.isig[6535.0:].deepcopy()
aMnO2.plot()

In [ ]:
aMnO2_model = aMnO2.create_model()
aMnO2_model.extend((pv4, model_refMn_III, model_refMn_IV))
aMnO2_model.set_signal_range(6536.0, 6550.0)

In [ ]:
%matplotlib inline
plt.close('all')
aMnO2_model.fit(optimizer="lm", bounded=True, return_info=False) 
aMnO2_model.plot(plot_components=True, plot_residual=False)
plt.savefig(path.joinpath(path_out, r'PreEdge_7_300_V3_0.tif'), pad_inches=0.05, bbox_inches='tight', dpi=300, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'PreEdge_7_600_V3_0.tif'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
aMnO2_model.print_current_values()

In [ ]:
# 转化成 DataFrame
fit_Mn_IV = pd.DataFrame([aMnO2_model.axes_manager[0].axis, aMnO2_model.as_signal(component_list=['Mn_IV']).data])
fit_Mn_IV = fit_Mn_IV.T
fit_Mn_IV.dropna(how='any', inplace=True)

fit_Mn_III = pd.DataFrame([aMnO2_model.axes_manager[0].axis, aMnO2_model.as_signal(component_list=['Mn_III']).data])
fit_Mn_III = fit_Mn_III.T
fit_Mn_III.dropna(how='any', inplace=True)

fit_Mn_II = pd.DataFrame([aMnO2_model.axes_manager[0].axis, aMnO2_model.as_signal(component_list=['Mn_II']).data])
fit_Mn_II = fit_Mn_II.T
fit_Mn_II.dropna(how='any', inplace=True)

# 计算比例
pre_edge_ratio_Mn_IV = scipy.integrate.simpson(y=fit_Mn_IV.iloc[:, 1], x=fit_Mn_IV.iloc[:, 0])/(scipy.integrate.simpson(y=fit_Mn_IV.iloc[:, 1], x=fit_Mn_IV.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_III.iloc[:, 1], x=fit_Mn_III.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_II.iloc[:, 1], x=fit_Mn_II.iloc[:, 0]))
pre_edge_ratio_Mn_III = scipy.integrate.simpson(y=fit_Mn_III.iloc[:, 1], x=fit_Mn_III.iloc[:, 0])/(scipy.integrate.simpson(y=fit_Mn_IV.iloc[:, 1], x=fit_Mn_IV.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_III.iloc[:, 1], x=fit_Mn_III.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_II.iloc[:, 1], x=fit_Mn_II.iloc[:, 0]))
pre_edge_ratio_Mn_II = scipy.integrate.simpson(y=fit_Mn_II.iloc[:, 1], x=fit_Mn_II.iloc[:, 0])/(scipy.integrate.simpson(y=fit_Mn_IV.iloc[:, 1], x=fit_Mn_IV.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_III.iloc[:, 1], x=fit_Mn_III.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_II.iloc[:, 1], x=fit_Mn_II.iloc[:, 0]))
pre_edge_ratio_Mn_IV, pre_edge_ratio_Mn_III, pre_edge_ratio_Mn_II

In [ ]:
Pre_Edge_aMnO2 = xr.Dataset(
    data_vars=dict(
        aMnO2 = (['Energy_aMnO2'], aMnO2.data),
        fit = (['Energy_fit'], aMnO2_model.as_signal().data),
        fit_Mn_IV = (['Energy_fit'], aMnO2_model.as_signal(component_list=['Mn_IV']).data),
        fit_Mn_III = (['Energy_fit'], aMnO2_model.as_signal(component_list=['Mn_III']).data),
        fit_Mn_II = (['Energy_fit'], aMnO2_model.as_signal(component_list=['Mn_II']).data),
        ),
    coords=dict(
        Energy_aMnO2 = aMnO2.axes_manager[0].axis,
        Energy_fit= aMnO2_model.axes_manager[0].axis,
    ),
    attrs=dict(
        ratio_Mn_IV = pre_edge_ratio_Mn_IV,
        ratio_Mn_III = pre_edge_ratio_Mn_III,
        ratio_Mn_II = pre_edge_ratio_Mn_II,
        chisq = aMnO2_model.chisq.data[0],
        red_chisq = aMnO2_model.red_chisq.data[0],
    ),
)

aMnO2_model.save_parameters2file(path.joinpath(path_out, r'Prisitne_Mn_4_model_parameters_ref_2_voigt_1'))
aMnO2_model.save(path.joinpath(path_out, r'Prisitne_Mn_4_model_ref_2_voigt_1'), 'model', overwrite=True)
Pre_Edge_aMnO2.to_netcdf(path.joinpath(path_out, r'Prisitne_Mn_4_result_ref_2_voigt_1.NETCDF4'), engine="h5netcdf")

In [ ]:
# 画图
%matplotlib inline
fig = plt.figure(figsize=(7, 2.5))
gs = gridspec.GridSpec(1, 2, width_ratios=[1, 1], height_ratios=None, wspace=0.0, hspace=0.0, figure=fig)

xas_colors_Mn = [colors[0], colors[4]]
xas_cmap_Mn = ListedColormap(mpl.colormaps['coolwarm'](np.linspace(0.5, 1.0, data.shape[1]//4-5)), name='xas_cmap_Mn', )
labels_Mn = [r'${\alpha}$-MnO$\mathrm{_2}$', r'0.2M Mn$\mathrm{^{2\!+}}$']

# 图 B
subfig = fig.add_subfigure(gs[0, 0], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0, 0, 1.0, 1.0))
ax.set_box_aspect(0.8)

ax.plot(data.iloc[:, 4], data.iloc[:, 7]+0.08, ls='-', color=colors[4], label=r'ref.MnO')
ax.plot(data.iloc[:, 0], data.iloc[:, 3]+0.08, ls='-', color=colors[5], label=r'ref.Mn$\mathrm{_2O_3}$')
ax.plot(data.iloc[:, 8], data.iloc[:, 11]+0.08, ls='-', color=colors[3], label=r'ref.Mn$\mathrm{O_2}$')
ax.legend(loc='upper left', bbox_to_anchor=(0.0, 1.02), ncols=1, frameon=False, fontsize=9, labelcolor='linecolor', columnspacing=0.0)

ax.plot(aMnO2.axes_manager[0].axis, aMnO2.data, ls='-', color='k', label=None)
ax.plot(fit_Mn_II.iloc[:, 0], fit_Mn_II.iloc[:, 1], ls='--', color=colors[4], label=None)
ax.plot(fit_Mn_III.iloc[:, 0], fit_Mn_III.iloc[:, 1], ls='--', color=colors[5], label=None)
ax.plot(fit_Mn_IV.iloc[:, 0], fit_Mn_IV.iloc[:, 1], ls='--', color=colors[3], label=None)
ax.plot(aMnO2_model.axes_manager[0].axis, aMnO2_model.as_signal().data, ls='--', color=colors[3], label=None)

ax.set_xlim(6535.0, 6550.0)
ax.set_xlabel(r'Energy (eV)', fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=5, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=2.5, offset=0))
ax.set_ylim(-0.01, 0.20)
ax.set_ylabel(r'Relative Intensity (arb.u.)', fontsize=11,)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.05))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.025))

# ax.text(-0.25, 1.0, r'b', transform=ax.transAxes, fontsize=14, va='center', ha='right', fontfamily='Arial', fontweight='bold')

ax.axvline(x=6540.3, ymax=0.65, ls='--')
ax.axvline(x=6543.0, ymax=0.7, ls='--')

plt.savefig(path.joinpath(path_out, r'PreEdge_8_300_V3_0.tif'), transparent=False, pad_inches=0.05, bbox_inches='tight', dpi=300, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'PreEdge_8_600_V3_0.tif'), transparent=False, pad_inches=0.05, bbox_inches='tight', dpi=600, pil_kwargs={"compression": "tiff_lzw"})
plt.show()

#### 拟合峰形, IV + III

In [ ]:
# 数据清洗
Mn_IV = data.iloc[:, [8, 11]].copy().dropna(axis=0, how='all')
Mn_III = data.iloc[:, [0, 3]].copy().dropna(axis=0, how='all')

Mn_IV.iloc[115:, 1] = 0
Mn_III.iloc[97:, 1] = 0

# 将 refMn_I//III/IV 做成一个 scale model
from hyperspy.axes import DataAxis
energy_refMn_IV = Mn_IV.iloc[:, 0].to_numpy(dtype='float64')
energy_refMn_IV = DataAxis(axis=energy_refMn_IV, index_in_array=None, name='Energy', units='eV',)
refMn_IV = hs.signals.Signal1D(Mn_IV.iloc[:, 1], axes=[energy_refMn_IV,])
model_refMn_IV = hs.model.components1D.ScalableFixedPattern(refMn_IV) 

energy_refMn_III = Mn_III.iloc[:, 0].to_numpy(dtype='float64')
energy_refMn_III = DataAxis(axis=energy_refMn_III, index_in_array=None, name='Energy', units='eV',)
refMn_III = hs.signals.Signal1D(Mn_III.iloc[:, 1], axes=[energy_refMn_III,])
model_refMn_III = hs.model.components1D.ScalableFixedPattern(refMn_III) 

# 初始化上述的模型
model_refMn_IV.name= r'Mn_IV'
# model_refMn_IV.xscale.value=1.0
# model_refMn_IV.shift.value=0
# model_refMn_IV.set_parameters_not_free(parameter_name_list=['xscale', 'shift',])
model_refMn_IV.yscale.bmin=0.0

model_refMn_III.name= r'Mn_III'
# model_refMn_III.xscale.value=1.0
# model_refMn_III.shift.value=0
# model_refMn_III.set_parameters_not_free(parameter_name_list=['xscale', 'shift',])
model_refMn_III.yscale.bmin=0.0

In [ ]:
Mn_aMnO2 = data.iloc[:, [12, 15]].copy().dropna(axis=0, how='all')

In [ ]:
from hyperspy.axes import DataAxis
energy_aMnO2 = Mn_aMnO2.iloc[:, 0].to_numpy(dtype='float64')
energy_aMnO2 = DataAxis(axis=energy_aMnO2, index_in_array=None, name='Energy', units='eV',)
aMnO2 = hs.signals.Signal1D(Mn_aMnO2.iloc[:, 1], axes=[energy_aMnO2,])
aMnO2 = aMnO2.isig[6535.0:].deepcopy()
aMnO2.plot()

In [ ]:
aMnO2_model = aMnO2.create_model()
aMnO2_model.extend((model_refMn_III, model_refMn_IV))
aMnO2_model.set_signal_range(6536.0, 6550.0)

In [ ]:
%matplotlib inline
plt.close('all')
aMnO2_model.fit(optimizer="lm", bounded=True, return_info=True) 
aMnO2_model.plot(plot_components=True, plot_residual=False)
plt.savefig(path.joinpath(path_out, r'PreEdge_9_300_V3_0.tif'), pad_inches=0.05, bbox_inches='tight', dpi=300, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'PreEdge_9_600_V3_0.tif'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
aMnO2_model.print_current_values()

In [ ]:
# 转化成 DataFrame
fit_Mn_IV = pd.DataFrame([aMnO2_model.axes_manager[0].axis, aMnO2_model.as_signal(component_list=['Mn_IV']).data])
fit_Mn_IV = fit_Mn_IV.T
fit_Mn_IV.dropna(how='any', inplace=True)

fit_Mn_III = pd.DataFrame([aMnO2_model.axes_manager[0].axis, aMnO2_model.as_signal(component_list=['Mn_III']).data])
fit_Mn_III = fit_Mn_III.T
fit_Mn_III.dropna(how='any', inplace=True)

# 计算比例
pre_edge_ratio_Mn_IV = scipy.integrate.simpson(y=fit_Mn_IV.iloc[:, 1], x=fit_Mn_IV.iloc[:, 0])/(scipy.integrate.simpson(y=fit_Mn_IV.iloc[:, 1], x=fit_Mn_IV.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_III.iloc[:, 1], x=fit_Mn_III.iloc[:, 0]))
pre_edge_ratio_Mn_III = scipy.integrate.simpson(y=fit_Mn_III.iloc[:, 1], x=fit_Mn_III.iloc[:, 0])/(scipy.integrate.simpson(y=fit_Mn_IV.iloc[:, 1], x=fit_Mn_IV.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_III.iloc[:, 1], x=fit_Mn_III.iloc[:, 0]))
pre_edge_ratio_Mn_IV, pre_edge_ratio_Mn_III

In [ ]:
Pre_Edge_aMnO2 = xr.Dataset(
    data_vars=dict(
        aMnO2 = (['Energy_aMnO2'], aMnO2.data),
        fit = (['Energy_fit'], aMnO2_model.as_signal().data),
        fit_Mn_IV = (['Energy_fit'], aMnO2_model.as_signal(component_list=['Mn_IV']).data),
        fit_Mn_III = (['Energy_fit'], aMnO2_model.as_signal(component_list=['Mn_III']).data),
        ),
    coords=dict(
        Energy_aMnO2 = aMnO2.axes_manager[0].axis,
        Energy_fit= aMnO2_model.axes_manager[0].axis,
    ),
    attrs=dict(
        ratio_Mn_IV = pre_edge_ratio_Mn_IV,
        ratio_Mn_III = pre_edge_ratio_Mn_III,
        chisq = aMnO2_model.chisq.data[0],
        red_chisq = aMnO2_model.red_chisq.data[0],
    ),
)

aMnO2_model.save_parameters2file(path.joinpath(path_out, r'Prisitne_Mn_4_model_parameters_ref_2'))
aMnO2_model.save(path.joinpath(path_out, r'Prisitne_Mn_4_model_ref_2'), 'model', overwrite=True)
Pre_Edge_aMnO2.to_netcdf(path.joinpath(path_out, r'Prisitne_Mn_4_result_ref_2.NETCDF4'), engine="h5netcdf")

In [ ]:
# 画图
%matplotlib inline
fig = plt.figure(figsize=(7, 2.5))
gs = gridspec.GridSpec(1, 2, width_ratios=[1, 1], height_ratios=None, wspace=0.0, hspace=0.0, figure=fig)

xas_colors_Mn = [colors[0], colors[4]]
xas_cmap_Mn = ListedColormap(mpl.colormaps['coolwarm'](np.linspace(0.5, 1.0, data.shape[1]//4-5)), name='xas_cmap_Mn', )
labels_Mn = [r'${\alpha}$-MnO$\mathrm{_2}$', r'0.2M Mn$\mathrm{^{2\!+}}$']

# 图 B
subfig = fig.add_subfigure(gs[0, 0], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0, 0, 1.0, 1.0))
ax.set_box_aspect(0.8)

ax.plot(data.iloc[:, 4], data.iloc[:, 7]+0.08, ls='-', color=colors[4], label=r'ref.MnO')
ax.plot(data.iloc[:, 0], data.iloc[:, 3]+0.08, ls='-', color=colors[5], label=r'ref.Mn$\mathrm{_2O_3}$')
ax.plot(data.iloc[:, 8], data.iloc[:, 11]+0.08, ls='-', color=colors[3], label=r'ref.Mn$\mathrm{O_2}$')
ax.legend(loc='upper left', bbox_to_anchor=(0.0, 1.02), ncols=1, frameon=False, fontsize=9, labelcolor='linecolor', columnspacing=0.0)

ax.plot(aMnO2.axes_manager[0].axis, aMnO2.data, ls='-', color='k', label=None)
ax.plot(fit_Mn_III.iloc[:, 0], fit_Mn_III.iloc[:, 1], ls='--', color=colors[5], label=None)
ax.plot(fit_Mn_IV.iloc[:, 0], fit_Mn_IV.iloc[:, 1], ls='--', color=colors[3], label=None)
ax.plot(aMnO2_model.axes_manager[0].axis, aMnO2_model.as_signal().data, ls='--', color=colors[3], label=None)

ax.set_xlim(6535.0, 6550.0)
ax.set_xlabel(r'Energy (eV)', fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=5, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=2.5, offset=0))
ax.set_ylim(-0.01, 0.20)
ax.set_ylabel(r'Relative Intensity (arb.u.)', fontsize=11,)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.05))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.025))

# ax.text(-0.25, 1.0, r'b', transform=ax.transAxes, fontsize=14, va='center', ha='right', fontfamily='Arial', fontweight='bold')

ax.axvline(x=6540.3, ymax=0.65, ls='--')
ax.axvline(x=6543.0, ymax=0.7, ls='--')

plt.savefig(path.joinpath(path_out, r'PreEdge_10_300_V3_0.tif'), transparent=False, pad_inches=0.05, bbox_inches='tight', dpi=300, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'PreEdge_10_600_V3_0.tif'), transparent=False, pad_inches=0.05, bbox_inches='tight', dpi=600, pil_kwargs={"compression": "tiff_lzw"})
plt.show()

#### 拟合峰形, IV + III (Voigt 峰 3 个) + II (Voigt 峰 1 个)

In [ ]:
# 数据清洗
Mn_IV = data.iloc[:, [8, 11]].copy().dropna(axis=0, how='all')
Mn_IV.iloc[115:, 1] = 0

# 将 refMn_III/IV 做成一个 scale model
from hyperspy.axes import DataAxis
energy_refMn_IV = Mn_IV.iloc[:, 0].to_numpy(dtype='float64')
energy_refMn_IV = DataAxis(axis=energy_refMn_IV, index_in_array=None, name='Energy', units='eV',)
refMn_IV = hs.signals.Signal1D(Mn_IV.iloc[:, 1], axes=[energy_refMn_IV,])
model_refMn_IV = hs.model.components1D.ScalableFixedPattern(refMn_IV) 

# 初始化上述的模型
model_refMn_IV.name= r'Mn_IV'
# model_refMn_IV.xscale.value=1.0
# model_refMn_IV.shift.value=0
# model_refMn_IV.set_parameters_not_free(parameter_name_list=['xscale', 'shift',])
model_refMn_IV.yscale.bmin=0.0

# III 的模型
pv1 = hs.model.components1D.Voigt(centre=6541.14, sigma=0.62343, gamma=0.55, module='scipy')
pv1.name = r'Mn_III_1'
pv1.set_parameters_not_free(parameter_name_list=['centre', 'sigma', 'gamma',])
pv1.centre.bmin = 6538.0
pv1.centre.bmax = 6542.5
pv1.area.bmin = 0

pv2 = hs.model.components1D.Voigt(centre=6544.73, sigma=0.62343, gamma=0.55, module='scipy')
pv2.set_parameters_not_free(parameter_name_list=['centre',])
pv2.name = r'Mn_III_2'
pv2.centre.bmin = 6543.0
pv2.centre.bmax = 6545.0
pv2.area.bmin = 0
pv2.sigma.twin = pv1.sigma
pv2.gamma.twin = pv1.gamma
pv2.area.twin_function_expr = '0.74805*x'
pv2.area.twin = pv1.area

pv3 = hs.model.components1D.Voigt(centre=6543.11, sigma=0.62343, gamma=0.55, module='scipy')
pv3.set_parameters_not_free(parameter_name_list=['centre',])
pv3.name = r'Mn_III_3'
pv3.centre.bmin = pv1.centre.bmax - 0.5
pv3.centre.bmax = pv2.centre.bmin + 0.5
pv3.area.bmin = 0
pv3.sigma.twin = pv1.sigma
pv3.gamma.twin = pv1.gamma
pv3.area.twin_function_expr = '0.867960*x'
pv3.area.twin = pv1.area

# II 的模型
pv4 = hs.model.components1D.Voigt(centre=6540.77, sigma=0.62343, gamma=0.55, module='scipy')
pv4.name = r'Mn_II'
pv4.set_parameters_not_free(parameter_name_list=['centre', 'sigma', 'gamma',])
pv4.centre.bmin = 6540.0
pv4.centre.bmax = 6541.5
pv4.area.bmin = 0


In [ ]:
Mn_aMnO2 = data.iloc[:, [12, 15]].copy().dropna(axis=0, how='all')

In [ ]:
from hyperspy.axes import DataAxis
energy_aMnO2 = Mn_aMnO2.iloc[:, 0].to_numpy(dtype='float64')
energy_aMnO2 = DataAxis(axis=energy_aMnO2, index_in_array=None, name='Energy', units='eV',)
aMnO2 = hs.signals.Signal1D(Mn_aMnO2.iloc[:, 1], axes=[energy_aMnO2,])
aMnO2 = aMnO2.isig[6535.0:].deepcopy()
aMnO2.plot()

In [ ]:
aMnO2_model = aMnO2.create_model()
aMnO2_model.extend((model_refMn_IV, pv1, pv2, pv3, pv4))
aMnO2_model.set_signal_range(6536.0, 6550.0)

In [ ]:
%matplotlib inline
plt.close('all')
aMnO2_model.fit(optimizer="lm", bounded=True, return_info=False) 
aMnO2_model.plot(plot_components=True, plot_residual=True)
plt.savefig(path.joinpath(path_out, r'PreEdge_11_300_V3_0.tif'), pad_inches=0.05, bbox_inches='tight', dpi=300, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'PreEdge_11_600_V3_0.tif'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
aMnO2_model.print_current_values()

In [ ]:
# 转化成 DataFrame
fit_Mn_IV = pd.DataFrame([aMnO2_model.axes_manager[0].axis, aMnO2_model.as_signal(component_list=['Mn_IV']).data])
fit_Mn_IV = fit_Mn_IV.T
fit_Mn_IV.dropna(how='any', inplace=True)

fit_Mn_III_1 = pd.DataFrame([aMnO2_model.axes_manager[0].axis, aMnO2_model.as_signal(component_list=['Mn_III_1']).data])
fit_Mn_III_1 = fit_Mn_III_1.T
fit_Mn_III_1.dropna(how='any', inplace=True)

fit_Mn_III_2 = pd.DataFrame([aMnO2_model.axes_manager[0].axis, aMnO2_model.as_signal(component_list=['Mn_III_2']).data])
fit_Mn_III_2 = fit_Mn_III_2.T
fit_Mn_III_2.dropna(how='any', inplace=True)

fit_Mn_III_3 = pd.DataFrame([aMnO2_model.axes_manager[0].axis, aMnO2_model.as_signal(component_list=['Mn_III_3']).data])
fit_Mn_III_3 = fit_Mn_III_3.T
fit_Mn_III_3.dropna(how='any', inplace=True)

fit_Mn_II = pd.DataFrame([aMnO2_model.axes_manager[0].axis, aMnO2_model.as_signal(component_list=['Mn_II']).data])
fit_Mn_II = fit_Mn_II.T
fit_Mn_II.dropna(how='any', inplace=True)

# 计算比例
pre_edge_ratio_Mn_IV = scipy.integrate.simpson(y=fit_Mn_IV.iloc[:, 1], x=fit_Mn_IV.iloc[:, 0])/(scipy.integrate.simpson(y=fit_Mn_IV.iloc[:, 1], x=fit_Mn_IV.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_III_1.iloc[:, 1], x=fit_Mn_III_1.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_III_2.iloc[:, 1], x=fit_Mn_III_2.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_III_3.iloc[:, 1], x=fit_Mn_III_3.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_II.iloc[:, 1], x=fit_Mn_II.iloc[:, 0]))
pre_edge_ratio_Mn_III = (scipy.integrate.simpson(y=fit_Mn_III_1.iloc[:, 1], x=fit_Mn_III_1.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_III_2.iloc[:, 1], x=fit_Mn_III_2.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_III_3.iloc[:, 1], x=fit_Mn_III_3.iloc[:, 0]))/(scipy.integrate.simpson(y=fit_Mn_IV.iloc[:, 1], x=fit_Mn_IV.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_III_1.iloc[:, 1], x=fit_Mn_III_1.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_III_2.iloc[:, 1], x=fit_Mn_III_2.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_III_3.iloc[:, 1], x=fit_Mn_III_3.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_II.iloc[:, 1], x=fit_Mn_II.iloc[:, 0]))
pre_edge_ratio_Mn_II = scipy.integrate.simpson(y=fit_Mn_II.iloc[:, 1], x=fit_Mn_II.iloc[:, 0])/(scipy.integrate.simpson(y=fit_Mn_IV.iloc[:, 1], x=fit_Mn_IV.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_III_1.iloc[:, 1], x=fit_Mn_III_1.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_III_2.iloc[:, 1], x=fit_Mn_III_2.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_III_3.iloc[:, 1], x=fit_Mn_III_3.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_II.iloc[:, 1], x=fit_Mn_II.iloc[:, 0]))

pre_edge_ratio_Mn_IV, pre_edge_ratio_Mn_III, pre_edge_ratio_Mn_II

In [ ]:
Pre_Edge_aMnO2 = xr.Dataset(
    data_vars=dict(
        aMnO2 = (['Energy_aMnO2'], aMnO2.data),
        fit = (['Energy_fit'], aMnO2_model.as_signal().data),
        fit_Mn_IV = (['Energy_fit'], aMnO2_model.as_signal(component_list=['Mn_IV']).data),
        fit_Mn_III_1 = (['Energy_fit'], aMnO2_model.as_signal(component_list=['Mn_III_1']).data),
        fit_Mn_III_2 = (['Energy_fit'], aMnO2_model.as_signal(component_list=['Mn_III_2']).data),
        fit_Mn_III_3 = (['Energy_fit'], aMnO2_model.as_signal(component_list=['Mn_III_3']).data),
        fit_Mn_II = (['Energy_fit'], aMnO2_model.as_signal(component_list=['Mn_II']).data),
        ),
    coords=dict(
        Energy_aMnO2 = aMnO2.axes_manager[0].axis,
        Energy_fit= aMnO2_model.axes_manager[0].axis,
    ),
    attrs=dict(
        ratio_Mn_IV = pre_edge_ratio_Mn_IV,
        ratio_Mn_III = pre_edge_ratio_Mn_III,
        ratio_Mn_II = pre_edge_ratio_Mn_II,
        chisq = aMnO2_model.chisq.data[0],
        red_chisq = aMnO2_model.red_chisq.data[0],
    ),
)

aMnO2_model.save_parameters2file(path.joinpath(path_out, r'Prisitne_Mn_4_model_parameters_ref_1_voigt_3_voigt_1'))
aMnO2_model.save(path.joinpath(path_out, r'Prisitne_Mn_4_model_ref_1_voigt_3_voigt_1'), 'model', overwrite=True)
Pre_Edge_aMnO2.to_netcdf(path.joinpath(path_out, r'Prisitne_Mn_4_result_ref_1_voigt_3_voigt_1.NETCDF4'), engine="h5netcdf")

In [ ]:
# 画图
%matplotlib inline
fig = plt.figure(figsize=(7, 2.5))
gs = gridspec.GridSpec(1, 2, width_ratios=[1, 1], height_ratios=None, wspace=0.0, hspace=0.0, figure=fig)

xas_colors_Mn = [colors[0], colors[4]]
xas_cmap_Mn = ListedColormap(mpl.colormaps['coolwarm'](np.linspace(0.5, 1.0, data.shape[1]//4-5)), name='xas_cmap_Mn', )
labels_Mn = [r'${\alpha}$-MnO$\mathrm{_2}$', r'0.2M Mn$\mathrm{^{2\!+}}$']

# 图 B
subfig = fig.add_subfigure(gs[0, 0], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0, 0, 1.0, 1.0))
ax.set_box_aspect(0.8)

ax.plot(data.iloc[:, 4], data.iloc[:, 7]+0.08, ls='-', color=colors[4], label=r'ref.MnO')
ax.plot(data.iloc[:, 0], data.iloc[:, 3]+0.08, ls='-', color=colors[5], label=r'ref.Mn$\mathrm{_2O_3}$')
ax.plot(data.iloc[:, 8], data.iloc[:, 11]+0.08, ls='-', color=colors[3], label=r'ref.Mn$\mathrm{O_2}$')
ax.legend(loc='upper left', bbox_to_anchor=(0.0, 1.02), ncols=1, frameon=False, fontsize=9, labelcolor='linecolor', columnspacing=0.0)

ax.plot(aMnO2.axes_manager[0].axis, aMnO2.data, ls='-', color='k', label=None)
# ax.plot(fit_Mn_III_3.iloc[:, 0], fit_Mn_III_3.iloc[:, 1], ls='--', color=colors[5], label=None)
# ax.plot(fit_Mn_III_2.iloc[:, 0], fit_Mn_III_2.iloc[:, 1], ls='--', color=colors[5], label=None)
# ax.plot(fit_Mn_III_1.iloc[:, 0], fit_Mn_III_1.iloc[:, 1], ls='--', color=colors[5], label=None)
ax.plot(fit_Mn_III_1.iloc[:, 0], fit_Mn_III_1.iloc[:, 1] + fit_Mn_III_2.iloc[:, 1] + fit_Mn_III_3.iloc[:, 1], ls='--', color=colors[5], label=None)
ax.plot(fit_Mn_IV.iloc[:, 0], fit_Mn_IV.iloc[:, 1], ls='--', color=colors[3], label=None)
ax.plot(fit_Mn_II.iloc[:, 0], fit_Mn_II.iloc[:, 1], ls='--', color=colors[1], label=None)
ax.plot(aMnO2_model.axes_manager[0].axis, aMnO2_model.as_signal().data, ls='--', color=colors[3], label=None)

ax.set_xlim(6535.0, 6550.0)
ax.set_xlabel(r'Energy (eV)', fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=5, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=2.5, offset=0))
ax.set_ylim(-0.01, 0.20)
ax.set_ylabel(r'Relative Intensity (arb.u.)', fontsize=11,)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.05))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.025))

# ax.text(-0.25, 1.0, r'b', transform=ax.transAxes, fontsize=14, va='center', ha='right', fontfamily='Arial', fontweight='bold')

ax.axvline(x=6540.3, ymax=0.65, ls='--')
ax.axvline(x=6543.0, ymax=0.7, ls='--')

plt.savefig(path.joinpath(path_out, r'PreEdge_12_300_V3_0.tif'), transparent=False, pad_inches=0.05, bbox_inches='tight', dpi=300, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'PreEdge_12_600_V3_0.tif'), transparent=False, pad_inches=0.05, bbox_inches='tight', dpi=600, pil_kwargs={"compression": "tiff_lzw"})
plt.show()

### OCV op

#### 拟合峰形, IV + III + II (pseudo-Voigt 峰 - 一个)

In [ ]:
# IV/III 的模型
Mn_IV = data.iloc[:, [8, 11]].copy().dropna(axis=0, how='all')
Mn_III = data.iloc[:, [0, 3]].copy().dropna(axis=0, how='all')

Mn_IV.iloc[115:, 1] = 0
Mn_III.iloc[97:, 1] = 0

# 将 refMn_I//III/IV 做成一个 scale model
from hyperspy.axes import DataAxis
energy_refMn_IV = Mn_IV.iloc[:, 0].to_numpy(dtype='float64')
energy_refMn_IV = DataAxis(axis=energy_refMn_IV, index_in_array=None, name='Energy', units='eV',)
refMn_IV = hs.signals.Signal1D(Mn_IV.iloc[:, 1], axes=[energy_refMn_IV,])
model_refMn_IV = hs.model.components1D.ScalableFixedPattern(refMn_IV) 

energy_refMn_III = Mn_III.iloc[:, 0].to_numpy(dtype='float64')
energy_refMn_III = DataAxis(axis=energy_refMn_III, index_in_array=None, name='Energy', units='eV',)
refMn_III = hs.signals.Signal1D(Mn_III.iloc[:, 1], axes=[energy_refMn_III,])
model_refMn_III = hs.model.components1D.ScalableFixedPattern(refMn_III) 

# 初始化上述的模型
model_refMn_IV.name= r'Mn_IV'
# model_refMn_IV.xscale.value=1.03564
# model_refMn_IV.shift.value=232.968
# model_refMn_IV.set_parameters_not_free(parameter_name_list=['xscale', 'shift',])
model_refMn_IV.yscale.bmin=0.0

model_refMn_III.name= r'Mn_III'
# model_refMn_III.xscale.value=1.13677
# model_refMn_III.shift.value=894.306
# model_refMn_III.set_parameters_not_free(parameter_name_list=['xscale', 'shift',])
model_refMn_III.yscale.bmin=0.0

# II 的模型
pv4 = hs.model.components1D.Voigt(centre=6540.77, sigma=0.62343, gamma=0.55, module='scipy')
pv4.name = r'Mn_II'
pv4.set_parameters_not_free(parameter_name_list=['centre', 'sigma', 'gamma',])
pv4.centre.bmin = 6540.0
pv4.centre.bmax = 6541.5
pv4.area.bmin = 0

In [ ]:
Mn_OCV = data.iloc[:, [24, 27]].copy().dropna(axis=0, how='all')

In [ ]:
from hyperspy.axes import DataAxis
energy_OCV = Mn_OCV.iloc[:, 0].to_numpy(dtype='float64')
energy_OCV = DataAxis(axis=energy_OCV, index_in_array=None, name='Energy', units='eV',)
OCV = hs.signals.Signal1D(Mn_OCV.iloc[:, 1], axes=[energy_OCV,])
OCV = OCV.isig[6532.0:].deepcopy()
OCV.plot()

In [ ]:
OCV_model = OCV.create_model()
OCV_model.extend((model_refMn_IV, model_refMn_III, pv4))
OCV_model.set_signal_range(6536.0, 6550.0)

In [ ]:
%matplotlib widget
plt.close('all')
OCV_model.fit(optimizer="lm", bounded=True, return_info=False) 
OCV_model.plot(plot_components=True, plot_residual=False)
plt.savefig(path.joinpath(path_out, r'PreEdge_13_300_V3_0.tif'), pad_inches=0.05, bbox_inches='tight', dpi=300, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'PreEdge_13_600_V3_0.tif'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
OCV_model.print_current_values()

In [ ]:
# 转化成 DataFrame
fit_Mn_IV = pd.DataFrame([OCV_model.axes_manager[0].axis, OCV_model.as_signal(component_list=['Mn_IV']).data])
fit_Mn_IV = fit_Mn_IV.T
fit_Mn_IV.dropna(how='any', inplace=True)

fit_Mn_III = pd.DataFrame([OCV_model.axes_manager[0].axis, OCV_model.as_signal(component_list=['Mn_III']).data])
fit_Mn_III = fit_Mn_III.T
fit_Mn_III.dropna(how='any', inplace=True)

fit_Mn_II = pd.DataFrame([OCV_model.axes_manager[0].axis, OCV_model.as_signal(component_list=['Mn_II']).data])
fit_Mn_II = fit_Mn_II.T
fit_Mn_II.dropna(how='any', inplace=True)

# 计算比例
pre_edge_ratio_Mn_IV = scipy.integrate.simpson(y=fit_Mn_IV.iloc[:, 1], x=fit_Mn_IV.iloc[:, 0])/(scipy.integrate.simpson(y=fit_Mn_IV.iloc[:, 1], x=fit_Mn_IV.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_III.iloc[:, 1], x=fit_Mn_III.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_II.iloc[:, 1], x=fit_Mn_II.iloc[:, 0]))
pre_edge_ratio_Mn_III = (scipy.integrate.simpson(y=fit_Mn_III.iloc[:, 1], x=fit_Mn_III.iloc[:, 0]))/(scipy.integrate.simpson(y=fit_Mn_IV.iloc[:, 1], x=fit_Mn_IV.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_III.iloc[:, 1], x=fit_Mn_III.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_II.iloc[:, 1], x=fit_Mn_II.iloc[:, 0]))
pre_edge_ratio_Mn_II = scipy.integrate.simpson(y=fit_Mn_II.iloc[:, 1], x=fit_Mn_II.iloc[:, 0])/(scipy.integrate.simpson(y=fit_Mn_IV.iloc[:, 1], x=fit_Mn_IV.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_III.iloc[:, 1], x=fit_Mn_III.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_II.iloc[:, 1], x=fit_Mn_II.iloc[:, 0]))
pre_edge_ratio_Mn_IV, pre_edge_ratio_Mn_III, pre_edge_ratio_Mn_II

In [ ]:
Pre_Edge_OCV = xr.Dataset(
    data_vars=dict(
        OCV = (['Energy_OCV'], OCV.data),
        fit = (['Energy_fit'], OCV_model.as_signal().data),
        fit_Mn_IV = (['Energy_fit'], OCV_model.as_signal(component_list=['Mn_IV']).data),
        fit_Mn_III = (['Energy_fit'], OCV_model.as_signal(component_list=['Mn_III']).data),
        fit_Mn_II = (['Energy_fit'], OCV_model.as_signal(component_list=['Mn_II']).data),
        ),
    coords=dict(
        Energy_OCV = OCV.axes_manager[0].axis,
        Energy_fit= OCV_model.axes_manager[0].axis,
    ),
    attrs=dict(
        ratio_Mn_IV = pre_edge_ratio_Mn_IV,
        ratio_Mn_III = pre_edge_ratio_Mn_III,
        ratio_Mn_II = pre_edge_ratio_Mn_II,
        chisq = OCV_model.chisq.data[0],
        red_chisq = OCV_model.red_chisq.data[0],
    ),
)

OCV_model.save_parameters2file(path.joinpath(path_out, r'OCV_model_parameters_ref_2_voigt_1'))
OCV_model.save(path.joinpath(path_out, r'OCV_model_ref_2_voigt_1'), 'model', overwrite=True)
Pre_Edge_OCV.to_netcdf(path.joinpath(path_out, r'OCV_result_ref_2_voigt_1.NETCDF4'), engine="h5netcdf")

In [ ]:
# 画图
%matplotlib inline
plt.close('all')
fig = plt.figure(figsize=(7, 2.5))
gs = gridspec.GridSpec(1, 2, width_ratios=[1, 1], height_ratios=None, wspace=0.0, hspace=0.0, figure=fig)

xas_colors_Mn = [colors[0], colors[4]]
xas_cmap_Mn = ListedColormap(mpl.colormaps['coolwarm'](np.linspace(0.5, 1.0, data.shape[1]//4-5)), name='xas_cmap_Mn', )
labels_Mn = [r'${\alpha}$-MnO$\mathrm{_2}$', r'0.2M Mn$\mathrm{^{2\!+}}$']

# 图 B
subfig = fig.add_subfigure(gs[0, 0], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0, 0, 1.0, 1.0))
ax.set_box_aspect(0.8)
pad = 0.05
ax.plot(data.iloc[:, 4], data.iloc[:, 7]+pad, ls='-', color=colors[4], label=r'ref.MnO')
ax.plot(data.iloc[:, 0], data.iloc[:, 3]+pad, ls='-', color=colors[5], label=r'ref.Mn$\mathrm{_2O_3}$')
ax.plot(data.iloc[:, 8], data.iloc[:, 11]+pad, ls='-', color=colors[3], label=r'ref.Mn$\mathrm{O_2}$')
ax.legend(loc='upper left', bbox_to_anchor=(0.0, 1.02), ncols=1, frameon=False, fontsize=9, labelcolor='linecolor', columnspacing=0.0)

ax.plot(OCV.axes_manager[0].axis, OCV.data, ls='-', color='k', label=None)
ax.plot(fit_Mn_II.iloc[:, 0], fit_Mn_II.iloc[:, 1], ls='--', color=colors[4], label=None)
ax.plot(fit_Mn_IV.iloc[:, 0], fit_Mn_IV.iloc[:, 1], ls='--', color=colors[3], label=None)
ax.plot(fit_Mn_III.iloc[:, 0], fit_Mn_III.iloc[:, 1], ls='--', color=colors[5], label=None)
ax.plot(OCV_model.axes_manager[0].axis, OCV_model.as_signal().data, ls='--', color=colors[3], label=None)

ax.set_xlim(6535.0, 6550.0)
ax.set_xlabel(r'Energy (eV)', fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=5, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=2.5, offset=0))
ax.set_ylim(-0.01, 0.16)
ax.set_ylabel(r'Relative Intensity (arb.u.)', fontsize=11,)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.04))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.02))

# ax.text(-0.25, 1.0, r'b', transform=ax.transAxes, fontsize=14, va='center', ha='right', fontfamily='Arial', fontweight='bold')

ax.axvline(x=6540.3, ymax=0.65, ls='--')
ax.axvline(x=6543.0, ymax=0.7, ls='--')

plt.savefig(path.joinpath(path_out, r'PreEdge_14_300_V3_0.tif'), transparent=False, pad_inches=0.05, bbox_inches='tight', dpi=300, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'PreEdge_14_600_V3_0.tif'), transparent=False, pad_inches=0.05, bbox_inches='tight', dpi=600, pil_kwargs={"compression": "tiff_lzw"})
plt.show()

#### 拟合峰形, IV + III (pseudo-Voigt 峰 - 三个) + II (pseudo-Voigt 峰 - 一个)

In [ ]:
# 数据清洗
Mn_IV = data.iloc[:, [8, 11]].copy().dropna(axis=0, how='all')
Mn_IV.iloc[115:, 1] = 0

# 将 refMn_I//III/IV 做成一个 scale model
from hyperspy.axes import DataAxis
energy_refMn_IV = Mn_IV.iloc[:, 0].to_numpy(dtype='float64')
energy_refMn_IV = DataAxis(axis=energy_refMn_IV, index_in_array=None, name='Energy', units='eV',)
refMn_IV = hs.signals.Signal1D(Mn_IV.iloc[:, 1], axes=[energy_refMn_IV,])
model_refMn_IV = hs.model.components1D.ScalableFixedPattern(refMn_IV) 

# 初始化上述的模型
model_refMn_IV.name= r'Mn_IV'
# model_refMn_IV.xscale.value=1.0
# model_refMn_IV.shift.value=0
# model_refMn_IV.set_parameters_not_free(parameter_name_list=['xscale', 'shift',])
model_refMn_IV.yscale.bmin=0.0

# III 的模型
pv1 = hs.model.components1D.Voigt(centre=6541.14, sigma=0.62343, gamma=0.55, module='scipy')
pv1.name = r'Mn_III_1'
pv1.set_parameters_not_free(parameter_name_list=['centre', 'sigma', 'gamma',])
pv1.centre.bmin = 6538.0
pv1.centre.bmax = 6542.5
pv1.area.bmin = 0

pv2 = hs.model.components1D.Voigt(centre=6544.73, sigma=0.62343, gamma=0.55, module='scipy')
pv2.set_parameters_not_free(parameter_name_list=['centre',])
pv2.name = r'Mn_III_2'
pv2.centre.bmin = 6543.0
pv2.centre.bmax = 6545.0
pv2.area.bmin = 0
pv2.sigma.twin = pv1.sigma
pv2.gamma.twin = pv1.gamma
pv2.area.twin_function_expr = '0.74805*x'
pv2.area.twin = pv1.area

pv3 = hs.model.components1D.Voigt(centre=6543.11, sigma=0.62343, gamma=0.55, module='scipy')
pv3.set_parameters_not_free(parameter_name_list=['centre',])
pv3.name = r'Mn_III_3'
pv3.centre.bmin = pv1.centre.bmax - 0.5
pv3.centre.bmax = pv2.centre.bmin + 0.5
pv3.area.bmin = 0
pv3.sigma.twin = pv1.sigma
pv3.gamma.twin = pv1.gamma
pv3.area.twin_function_expr = '0.867960*x'
pv3.area.twin = pv1.area

# II 的模型
pv4 = hs.model.components1D.Voigt(centre=6540.77, sigma=0.62343, gamma=0.55, module='scipy')
pv4.name = r'Mn_II'
pv4.set_parameters_not_free(parameter_name_list=['centre', 'sigma', 'gamma',])
pv4.centre.bmin = 6540.0
pv4.centre.bmax = 6541.5
pv4.area.bmin = 0


In [ ]:
Mn_OCV = data.iloc[:, [24, 27]].copy().dropna(axis=0, how='all')

In [ ]:
from hyperspy.axes import DataAxis
energy_OCV = Mn_OCV.iloc[:, 0].to_numpy(dtype='float64')
energy_OCV = DataAxis(axis=energy_OCV, index_in_array=None, name='Energy', units='eV',)
OCV = hs.signals.Signal1D(Mn_OCV.iloc[:, 1], axes=[energy_OCV,])
OCV = OCV.isig[6532.0:].deepcopy()
OCV.plot()

In [ ]:
OCV_model = OCV.create_model()
OCV_model.extend((model_refMn_IV, pv1, pv2, pv3, pv4))
OCV_model.set_signal_range(6536.0, 6550.0)

In [ ]:
%matplotlib widget
plt.close('all')
OCV_model.fit(optimizer="lm", bounded=True, return_info=False) 
OCV_model.plot(plot_components=True, plot_residual=False)
plt.savefig(path.joinpath(path_out, r'PreEdge_15_300_V3_0.tif'), pad_inches=0.05, bbox_inches='tight', dpi=300, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'PreEdge_15_600_V3_0.tif'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
OCV_model.print_current_values()

In [ ]:
# 转化成 DataFrame
fit_Mn_IV = pd.DataFrame([OCV_model.axes_manager[0].axis, OCV_model.as_signal(component_list=['Mn_IV']).data])
fit_Mn_IV = fit_Mn_IV.T
fit_Mn_IV.dropna(how='any', inplace=True)

fit_Mn_III_1 = pd.DataFrame([OCV_model.axes_manager[0].axis, OCV_model.as_signal(component_list=['Mn_III_1']).data])
fit_Mn_III_1 = fit_Mn_III_1.T
fit_Mn_III_1.dropna(how='any', inplace=True)

fit_Mn_III_2 = pd.DataFrame([OCV_model.axes_manager[0].axis, OCV_model.as_signal(component_list=['Mn_III_2']).data])
fit_Mn_III_2 = fit_Mn_III_2.T
fit_Mn_III_2.dropna(how='any', inplace=True)

fit_Mn_III_3 = pd.DataFrame([OCV_model.axes_manager[0].axis, OCV_model.as_signal(component_list=['Mn_III_3']).data])
fit_Mn_III_3 = fit_Mn_III_3.T
fit_Mn_III_3.dropna(how='any', inplace=True)

fit_Mn_II = pd.DataFrame([OCV_model.axes_manager[0].axis, OCV_model.as_signal(component_list=['Mn_II']).data])
fit_Mn_II = fit_Mn_II.T
fit_Mn_II.dropna(how='any', inplace=True)

# 计算比例
pre_edge_ratio_Mn_IV = scipy.integrate.simpson(y=fit_Mn_IV.iloc[:, 1], x=fit_Mn_IV.iloc[:, 0])/(scipy.integrate.simpson(y=fit_Mn_IV.iloc[:, 1], x=fit_Mn_IV.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_III_1.iloc[:, 1], x=fit_Mn_III_1.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_III_2.iloc[:, 1], x=fit_Mn_III_2.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_III_3.iloc[:, 1], x=fit_Mn_III_3.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_II.iloc[:, 1], x=fit_Mn_II.iloc[:, 0]))
pre_edge_ratio_Mn_III = (scipy.integrate.simpson(y=fit_Mn_III_1.iloc[:, 1], x=fit_Mn_III_1.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_III_2.iloc[:, 1], x=fit_Mn_III_2.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_III_3.iloc[:, 1], x=fit_Mn_III_3.iloc[:, 0]))/(scipy.integrate.simpson(y=fit_Mn_IV.iloc[:, 1], x=fit_Mn_IV.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_III_1.iloc[:, 1], x=fit_Mn_III_1.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_III_2.iloc[:, 1], x=fit_Mn_III_2.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_III_3.iloc[:, 1], x=fit_Mn_III_3.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_II.iloc[:, 1], x=fit_Mn_II.iloc[:, 0]))
pre_edge_ratio_Mn_II = scipy.integrate.simpson(y=fit_Mn_II.iloc[:, 1], x=fit_Mn_II.iloc[:, 0])/(scipy.integrate.simpson(y=fit_Mn_IV.iloc[:, 1], x=fit_Mn_IV.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_III.iloc[:, 1], x=fit_Mn_III.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_II.iloc[:, 1], x=fit_Mn_II.iloc[:, 0]))
pre_edge_ratio_Mn_IV, pre_edge_ratio_Mn_III, pre_edge_ratio_Mn_II

In [ ]:
Pre_Edge_OCV = xr.Dataset(
    data_vars=dict(
        OCV = (['Energy_OCV'], OCV.data),
        fit = (['Energy_fit'], OCV_model.as_signal().data),
        fit_Mn_IV = (['Energy_fit'], OCV_model.as_signal(component_list=['Mn_IV']).data),
        fit_Mn_III_1 = (['Energy_fit'], OCV_model.as_signal(component_list=['Mn_III_1']).data),
        fit_Mn_III_2 = (['Energy_fit'], OCV_model.as_signal(component_list=['Mn_III_2']).data),
        fit_Mn_III_3 = (['Energy_fit'], OCV_model.as_signal(component_list=['Mn_III_3']).data),
        fit_Mn_II = (['Energy_fit'], OCV_model.as_signal(component_list=['Mn_II']).data),
        ),
    coords=dict(
        Energy_OCV = OCV.axes_manager[0].axis,
        Energy_fit= OCV_model.axes_manager[0].axis,
    ),
    attrs=dict(
        ratio_Mn_IV = pre_edge_ratio_Mn_IV,
        ratio_Mn_III = pre_edge_ratio_Mn_III,
        ratio_Mn_II = pre_edge_ratio_Mn_II,
        chisq = OCV_model.chisq.data[0],
        red_chisq = OCV_model.red_chisq.data[0],
    ),
)

OCV_model.save_parameters2file(path.joinpath(path_out, r'OCV_model_parameters_ref_1_voigt_3_voigt_1'))
OCV_model.save(path.joinpath(path_out, r'OCV_model_ref_1_voigt_3_voigt_1'), 'model', overwrite=True)
Pre_Edge_OCV.to_netcdf(path.joinpath(path_out, r'OCV_result_ref_1_voigt_3_voigt_1.NETCDF4'), engine="h5netcdf")

In [ ]:
# 画图
%matplotlib inline
plt.close('all')
fig = plt.figure(figsize=(7, 2.5))
gs = gridspec.GridSpec(1, 2, width_ratios=[1, 1], height_ratios=None, wspace=0.0, hspace=0.0, figure=fig)

xas_colors_Mn = [colors[0], colors[4]]
xas_cmap_Mn = ListedColormap(mpl.colormaps['coolwarm'](np.linspace(0.5, 1.0, data.shape[1]//4-5)), name='xas_cmap_Mn', )
labels_Mn = [r'${\alpha}$-MnO$\mathrm{_2}$', r'0.2M Mn$\mathrm{^{2\!+}}$']

# 图
subfig = fig.add_subfigure(gs[0, 0], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0, 0, 1.0, 1.0))
ax.set_box_aspect(0.8)
pad = 0.05
ax.plot(data.iloc[:, 4], data.iloc[:, 7]+pad, ls='-', color=colors[4], label=r'ref.MnO')
ax.plot(data.iloc[:, 0], data.iloc[:, 3]+pad, ls='-', color=colors[5], label=r'ref.Mn$\mathrm{_2O_3}$')
ax.plot(data.iloc[:, 8], data.iloc[:, 11]+pad, ls='-', color=colors[3], label=r'ref.Mn$\mathrm{O_2}$')
ax.legend(loc='upper left', bbox_to_anchor=(0.0, 1.02), ncols=1, frameon=False, fontsize=9, labelcolor='linecolor', columnspacing=0.0)

ax.plot(OCV.axes_manager[0].axis, OCV.data, ls='-', color='k', label=None)
ax.plot(fit_Mn_II.iloc[:, 0], fit_Mn_II.iloc[:, 1], ls='--', color=colors[4], label=None)
ax.plot(fit_Mn_IV.iloc[:, 0], fit_Mn_IV.iloc[:, 1], ls='--', color=colors[3], label=None)
ax.plot(fit_Mn_III_1.iloc[:, 0], fit_Mn_III_1.iloc[:, 1] + fit_Mn_III_2.iloc[:, 1] + fit_Mn_III_3.iloc[:, 1], ls='--', color=colors[5], label=None)
ax.plot(OCV_model.axes_manager[0].axis, OCV_model.as_signal().data, ls='--', color=colors[3], label=None)

ax.set_xlim(6535.0, 6550.0)
ax.set_xlabel(r'Energy (eV)', fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=5, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=2.5, offset=0))
ax.set_ylim(-0.01, 0.16)
ax.set_ylabel(r'Relative Intensity (arb.u.)', fontsize=11,)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.04))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.02))

# ax.text(-0.25, 1.0, r'b', transform=ax.transAxes, fontsize=14, va='center', ha='right', fontfamily='Arial', fontweight='bold')

ax.axvline(x=6540.3, ymax=0.65, ls='--')
ax.axvline(x=6543.0, ymax=0.7, ls='--')

plt.savefig(path.joinpath(path_out, r'PreEdge_16_300_V3_0.tif'), transparent=False, pad_inches=0.05, bbox_inches='tight', dpi=300, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'PreEdge_16_600_V3_0.tif'), transparent=False, pad_inches=0.05, bbox_inches='tight', dpi=600, pil_kwargs={"compression": "tiff_lzw"})
plt.show()

### FD1st op

#### 拟合峰形, IV + III + II (pseudo-Voigt 峰 - 一个)

In [ ]:
# IV/III 的模型
Mn_IV = data.iloc[:, [8, 11]].copy().dropna(axis=0, how='all')
Mn_III = data.iloc[:, [0, 3]].copy().dropna(axis=0, how='all')

Mn_IV.iloc[115:, 1] = 0
Mn_III.iloc[97:, 1] = 0

# 将 refMn_I//III/IV 做成一个 scale model
from hyperspy.axes import DataAxis
energy_refMn_IV = Mn_IV.iloc[:, 0].to_numpy(dtype='float64')
energy_refMn_IV = DataAxis(axis=energy_refMn_IV, index_in_array=None, name='Energy', units='eV',)
refMn_IV = hs.signals.Signal1D(Mn_IV.iloc[:, 1], axes=[energy_refMn_IV,])
model_refMn_IV = hs.model.components1D.ScalableFixedPattern(refMn_IV) 

energy_refMn_III = Mn_III.iloc[:, 0].to_numpy(dtype='float64')
energy_refMn_III = DataAxis(axis=energy_refMn_III, index_in_array=None, name='Energy', units='eV',)
refMn_III = hs.signals.Signal1D(Mn_III.iloc[:, 1], axes=[energy_refMn_III,])
model_refMn_III = hs.model.components1D.ScalableFixedPattern(refMn_III) 

# 初始化上述的模型
model_refMn_IV.name= r'Mn_IV'
# model_refMn_IV.xscale.value=1.03564
# model_refMn_IV.shift.value=232.968
# model_refMn_IV.set_parameters_not_free(parameter_name_list=['xscale', 'shift',])
model_refMn_IV.yscale.bmin=0.0

model_refMn_III.name= r'Mn_III'
# model_refMn_III.xscale.value=1.13677
# model_refMn_III.shift.value=894.306
# model_refMn_III.set_parameters_not_free(parameter_name_list=['xscale', 'shift',])
model_refMn_III.yscale.bmin=0.0

# II 的模型
pv4 = hs.model.components1D.Voigt(centre=6540.77, sigma=0.62343, gamma=0.55, module='scipy')
pv4.name = r'Mn_II'
pv4.set_parameters_not_free(parameter_name_list=['centre', 'sigma', 'gamma',])
pv4.centre.bmin = 6540.0
pv4.centre.bmax = 6541.5
pv4.area.bmin = 0

In [ ]:
Mn_FD1st = data.iloc[:, [-4, -1]].copy().dropna(axis=0, how='all')

In [ ]:
from hyperspy.axes import DataAxis
energy_Mn_FD1st = Mn_FD1st.iloc[:, 0].to_numpy(dtype='float64')
energy_Mn_FD1st = DataAxis(axis=energy_Mn_FD1st, index_in_array=None, name='Energy', units='eV',)
FD1st = hs.signals.Signal1D(Mn_FD1st.iloc[:, 1], axes=[energy_Mn_FD1st,])
FD1st = FD1st.isig[6532.0:].deepcopy()
FD1st.plot()

In [ ]:
FD1st_model = FD1st.create_model()
FD1st_model.extend((pv4, model_refMn_III, model_refMn_IV))
FD1st_model.set_signal_range(6536.0, 6550.0)

In [ ]:
%matplotlib widget
plt.close('all')
FD1st_model.fit(optimizer="lm", bounded=True, return_info=False) 
FD1st_model.plot(plot_components=True, plot_residual=False)
plt.savefig(path.joinpath(path_out, r'PreEdge_17_300_V3_0.tif'), pad_inches=0.05, bbox_inches='tight', dpi=300, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'PreEdge_17_600_V3_0.tif'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
FD1st_model.print_current_values()

In [ ]:
# 转化成 DataFrame
fit_Mn_IV = pd.DataFrame([FD1st_model.axes_manager[0].axis, FD1st_model.as_signal(component_list=['Mn_IV']).data])
fit_Mn_IV = fit_Mn_IV.T
fit_Mn_IV.dropna(how='any', inplace=True)

fit_Mn_III = pd.DataFrame([FD1st_model.axes_manager[0].axis, FD1st_model.as_signal(component_list=['Mn_III']).data])
fit_Mn_III = fit_Mn_III.T
fit_Mn_III.dropna(how='any', inplace=True)

fit_Mn_II = pd.DataFrame([FD1st_model.axes_manager[0].axis, FD1st_model.as_signal(component_list=['Mn_II']).data])
fit_Mn_II = fit_Mn_II.T
fit_Mn_II.dropna(how='any', inplace=True)

# 计算比例
pre_edge_ratio_Mn_IV = scipy.integrate.simpson(y=fit_Mn_IV.iloc[:, 1], x=fit_Mn_IV.iloc[:, 0])/(scipy.integrate.simpson(y=fit_Mn_IV.iloc[:, 1], x=fit_Mn_IV.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_III.iloc[:, 1], x=fit_Mn_III.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_II.iloc[:, 1], x=fit_Mn_II.iloc[:, 0]))
pre_edge_ratio_Mn_III = scipy.integrate.simpson(y=fit_Mn_III.iloc[:, 1], x=fit_Mn_III.iloc[:, 0])/(scipy.integrate.simpson(y=fit_Mn_IV.iloc[:, 1], x=fit_Mn_IV.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_III.iloc[:, 1], x=fit_Mn_III.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_II.iloc[:, 1], x=fit_Mn_II.iloc[:, 0]))
pre_edge_ratio_Mn_II = scipy.integrate.simpson(y=fit_Mn_II.iloc[:, 1], x=fit_Mn_II.iloc[:, 0])/(scipy.integrate.simpson(y=fit_Mn_IV.iloc[:, 1], x=fit_Mn_IV.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_III.iloc[:, 1], x=fit_Mn_III.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_II.iloc[:, 1], x=fit_Mn_II.iloc[:, 0]))
pre_edge_ratio_Mn_IV, pre_edge_ratio_Mn_III, pre_edge_ratio_Mn_II

In [ ]:
Pre_Edge_aMnO2 = xr.Dataset(
    data_vars=dict(
        aMnO2 = (['Energy_aMnO2'], FD1st.data),
        fit = (['Energy_fit'], FD1st_model.as_signal().data),
        fit_Mn_IV = (['Energy_fit'], FD1st_model.as_signal(component_list=['Mn_IV']).data),
        fit_Mn_III = (['Energy_fit'], FD1st_model.as_signal(component_list=['Mn_III']).data),
        fit_Mn_II = (['Energy_fit'], FD1st_model.as_signal(component_list=['Mn_II']).data),
        ),
    coords=dict(
        Energy_aMnO2 = FD1st.axes_manager[0].axis,
        Energy_fit= FD1st_model.axes_manager[0].axis,
    ),
    attrs=dict(
        ratio_Mn_IV = pre_edge_ratio_Mn_IV,
        ratio_Mn_III = pre_edge_ratio_Mn_III,
        ratio_Mn_II = pre_edge_ratio_Mn_II,
    ),
)

FD1st_model.save_parameters2file(path.joinpath(path_out, r'FD1st_model_parameters_ref_2_voigt_1'))
FD1st_model.save(path.joinpath(path_out, r'FD1st_model_ref_2_voigt_1'), 'model', overwrite=True)
Pre_Edge_aMnO2.to_netcdf(path.joinpath(path_out, r'FD1st_result_ref_2_voigt_1.NETCDF4'), engine="h5netcdf")

In [ ]:
# 画图
plt.close('all')
%matplotlib inline
fig = plt.figure(figsize=(7, 2.5))
gs = gridspec.GridSpec(1, 2, width_ratios=[1, 1], height_ratios=None, wspace=0.0, hspace=0.0, figure=fig)

xas_colors_Mn = [colors[0], colors[4]]
xas_cmap_Mn = ListedColormap(mpl.colormaps['coolwarm'](np.linspace(0.5, 1.0, data.shape[1]//4-5)), name='xas_cmap_Mn', )
labels_Mn = [r'${\alpha}$-MnO$\mathrm{_2}$', r'0.2M Mn$\mathrm{^{2\!+}}$']

# 图 
subfig = fig.add_subfigure(gs[0, 0], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0, 0, 1.0, 1.0))
ax.set_box_aspect(0.8)
pad = 0.03
ax.plot(data.iloc[:, 4], data.iloc[:, 7]+pad, ls='-', color=colors[4], label=r'ref.MnO')
ax.plot(data.iloc[:, 0], data.iloc[:, 3]+pad, ls='-', color=colors[5], label=r'ref.Mn$\mathrm{_2O_3}$')
ax.plot(data.iloc[:, 8], data.iloc[:, 11]+pad, ls='-', color=colors[3], label=r'ref.Mn$\mathrm{O_2}$')
ax.legend(loc='upper left', bbox_to_anchor=(0.0, 1.02), ncols=1, frameon=False, fontsize=9, labelcolor='linecolor', columnspacing=0.0)

ax.plot(FD1st.axes_manager[0].axis, FD1st.data, ls='-', color='k', label=None)
ax.plot(fit_Mn_II.iloc[:, 0], fit_Mn_II.iloc[:, 1], ls='--', color=colors[4], label=None)
ax.plot(fit_Mn_III.iloc[:, 0], fit_Mn_III.iloc[:, 1], ls='--', color=colors[5], label=None)
ax.plot(fit_Mn_IV.iloc[:, 0], fit_Mn_IV.iloc[:, 1], ls='--', color=colors[3], label=None)
ax.plot(FD1st_model.axes_manager[0].axis, FD1st_model.as_signal().data, ls='--', color=colors[3], label=None)

ax.set_xlim(6535.0, 6550.0)
ax.set_xlabel(r'Energy (eV)', fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=5, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=2.5, offset=0))
ax.set_ylim(-0.01, 0.12)
ax.set_ylabel(r'Relative Intensity (arb.u.)', fontsize=11,)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.03))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.015))

# ax.text(-0.25, 1.0, r'b', transform=ax.transAxes, fontsize=14, va='center', ha='right', fontfamily='Arial', fontweight='bold')

ax.axvline(x=6540.3, ymax=0.65, ls='--')
ax.axvline(x=6543.0, ymax=0.7, ls='--')

plt.savefig(path.joinpath(path_out, r'PreEdge_18_300_V3_0.tif'), transparent=False, pad_inches=0.05, bbox_inches='tight', dpi=300, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'PreEdge_18_600_V3_0.tif'), transparent=False, pad_inches=0.05, bbox_inches='tight', dpi=600, pil_kwargs={"compression": "tiff_lzw"})
plt.show()

#### 拟合峰形, IV + III (pseudo-Voigt 峰 - 三个) + II (pseudo-Voigt 峰 - 一个)

In [ ]:
# 数据清洗
Mn_IV = data.iloc[:, [8, 11]].copy().dropna(axis=0, how='all')
Mn_IV.iloc[115:, 1] = 0

# 将 refMn_I//III/IV 做成一个 scale model
from hyperspy.axes import DataAxis
energy_refMn_IV = Mn_IV.iloc[:, 0].to_numpy(dtype='float64')
energy_refMn_IV = DataAxis(axis=energy_refMn_IV, index_in_array=None, name='Energy', units='eV',)
refMn_IV = hs.signals.Signal1D(Mn_IV.iloc[:, 1], axes=[energy_refMn_IV,])
model_refMn_IV = hs.model.components1D.ScalableFixedPattern(refMn_IV) 

# 初始化上述的模型
model_refMn_IV.name= r'Mn_IV'
# model_refMn_IV.xscale.value=1.0
# model_refMn_IV.shift.value=0
# model_refMn_IV.set_parameters_not_free(parameter_name_list=['xscale', 'shift',])
model_refMn_IV.yscale.bmin=0.0

# III 的模型
pv1 = hs.model.components1D.Voigt(centre=6541.14, sigma=0.62343, gamma=0.55, module='scipy')
pv1.name = r'Mn_III_1'
pv1.set_parameters_not_free(parameter_name_list=['centre', 'sigma', 'gamma',])
pv1.centre.bmin = 6538.0
pv1.centre.bmax = 6542.5
pv1.area.bmin = 0

pv2 = hs.model.components1D.Voigt(centre=6544.73, sigma=0.62343, gamma=0.55, module='scipy')
pv2.set_parameters_not_free(parameter_name_list=['centre',])
pv2.name = r'Mn_III_2'
pv2.centre.bmin = 6543.0
pv2.centre.bmax = 6545.0
pv2.area.bmin = 0
pv2.sigma.twin = pv1.sigma
pv2.gamma.twin = pv1.gamma
pv2.area.twin_function_expr = '0.74805*x'
pv2.area.twin = pv1.area

pv3 = hs.model.components1D.Voigt(centre=6543.11, sigma=0.62343, gamma=0.55, module='scipy')
pv3.set_parameters_not_free(parameter_name_list=['centre',])
pv3.name = r'Mn_III_3'
pv3.centre.bmin = pv1.centre.bmax - 0.5
pv3.centre.bmax = pv2.centre.bmin + 0.5
pv3.area.bmin = 0
pv3.sigma.twin = pv1.sigma
pv3.gamma.twin = pv1.gamma
pv3.area.twin_function_expr = '0.867960*x'
pv3.area.twin = pv1.area

# II 的模型
pv4 = hs.model.components1D.Voigt(centre=6540.77, sigma=0.62343, gamma=0.55, module='scipy')
pv4.name = r'Mn_II'
pv4.set_parameters_not_free(parameter_name_list=['centre', 'sigma', 'gamma',])
pv4.centre.bmin = 6540.0
pv4.centre.bmax = 6541.5
pv4.area.bmin = 0


In [ ]:
Mn_FD1st = data.iloc[:, [-4, -1]].copy().dropna(axis=0, how='all')

In [ ]:
from hyperspy.axes import DataAxis
energy_Mn_FD1st = Mn_FD1st.iloc[:, 0].to_numpy(dtype='float64')
energy_Mn_FD1st = DataAxis(axis=energy_Mn_FD1st, index_in_array=None, name='Energy', units='eV',)
FD1st = hs.signals.Signal1D(Mn_FD1st.iloc[:, 1], axes=[energy_Mn_FD1st,])
FD1st = FD1st.isig[6532.0:].deepcopy()
FD1st.plot()

In [ ]:
FD1st_model = FD1st.create_model()
FD1st_model.extend((model_refMn_IV, pv1, pv2, pv3, pv4))
FD1st_model.set_signal_range(6536.0, 6550.0)

In [ ]:
%matplotlib widget
plt.close('all')
FD1st_model.fit(optimizer="lm", bounded=True, return_info=False) 
FD1st_model.plot(plot_components=True, plot_residual=False)
plt.savefig(path.joinpath(path_out, r'PreEdge_19_300_V3_0.tif'), pad_inches=0.05, bbox_inches='tight', dpi=300, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'PreEdge_19_600_V3_0.tif'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
FD1st_model.print_current_values()

In [ ]:
# 转化成 DataFrame
fit_Mn_IV = pd.DataFrame([FD1st_model.axes_manager[0].axis, FD1st_model.as_signal(component_list=['Mn_IV']).data])
fit_Mn_IV = fit_Mn_IV.T
fit_Mn_IV.dropna(how='any', inplace=True)

fit_Mn_III_1 = pd.DataFrame([FD1st_model.axes_manager[0].axis, FD1st_model.as_signal(component_list=['Mn_III_1']).data])
fit_Mn_III_1 = fit_Mn_III_1.T
fit_Mn_III_1.dropna(how='any', inplace=True)

fit_Mn_III_2 = pd.DataFrame([FD1st_model.axes_manager[0].axis, FD1st_model.as_signal(component_list=['Mn_III_2']).data])
fit_Mn_III_2 = fit_Mn_III_2.T
fit_Mn_III_2.dropna(how='any', inplace=True)

fit_Mn_III_3 = pd.DataFrame([FD1st_model.axes_manager[0].axis, FD1st_model.as_signal(component_list=['Mn_III_3']).data])
fit_Mn_III_3 = fit_Mn_III_3.T
fit_Mn_III_3.dropna(how='any', inplace=True)

fit_Mn_II = pd.DataFrame([FD1st_model.axes_manager[0].axis, FD1st_model.as_signal(component_list=['Mn_II']).data])
fit_Mn_II = fit_Mn_II.T
fit_Mn_II.dropna(how='any', inplace=True)

# 计算比例
pre_edge_ratio_Mn_IV = scipy.integrate.simpson(y=fit_Mn_IV.iloc[:, 1], x=fit_Mn_IV.iloc[:, 0])/(scipy.integrate.simpson(y=fit_Mn_IV.iloc[:, 1], x=fit_Mn_IV.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_III_1.iloc[:, 1], x=fit_Mn_III_1.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_III_2.iloc[:, 1], x=fit_Mn_III_2.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_III_3.iloc[:, 1], x=fit_Mn_III_3.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_II.iloc[:, 1], x=fit_Mn_II.iloc[:, 0]))
pre_edge_ratio_Mn_III = (scipy.integrate.simpson(y=fit_Mn_III_1.iloc[:, 1], x=fit_Mn_III_1.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_III_2.iloc[:, 1], x=fit_Mn_III_2.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_III_3.iloc[:, 1], x=fit_Mn_III_3.iloc[:, 0]))/(scipy.integrate.simpson(y=fit_Mn_IV.iloc[:, 1], x=fit_Mn_IV.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_III_1.iloc[:, 1], x=fit_Mn_III_1.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_III_2.iloc[:, 1], x=fit_Mn_III_2.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_III_3.iloc[:, 1], x=fit_Mn_III_3.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_II.iloc[:, 1], x=fit_Mn_II.iloc[:, 0]))
pre_edge_ratio_Mn_II = scipy.integrate.simpson(y=fit_Mn_II.iloc[:, 1], x=fit_Mn_II.iloc[:, 0])/(scipy.integrate.simpson(y=fit_Mn_IV.iloc[:, 1], x=fit_Mn_IV.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_III.iloc[:, 1], x=fit_Mn_III.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_II.iloc[:, 1], x=fit_Mn_II.iloc[:, 0]))
pre_edge_ratio_Mn_IV, pre_edge_ratio_Mn_III, pre_edge_ratio_Mn_II

In [ ]:
Pre_Edge_FD1st = xr.Dataset(
    data_vars=dict(
        FD1st = (['Energy_FD1st'], FD1st.data),
        fit = (['Energy_fit'], FD1st_model.as_signal().data),
        fit_Mn_IV = (['Energy_fit'], FD1st_model.as_signal(component_list=['Mn_IV']).data),
        fit_Mn_III_1 = (['Energy_fit'], FD1st_model.as_signal(component_list=['Mn_III_1']).data),
        fit_Mn_III_2 = (['Energy_fit'], FD1st_model.as_signal(component_list=['Mn_III_2']).data),
        fit_Mn_III_3 = (['Energy_fit'], FD1st_model.as_signal(component_list=['Mn_III_3']).data),
        fit_Mn_II = (['Energy_fit'], FD1st_model.as_signal(component_list=['Mn_II']).data),
        ),
    coords=dict(
        Energy_FD1st = FD1st.axes_manager[0].axis,
        Energy_fit= FD1st_model.axes_manager[0].axis,
    ),
    attrs=dict(
        ratio_Mn_IV = pre_edge_ratio_Mn_IV,
        ratio_Mn_III = pre_edge_ratio_Mn_III,
        ratio_Mn_II = pre_edge_ratio_Mn_II,
        chisq = FD1st_model.chisq.data[0],
        red_chisq = FD1st_model.red_chisq.data[0],
    ),
)

FD1st_model.save_parameters2file(path.joinpath(path_out, r'FD1st_model_parameters_ref_1_voigt_3_voigt_1'))
FD1st_model.save(path.joinpath(path_out, r'FD1st_model_ref_1_voigt_3_voigt_1'), 'model', overwrite=True)
Pre_Edge_OCV.to_netcdf(path.joinpath(path_out, r'FD1st_result_ref_1_voigt_3_voigt_1.NETCDF4'), engine="h5netcdf")

In [ ]:
# 画图
%matplotlib inline
plt.close('all')
fig = plt.figure(figsize=(7, 2.5))
gs = gridspec.GridSpec(1, 2, width_ratios=[1, 1], height_ratios=None, wspace=0.0, hspace=0.0, figure=fig)

xas_colors_Mn = [colors[0], colors[4]]
xas_cmap_Mn = ListedColormap(mpl.colormaps['coolwarm'](np.linspace(0.5, 1.0, data.shape[1]//4-5)), name='xas_cmap_Mn', )
labels_Mn = [r'${\alpha}$-MnO$\mathrm{_2}$', r'0.2M Mn$\mathrm{^{2\!+}}$']

# 图 B
subfig = fig.add_subfigure(gs[0, 0], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0, 0, 1.0, 1.0))
ax.set_box_aspect(0.8)
pad = 0.03
ax.plot(data.iloc[:, 4], data.iloc[:, 7]+pad, ls='-', color=colors[4], label=r'ref.MnO')
ax.plot(data.iloc[:, 0], data.iloc[:, 3]+pad, ls='-', color=colors[5], label=r'ref.Mn$\mathrm{_2O_3}$')
ax.plot(data.iloc[:, 8], data.iloc[:, 11]+pad, ls='-', color=colors[3], label=r'ref.Mn$\mathrm{O_2}$')
ax.legend(loc='upper left', bbox_to_anchor=(0.0, 1.02), ncols=1, frameon=False, fontsize=9, labelcolor='linecolor', columnspacing=0.0)

ax.plot(FD1st.axes_manager[0].axis, FD1st.data, ls='-', color='k', label=None)
ax.plot(fit_Mn_II.iloc[:, 0], fit_Mn_II.iloc[:, 1], ls='--', color=colors[4], label=None)
ax.plot(fit_Mn_IV.iloc[:, 0], fit_Mn_IV.iloc[:, 1], ls='--', color=colors[3], label=None)
ax.plot(fit_Mn_III_1.iloc[:, 0], fit_Mn_III_1.iloc[:, 1] + fit_Mn_III_2.iloc[:, 1] + fit_Mn_III_3.iloc[:, 1], ls='--', color=colors[5], label=None)
ax.plot(FD1st_model.axes_manager[0].axis, FD1st_model.as_signal().data, ls='--', color=colors[3], label=None)

ax.set_xlim(6535.0, 6550.0)
ax.set_xlabel(r'Energy (eV)', fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=5, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=2.5, offset=0))
ax.set_ylim(-0.01, 0.12)
ax.set_ylabel(r'Relative Intensity (arb.u.)', fontsize=11,)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.03))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.015))

# ax.text(-0.25, 1.0, r'b', transform=ax.transAxes, fontsize=14, va='center', ha='right', fontfamily='Arial', fontweight='bold')

ax.axvline(x=6540.3, ymax=0.65, ls='--')
ax.axvline(x=6543.0, ymax=0.7, ls='--')

plt.savefig(path.joinpath(path_out, r'PreEdge_20_300_V3_0.tif'), transparent=False, pad_inches=0.05, bbox_inches='tight', dpi=300, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'PreEdge_20_600_V3_0.tif'), transparent=False, pad_inches=0.05, bbox_inches='tight', dpi=600, pil_kwargs={"compression": "tiff_lzw"})
plt.show()

### FD1st ex situ

#### 拟合峰形, IV + III + II (pseudo-Voigt 峰 - 一个)

In [ ]:
# IV/III 的模型
Mn_IV = data.iloc[:, [8, 11]].copy().dropna(axis=0, how='all')
Mn_III = data.iloc[:, [0, 3]].copy().dropna(axis=0, how='all')

Mn_IV.iloc[115:, 1] = 0
Mn_III.iloc[97:, 1] = 0

# 将 refMn_I//III/IV 做成一个 scale model
from hyperspy.axes import DataAxis
energy_refMn_IV = Mn_IV.iloc[:, 0].to_numpy(dtype='float64')
energy_refMn_IV = DataAxis(axis=energy_refMn_IV, index_in_array=None, name='Energy', units='eV',)
refMn_IV = hs.signals.Signal1D(Mn_IV.iloc[:, 1], axes=[energy_refMn_IV,])
model_refMn_IV = hs.model.components1D.ScalableFixedPattern(refMn_IV) 

energy_refMn_III = Mn_III.iloc[:, 0].to_numpy(dtype='float64')
energy_refMn_III = DataAxis(axis=energy_refMn_III, index_in_array=None, name='Energy', units='eV',)
refMn_III = hs.signals.Signal1D(Mn_III.iloc[:, 1], axes=[energy_refMn_III,])
model_refMn_III = hs.model.components1D.ScalableFixedPattern(refMn_III) 

# 初始化上述的模型
model_refMn_IV.name= r'Mn_IV'
# model_refMn_IV.xscale.value=1.03564
# model_refMn_IV.shift.value=232.968
# model_refMn_IV.set_parameters_not_free(parameter_name_list=['xscale', 'shift',])
model_refMn_IV.yscale.bmin=0.0

model_refMn_III.name= r'Mn_III'
# model_refMn_III.xscale.value=1.13677
# model_refMn_III.shift.value=894.306
# model_refMn_III.set_parameters_not_free(parameter_name_list=['xscale', 'shift',])
model_refMn_III.yscale.bmin=0.0

# II 的模型
pv4 = hs.model.components1D.Voigt(centre=6540.77, sigma=0.62343, gamma=0.55, module='scipy')
pv4.name = r'Mn_II'
pv4.set_parameters_not_free(parameter_name_list=['centre', 'sigma', 'gamma',])
pv4.centre.bmin = 6540.0
pv4.centre.bmax = 6541.5
pv4.area.bmin = 0

In [ ]:
Mn_FD1st_ex = data.iloc[:, [20, 23]].copy().dropna(axis=0, how='all')

In [ ]:
from hyperspy.axes import DataAxis
energy_Mn_FD1st_ex = Mn_FD1st_ex.iloc[:, 0].to_numpy(dtype='float64')
energy_Mn_FD1st_ex = DataAxis(axis=energy_Mn_FD1st_ex, index_in_array=None, name='Energy', units='eV',)
FD1st_ex = hs.signals.Signal1D(Mn_FD1st_ex.iloc[:, 1], axes=[energy_Mn_FD1st_ex,])
FD1st_ex = FD1st_ex.isig[6532.0:].deepcopy()
FD1st_ex.plot()

In [ ]:
FD1st_ex_model = FD1st_ex.create_model()
FD1st_ex_model.extend((pv4, model_refMn_III, model_refMn_IV))
FD1st_ex_model.set_signal_range(6536.0, 6550.0)

In [ ]:
%matplotlib widget
plt.close('all')
FD1st_ex_model.fit(optimizer="lm", bounded=True, return_info=False) 
FD1st_ex_model.plot(plot_components=True, plot_residual=False)
plt.savefig(path.joinpath(path_out, r'PreEdge_21_300_V3_0.tif'), pad_inches=0.05, bbox_inches='tight', dpi=300, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'PreEdge_21_600_V3_0.tif'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
FD1st_ex_model.print_current_values()

In [ ]:
# 转化成 DataFrame
fit_Mn_IV = pd.DataFrame([FD1st_ex_model.axes_manager[0].axis, FD1st_ex_model.as_signal(component_list=['Mn_IV']).data])
fit_Mn_IV = fit_Mn_IV.T
fit_Mn_IV.dropna(how='any', inplace=True)

fit_Mn_III = pd.DataFrame([FD1st_ex_model.axes_manager[0].axis, FD1st_ex_model.as_signal(component_list=['Mn_III']).data])
fit_Mn_III = fit_Mn_III.T
fit_Mn_III.dropna(how='any', inplace=True)

fit_Mn_II = pd.DataFrame([FD1st_ex_model.axes_manager[0].axis, FD1st_ex_model.as_signal(component_list=['Mn_II']).data])
fit_Mn_II = fit_Mn_II.T
fit_Mn_II.dropna(how='any', inplace=True)

# 计算比例
pre_edge_ratio_Mn_IV = scipy.integrate.simpson(y=fit_Mn_IV.iloc[:, 1], x=fit_Mn_IV.iloc[:, 0])/(scipy.integrate.simpson(y=fit_Mn_IV.iloc[:, 1], x=fit_Mn_IV.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_III.iloc[:, 1], x=fit_Mn_III.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_II.iloc[:, 1], x=fit_Mn_II.iloc[:, 0]))
pre_edge_ratio_Mn_III = scipy.integrate.simpson(y=fit_Mn_III.iloc[:, 1], x=fit_Mn_III.iloc[:, 0])/(scipy.integrate.simpson(y=fit_Mn_IV.iloc[:, 1], x=fit_Mn_IV.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_III.iloc[:, 1], x=fit_Mn_III.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_II.iloc[:, 1], x=fit_Mn_II.iloc[:, 0]))
pre_edge_ratio_Mn_II = scipy.integrate.simpson(y=fit_Mn_II.iloc[:, 1], x=fit_Mn_II.iloc[:, 0])/(scipy.integrate.simpson(y=fit_Mn_IV.iloc[:, 1], x=fit_Mn_IV.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_III.iloc[:, 1], x=fit_Mn_III.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_II.iloc[:, 1], x=fit_Mn_II.iloc[:, 0]))
pre_edge_ratio_Mn_IV, pre_edge_ratio_Mn_III, pre_edge_ratio_Mn_II

In [ ]:
Pre_Edge_aMnO2 = xr.Dataset(
    data_vars=dict(
        aMnO2 = (['Energy_aMnO2'], FD1st_ex.data),
        fit = (['Energy_fit'], FD1st_ex_model.as_signal().data),
        fit_Mn_IV = (['Energy_fit'], FD1st_ex_model.as_signal(component_list=['Mn_IV']).data),
        fit_Mn_III = (['Energy_fit'], FD1st_ex_model.as_signal(component_list=['Mn_III']).data),
        fit_Mn_II = (['Energy_fit'], FD1st_ex_model.as_signal(component_list=['Mn_II']).data),
        ),
    coords=dict(
        Energy_aMnO2 = FD1st_ex.axes_manager[0].axis,
        Energy_fit= FD1st_ex_model.axes_manager[0].axis,
    ),
    attrs=dict(
        ratio_Mn_IV = pre_edge_ratio_Mn_IV,
        ratio_Mn_III = pre_edge_ratio_Mn_III,
        ratio_Mn_II = pre_edge_ratio_Mn_II,
        chisq = FD1st_ex_model.chisq.data[0],
        red_chisq = FD1st_ex_model.red_chisq.data[0],
    ),
)

FD1st_ex_model.save_parameters2file(path.joinpath(path_out, r'FD1st_ex_model_parameters_ref_2_voigt_1'))
FD1st_ex_model.save(path.joinpath(path_out, r'FD1st_ex_model_ref_2_voigt_1'), 'model', overwrite=True)
Pre_Edge_aMnO2.to_netcdf(path.joinpath(path_out, r'FD1st_ex_result_ref_2_voigt_1.NETCDF4'), engine="h5netcdf")

In [ ]:
# 画图
plt.close('all')
%matplotlib inline
fig = plt.figure(figsize=(7, 2.5))
gs = gridspec.GridSpec(1, 2, width_ratios=[1, 1], height_ratios=None, wspace=0.0, hspace=0.0, figure=fig)

xas_colors_Mn = [colors[0], colors[4]]
xas_cmap_Mn = ListedColormap(mpl.colormaps['coolwarm'](np.linspace(0.5, 1.0, data.shape[1]//4-5)), name='xas_cmap_Mn', )
labels_Mn = [r'${\alpha}$-MnO$\mathrm{_2}$', r'0.2M Mn$\mathrm{^{2\!+}}$']

# 图 B
subfig = fig.add_subfigure(gs[0, 0], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0, 0, 1.0, 1.0))
ax.set_box_aspect(0.8)

ax.plot(data.iloc[:, 4], data.iloc[:, 7]+0.08, ls='-', color=colors[4], label=r'ref.MnO')
ax.plot(data.iloc[:, 0], data.iloc[:, 3]+0.08, ls='-', color=colors[5], label=r'ref.Mn$\mathrm{_2O_3}$')
ax.plot(data.iloc[:, 8], data.iloc[:, 11]+0.08, ls='-', color=colors[3], label=r'ref.Mn$\mathrm{O_2}$')
ax.legend(loc='upper left', bbox_to_anchor=(0.0, 1.02), ncols=1, frameon=False, fontsize=9, labelcolor='linecolor', columnspacing=0.0)

ax.plot(FD1st_ex.axes_manager[0].axis, FD1st_ex.data, ls='-', color='k', label=None)
ax.plot(fit_Mn_II.iloc[:, 0], fit_Mn_II.iloc[:, 1], ls='--', color=colors[4], label=None)
ax.plot(fit_Mn_III.iloc[:, 0], fit_Mn_III.iloc[:, 1], ls='--', color=colors[5], label=None)
ax.plot(fit_Mn_IV.iloc[:, 0], fit_Mn_IV.iloc[:, 1], ls='--', color=colors[3], label=None)
ax.plot(FD1st_ex_model.axes_manager[0].axis, FD1st_ex_model.as_signal().data, ls='--', color=colors[3], label=None)

ax.set_xlim(6535.0, 6550.0)
ax.set_xlabel(r'Energy (eV)', fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=5, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=2.5, offset=0))
ax.set_ylim(-0.01, 0.20)
ax.set_ylabel(r'Relative Intensity (arb.u.)', fontsize=11,)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.05))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.025))

# ax.text(-0.25, 1.0, r'b', transform=ax.transAxes, fontsize=14, va='center', ha='right', fontfamily='Arial', fontweight='bold')

ax.axvline(x=6540.3, ymax=0.65, ls='--')
ax.axvline(x=6543.0, ymax=0.7, ls='--')

plt.savefig(path.joinpath(path_out, r'PreEdge_22_300_V3_0.tif'), transparent=False, pad_inches=0.05, bbox_inches='tight', dpi=300, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'PreEdge_22_600_V3_0.tif'), transparent=False, pad_inches=0.05, bbox_inches='tight', dpi=600, pil_kwargs={"compression": "tiff_lzw"})
plt.show()

#### 拟合峰形, IV + III (pseudo-Voigt 峰 - 三个) + II (pseudo-Voigt 峰 - 一个)

In [ ]:
# 数据清洗
Mn_IV = data.iloc[:, [8, 11]].copy().dropna(axis=0, how='all')
Mn_IV.iloc[115:, 1] = 0

# 将 refMn_I//III/IV 做成一个 scale model
from hyperspy.axes import DataAxis
energy_refMn_IV = Mn_IV.iloc[:, 0].to_numpy(dtype='float64')
energy_refMn_IV = DataAxis(axis=energy_refMn_IV, index_in_array=None, name='Energy', units='eV',)
refMn_IV = hs.signals.Signal1D(Mn_IV.iloc[:, 1], axes=[energy_refMn_IV,])
model_refMn_IV = hs.model.components1D.ScalableFixedPattern(refMn_IV) 

# 初始化上述的模型
model_refMn_IV.name= r'Mn_IV'
# model_refMn_IV.xscale.value=1.0
# model_refMn_IV.shift.value=0
# model_refMn_IV.set_parameters_not_free(parameter_name_list=['xscale', 'shift',])
model_refMn_IV.yscale.bmin=0.0

# III 的模型
pv1 = hs.model.components1D.Voigt(centre=6541.14, sigma=0.62343, gamma=0.55, module='scipy')
pv1.name = r'Mn_III_1'
pv1.set_parameters_not_free(parameter_name_list=['centre', 'sigma', 'gamma',])
pv1.centre.bmin = 6538.0
pv1.centre.bmax = 6542.5
pv1.area.bmin = 0

pv2 = hs.model.components1D.Voigt(centre=6544.73, sigma=0.62343, gamma=0.55, module='scipy')
pv2.set_parameters_not_free(parameter_name_list=['centre',])
pv2.name = r'Mn_III_2'
pv2.centre.bmin = 6543.0
pv2.centre.bmax = 6545.0
pv2.area.bmin = 0
pv2.sigma.twin = pv1.sigma
pv2.gamma.twin = pv1.gamma
pv2.area.twin_function_expr = '0.74805*x'
pv2.area.twin = pv1.area

pv3 = hs.model.components1D.Voigt(centre=6543.11, sigma=0.62343, gamma=0.55, module='scipy')
pv3.set_parameters_not_free(parameter_name_list=['centre',])
pv3.name = r'Mn_III_3'
pv3.centre.bmin = pv1.centre.bmax - 0.5
pv3.centre.bmax = pv2.centre.bmin + 0.5
pv3.area.bmin = 0
pv3.sigma.twin = pv1.sigma
pv3.gamma.twin = pv1.gamma
pv3.area.twin_function_expr = '0.867960*x'
pv3.area.twin = pv1.area

# II 的模型
pv4 = hs.model.components1D.Voigt(centre=6540.77, sigma=0.62343, gamma=0.55, module='scipy')
pv4.name = r'Mn_II'
pv4.set_parameters_not_free(parameter_name_list=['centre', 'sigma', 'gamma',])
pv4.centre.bmin = 6540.0
pv4.centre.bmax = 6541.5
pv4.area.bmin = 0


In [ ]:
Mn_FD1st_ex = data.iloc[:, [20, 23]].copy().dropna(axis=0, how='all')

In [ ]:
from hyperspy.axes import DataAxis
energy_Mn_FD1st_ex = Mn_FD1st_ex.iloc[:, 0].to_numpy(dtype='float64')
energy_Mn_FD1st_ex = DataAxis(axis=energy_Mn_FD1st_ex, index_in_array=None, name='Energy', units='eV',)
FD1st_ex = hs.signals.Signal1D(Mn_FD1st_ex.iloc[:, 1], axes=[energy_Mn_FD1st_ex,])
FD1st_ex = FD1st_ex.isig[6532.0:].deepcopy()
FD1st_ex.plot()

In [ ]:
FD1st_ex_model = FD1st_ex.create_model()
FD1st_ex_model.extend((model_refMn_IV, pv1, pv2, pv3, pv4))
FD1st_ex_model.set_signal_range(6536.0, 6550.0)

In [ ]:
%matplotlib widget
plt.close('all')
FD1st_ex_model.fit(optimizer="lm", bounded=True, return_info=False) 
FD1st_ex_model.plot(plot_components=True, plot_residual=False)
plt.savefig(path.joinpath(path_out, r'PreEdge_25_300_V3_0.tif'), pad_inches=0.05, bbox_inches='tight', dpi=300, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'PreEdge_25_600_V3_0.tif'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
FD1st_ex_model.print_current_values()

In [ ]:
# 转化成 DataFrame
fit_Mn_IV = pd.DataFrame([FD1st_ex_model.axes_manager[0].axis, FD1st_ex_model.as_signal(component_list=['Mn_IV']).data])
fit_Mn_IV = fit_Mn_IV.T
fit_Mn_IV.dropna(how='any', inplace=True)

fit_Mn_III_1 = pd.DataFrame([FD1st_ex_model.axes_manager[0].axis, FD1st_ex_model.as_signal(component_list=['Mn_III_1']).data])
fit_Mn_III_1 = fit_Mn_III_1.T
fit_Mn_III_1.dropna(how='any', inplace=True)

fit_Mn_III_2 = pd.DataFrame([FD1st_ex_model.axes_manager[0].axis, FD1st_ex_model.as_signal(component_list=['Mn_III_2']).data])
fit_Mn_III_2 = fit_Mn_III_2.T
fit_Mn_III_2.dropna(how='any', inplace=True)

fit_Mn_III_3 = pd.DataFrame([FD1st_ex_model.axes_manager[0].axis, FD1st_ex_model.as_signal(component_list=['Mn_III_3']).data])
fit_Mn_III_3 = fit_Mn_III_3.T
fit_Mn_III_3.dropna(how='any', inplace=True)

fit_Mn_II = pd.DataFrame([FD1st_ex_model.axes_manager[0].axis, FD1st_ex_model.as_signal(component_list=['Mn_II']).data])
fit_Mn_II = fit_Mn_II.T
fit_Mn_II.dropna(how='any', inplace=True)

# 计算比例
pre_edge_ratio_Mn_IV = scipy.integrate.simpson(y=fit_Mn_IV.iloc[:, 1], x=fit_Mn_IV.iloc[:, 0])/(scipy.integrate.simpson(y=fit_Mn_IV.iloc[:, 1], x=fit_Mn_IV.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_III_1.iloc[:, 1], x=fit_Mn_III_1.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_III_2.iloc[:, 1], x=fit_Mn_III_2.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_III_3.iloc[:, 1], x=fit_Mn_III_3.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_II.iloc[:, 1], x=fit_Mn_II.iloc[:, 0]))
pre_edge_ratio_Mn_III = (scipy.integrate.simpson(y=fit_Mn_III_1.iloc[:, 1], x=fit_Mn_III_1.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_III_2.iloc[:, 1], x=fit_Mn_III_2.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_III_3.iloc[:, 1], x=fit_Mn_III_3.iloc[:, 0]))/(scipy.integrate.simpson(y=fit_Mn_IV.iloc[:, 1], x=fit_Mn_IV.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_III_1.iloc[:, 1], x=fit_Mn_III_1.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_III_2.iloc[:, 1], x=fit_Mn_III_2.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_III_3.iloc[:, 1], x=fit_Mn_III_3.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_II.iloc[:, 1], x=fit_Mn_II.iloc[:, 0]))
pre_edge_ratio_Mn_II = scipy.integrate.simpson(y=fit_Mn_II.iloc[:, 1], x=fit_Mn_II.iloc[:, 0])/(scipy.integrate.simpson(y=fit_Mn_IV.iloc[:, 1], x=fit_Mn_IV.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_III.iloc[:, 1], x=fit_Mn_III.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_II.iloc[:, 1], x=fit_Mn_II.iloc[:, 0]))
pre_edge_ratio_Mn_IV, pre_edge_ratio_Mn_III, pre_edge_ratio_Mn_II

In [ ]:
Pre_Edge_FD1st = xr.Dataset(
    data_vars=dict(
        FD1st = (['Energy_FD1st'], FD1st_ex.data),
        fit = (['Energy_fit'], FD1st_ex_model.as_signal().data),
        fit_Mn_IV = (['Energy_fit'], FD1st_ex_model.as_signal(component_list=['Mn_IV']).data),
        fit_Mn_III_1 = (['Energy_fit'], FD1st_ex_model.as_signal(component_list=['Mn_III_1']).data),
        fit_Mn_III_2 = (['Energy_fit'], FD1st_ex_model.as_signal(component_list=['Mn_III_2']).data),
        fit_Mn_III_3 = (['Energy_fit'], FD1st_ex_model.as_signal(component_list=['Mn_III_3']).data),
        fit_Mn_II = (['Energy_fit'], FD1st_ex_model.as_signal(component_list=['Mn_II']).data),
        ),
    coords=dict(
        Energy_FD1st = FD1st_ex.axes_manager[0].axis,
        Energy_fit= FD1st_ex_model.axes_manager[0].axis,
    ),
    attrs=dict(
        ratio_Mn_IV = pre_edge_ratio_Mn_IV,
        ratio_Mn_III = pre_edge_ratio_Mn_III,
        ratio_Mn_II = pre_edge_ratio_Mn_II,
        chisq = FD1st_ex_model.chisq.data[0],
        red_chisq = FD1st_ex_model.red_chisq.data[0],
    ),
)

FD1st_ex_model.save_parameters2file(path.joinpath(path_out, r'FD1st_ex_model_parameters_ref_1_voigt_3_voigt_1'))
FD1st_ex_model.save(path.joinpath(path_out, r'FD1st_ex_model_ref_1_voigt_3_voigt_1'), 'model', overwrite=True)
Pre_Edge_FD1st.to_netcdf(path.joinpath(path_out, r'FD1st_ex_result_ref_1_voigt_3_voigt_1.NETCDF4'), engine="h5netcdf")

In [ ]:
# 画图
%matplotlib inline
plt.close('all')
fig = plt.figure(figsize=(7, 2.5))
gs = gridspec.GridSpec(1, 2, width_ratios=[1, 1], height_ratios=None, wspace=0.0, hspace=0.0, figure=fig)

xas_colors_Mn = [colors[0], colors[4]]
xas_cmap_Mn = ListedColormap(mpl.colormaps['coolwarm'](np.linspace(0.5, 1.0, data.shape[1]//4-5)), name='xas_cmap_Mn', )
labels_Mn = [r'${\alpha}$-MnO$\mathrm{_2}$', r'0.2M Mn$\mathrm{^{2\!+}}$']

# 图 
subfig = fig.add_subfigure(gs[0, 0], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0, 0, 1.0, 1.0))
ax.set_box_aspect(0.8)
pad = 0.08
ax.plot(data.iloc[:, 4], data.iloc[:, 7]+pad, ls='-', color=colors[4], label=r'ref.MnO')
ax.plot(data.iloc[:, 0], data.iloc[:, 3]+pad, ls='-', color=colors[5], label=r'ref.Mn$\mathrm{_2O_3}$')
ax.plot(data.iloc[:, 8], data.iloc[:, 11]+pad, ls='-', color=colors[3], label=r'ref.Mn$\mathrm{O_2}$')
ax.legend(loc='upper left', bbox_to_anchor=(0.0, 1.02), ncols=1, frameon=False, fontsize=9, labelcolor='linecolor', columnspacing=0.0)

ax.plot(FD1st_ex.axes_manager[0].axis, FD1st_ex.data, ls='-', color='k', label=None)
ax.plot(fit_Mn_II.iloc[:, 0], fit_Mn_II.iloc[:, 1], ls='--', color=colors[4], label=None)
ax.plot(fit_Mn_IV.iloc[:, 0], fit_Mn_IV.iloc[:, 1], ls='--', color=colors[3], label=None)
ax.plot(fit_Mn_III_1.iloc[:, 0], fit_Mn_III_1.iloc[:, 1] + fit_Mn_III_2.iloc[:, 1] + fit_Mn_III_3.iloc[:, 1], ls='--', color=colors[5], label=None)
ax.plot(FD1st_ex_model.axes_manager[0].axis, FD1st_ex_model.as_signal().data, ls='--', color=colors[3], label=None)

ax.set_xlim(6535.0, 6550.0)
ax.set_xlabel(r'Energy (eV)', fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=5, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=2.5, offset=0))
ax.set_ylim(-0.01, 0.20)
ax.set_ylabel(r'Relative Intensity (arb.u.)', fontsize=11,)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.05))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.025))

# ax.text(-0.25, 1.0, r'b', transform=ax.transAxes, fontsize=14, va='center', ha='right', fontfamily='Arial', fontweight='bold')

ax.axvline(x=6540.3, ymax=0.65, ls='--')
ax.axvline(x=6543.0, ymax=0.7, ls='--')

plt.savefig(path.joinpath(path_out, r'PreEdge_26_300_V3_0.tif'), transparent=False, pad_inches=0.05, bbox_inches='tight', dpi=300, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'PreEdge_26_600_V3_0.tif'), transparent=False, pad_inches=0.05, bbox_inches='tight', dpi=600, pil_kwargs={"compression": "tiff_lzw"})
plt.show()

# V3

### Operando Data,除去 Mn2+ 的部分, pre-edge 数据获取

In [ ]:
# 读取数据
path_file = path(r'C:\Users\chengliu\OneDrive - UAB\ICMAB-Data\Zn-Mn\Uno\Result\XAS\Operando\αMnO2\XAS\CLAESS-2022-10\Results\cell3\Oct2022_cell3_P2a\XANES\Mn\TrialsOne_αMnO2_Pristine\MCR')
Mn_raw = xr.open_dataset(path.joinpath(path_file, r'Raw_data.NETCDF4'), engine='h5netcdf')
Mn_mcr_2_hm = xr.open_dataset(path.joinpath(path_file, r'PcaMcr_onereaction_2e_data.NETCDF4'), engine='h5netcdf')

In [ ]:
MnO2_rest = Mn_raw['Raw_spectrum'].sel(Energy=slice(Mn_mcr_2_hm['Energy'].data[0], Mn_mcr_2_hm['Energy'].data[-1])).data - (Mn_mcr_2_hm['MCR_HM_C'][:,1]*Mn_mcr_2_hm['MCR_HM_St'][1,:]).data.T

MnO2_Rest = xr.DataArray(
    data=MnO2_rest,
    coords={
        'Spectrum': range(MnO2_rest.shape[1]),
        'Energy': Mn_raw['Raw_spectrum'].sel(Energy=slice(Mn_mcr_2_hm['Energy'].data[0], Mn_mcr_2_hm['Energy'].data[-1]))['Energy'],
    },
    dims=['Energy', 'Spectrum'],
    name = r'MnO2_Rest',
)

#### PCA 降噪

In [ ]:
MnO2_rest = scp.NDDataset(data=MnO2_Rest.data.T,
                     name="opPre_Edge_Mn_Oct2022",
                     author="Cheng Liu, and Dino Tonti",
                     description="XANES obtained from CLAESS, alba, July2023",
                     history="creation",
                     title='absorption',
                     units=ur.absorbance)

MnO2_rest.y = scp.Coord.arange(MnO2_Rest['Spectrum'].shape[0], title='sperctum number')
MnO2_rest.x = scp.Coord(MnO2_Rest.coords['Energy'], title='Energy', units=ur.eV)

MnO2_rest_denoise = scp.denoise(MnO2_rest)
MnO2_Rest_Denoise = MnO2_rest_denoise.to_xarray().rename('MnO2_Rest_Denoise')
MnO2_Rest_Denoise.attrs['units'] = r'a.u.'
MnO2_rest_denoise.plot()

In [ ]:
# 保存数据
xr.merge([Mn_raw['Raw_spectrum'], Mn_raw['Raw_ref'], Mn_mcr_2_hm['MCR_HM_C'], Mn_mcr_2_hm['MCR_HM_St'], MnO2_Rest, MnO2_Rest_Denoise],).to_netcdf(path.joinpath(path_out, r'opPreEdge_Mn.NETCDF4'), engine="h5netcdf")

MnO2_Rest1 = MnO2_Rest.to_pandas()
MnO2_Rest1.columns = Mn_raw['Spectrum'].data
MnO2_Rest1.reset_index().to_csv(path.joinpath(path_out, r'opPreEdge_Mn_No_Mn_2.csv'), header=True, index=False)

MnO2_Rest_Denoise1 = MnO2_Rest_Denoise.to_pandas().T
MnO2_Rest_Denoise1.columns = Mn_raw['Spectrum'].data
MnO2_Rest_Denoise1.reset_index().to_csv(path.joinpath(path_out, r'opPreEdge_Mn_No_Mn_2_denoise.csv'), header=True, index=False)

In [ ]:
%matplotlib inline
plt.close('all')

fig = plt.figure(figsize=(7.0, 2.5))
gs = gridspec.GridSpec(1, 3, width_ratios=[1,1,1], height_ratios=None, wspace=0, hspace=0, figure=fig)

xascolors = [colors[0], colors[4]]
labels = [r'0.2M Mn$\mathrm{^{2\!+}}$', r'${\alpha}$MnO$\mathrm{_2}$']
Mn_cmap = ListedColormap(mpl.colormaps['sunset'](np.linspace(0.5, 1.0, MnO2_Rest.shape[1])), name='Mn_cmap', )

# 图 A
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_subplot()
ax.set_position([0, 0, 1.0, 1.0])
ax.set_box_aspect(0.8)

for i in range(2):
    ax.plot(Mn_raw['Energy'].sel(Energy=slice(6537.5, 6550.5)), Mn_raw['Raw_ref'].sel(Energy=slice(6537.5, 6550.5))[:, i], c=xascolors[i], ls='--', lw=1.0, label=labels[i], zorder=5)
ax.legend(loc='upper left', bbox_to_anchor=(0.0, 1.03), ncols=1, frameon=False, labelcolor='linecolor', fontsize=10)

for i in range(Mn_raw['Raw_spectrum'].shape[1]):
    ax.plot(Mn_raw['Energy'].sel(Energy=slice(6537.5, 6550.5)), Mn_raw['Raw_spectrum'].sel(Energy=slice(6537.5, 6550.5))[:, i], c=Mn_cmap.colors[i], ls='-', lw=1.0)
    
ax.set_xlim(6537.5, 6550.5)
ax.set_xlabel(r'Energy (eV)', fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=2, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=1, offset=0))
ax.set_ylim(0, 0.3)
ax.set_ylabel(r"Absorption (arb.u.)", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.06))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.03))
ax.tick_params(axis='both', labelsize=9) 

colorbar = Colorbar(ax=ax.inset_axes([0.65, 0.05, 0.3, 0.06]), location='bottom', orientation='horizontal', cmap=Mn_cmap, ticklocation='top', spacing = 'proportional', drawedges=False)
colorbar.set_ticks([])

ax.text(0.65, 0.18, 'OCV', transform=ax.transAxes, fontsize=9, va='top', ha='left', fontfamily='Arial', )
ax.text(0.86, 0.18, '0.9 V', transform=ax.transAxes, fontsize=9, va='top', ha='left', fontfamily='Arial', )

# 图 B
subfig = fig.add_subfigure(gs[0, 1])
ax = subfig.add_subplot()
ax.set_position([0.3, 0, 1.0, 1.0])
ax.set_box_aspect(0.8)

for i in range(2):
    ax.plot(Mn_raw['Energy'].sel(Energy=slice(6537.5, 6550.5)), Mn_raw['Raw_ref'].sel(Energy=slice(6537.5, 6550.5))[:, i], c=xascolors[i], ls='--', lw=1.0, label=labels[i], zorder=1)
ax.legend(loc='upper left', bbox_to_anchor=(0.0, 1.03), ncols=1, frameon=False, labelcolor='linecolor', fontsize=10)

for i in range(MnO2_Rest.shape[1]):
    ax.plot(MnO2_Rest['Energy'].sel(Energy=slice(6537.5, 6550.5)), MnO2_Rest.sel(Energy=slice(6537.5, 6550.5))[:, i], c=Mn_cmap.colors[i], ls='-', lw=1.0)
    
ax.set_xlim(6537.5, 6550.5)
ax.set_xlabel(r'Energy (eV)', fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=2, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=1, offset=0))
ax.set_ylim(0, 0.12)
ax.set_ylabel(r"Absorption (arb.u.)", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.03))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.015))
ax.tick_params(axis='both', labelsize=9) 

colorbar = Colorbar(ax=ax.inset_axes([0.65, 0.05, 0.3, 0.06]), location='bottom', orientation='horizontal', cmap=Mn_cmap, ticklocation='top', spacing = 'proportional', drawedges=False)
colorbar.set_ticks([])

ax.text(0.65, 0.18, 'OCV', transform=ax.transAxes, fontsize=9, va='top', ha='left', fontfamily='Arial', )
ax.text(0.86, 0.18, '0.9 V', transform=ax.transAxes, fontsize=9, va='top', ha='left', fontfamily='Arial', )

# 图 C
subfig = fig.add_subfigure(gs[0, 2])
ax = subfig.add_subplot()
ax.set_position([0.6, 0, 1.0, 1.0])
ax.set_box_aspect(0.8)

for i in range(2):
    ax.plot(Mn_raw['Energy'].sel(Energy=slice(6537.5, 6550.5)), Mn_raw['Raw_ref'].sel(Energy=slice(6537.5, 6550.5))[:, i], c=xascolors[i], ls='--', lw=1.0, label=labels[i], zorder=5)
ax.legend(loc='upper left', bbox_to_anchor=(0.0, 1.03), ncols=1, frameon=False, labelcolor='linecolor', fontsize=10)

for i in range(MnO2_Rest_Denoise.shape[0]):
    ax.plot(MnO2_Rest_Denoise['Energy'].sel(Energy=slice(6537.5, 6550.5)), MnO2_Rest_Denoise.sel(Energy=slice(6537.5, 6550.5))[i, :], c=Mn_cmap.colors[i], ls='-', lw=1.0)

ax.set_xlim(6537.5, 6550.5)
ax.set_xlabel(r'Energy (eV)', fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=2, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=1, offset=0))
ax.set_ylim(-0.01, 0.12)
ax.set_ylabel(r"Absorption (arb.u.)", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.03))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.015))
ax.tick_params(axis='both', labelsize=9) 

colorbar = Colorbar(ax=ax.inset_axes([0.65, 0.05, 0.3, 0.06]), location='bottom', orientation='horizontal', cmap=Mn_cmap, ticklocation='top', spacing = 'proportional', drawedges=False)
colorbar.set_ticks([])

ax.text(0.65, 0.18, 'OCV', transform=ax.transAxes, fontsize=9, va='top', ha='left', fontfamily='Arial', )
ax.text(0.86, 0.18, '0.9 V', transform=ax.transAxes, fontsize=9, va='top', ha='left', fontfamily='Arial', )

plt.savefig(path.joinpath(path_out, r'PreEdge_1_300_V3_0.tif'), transparent=False, pad_inches=0.05, bbox_inches='tight', dpi=300, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'PreEdge_1_600_V3_0.tif'), transparent=False, pad_inches=0.05, bbox_inches='tight', dpi=600, pil_kwargs={"compression": "tiff_lzw"})
plt.show()

#### 找到所有的 data, 并除去背景

In [ ]:
def index(energy, value):
    return np.argmin(abs(energy-value))

In [ ]:
root_dir = path(r'C:\Users\chengliu\OneDrive - UAB\ICMAB-Data\Zn-Mn\Uno\Result\XAS\Operando\αMnO2\XAS\CLAESS-2022-10\Results\cell3\Oct2022_cell3_P2a\PreEdge\PreEdge\V3\2-bg_Remove\MnO')
dat_files = list(root_dir.rglob('*.dat'))

df=[]
for file in dat_files:
    data = pd.read_csv(file, sep='\s+', skiprows=0, comment='#', header=0, index_col=None, usecols=[0, 1, 2, 3])
    data_baseline = data.iloc[:, 2:4].copy().dropna(axis=0, how='any', ignore_index=True, inplace=False)
    data = data.iloc[:, 0:2].copy().dropna(axis=0, how='any', ignore_index=True, inplace=False)
    data = data.truncate(before=index(data.iloc[:, 0], data_baseline.iloc[0, 0]), after=index(data.iloc[:, 0], data_baseline.iloc[-1, 0]), axis=0, copy=True).reset_index(drop=True)
    data2 = data.iloc[:,1] - data_baseline.iloc[:,1]
    df.append(data)
    df.append(data_baseline.iloc[:,1])
    df.append(data2)  
data = pd.concat(df, ignore_index=True, axis=1)
data.to_csv(path.joinpath(path_out, r'PreEdge.csv'), index=None, header=True)

dat_files

#### 读取数据

In [ ]:
path_file = path(r'C:\Users\chengliu\OneDrive - UAB\ICMAB-Data\Zn-Mn\Uno\Result\XAS\Operando\αMnO2\XAS\CLAESS-2022-10\Results\cell3\Oct2022_cell3_P2a\PreEdge\PreEdge\V3\PreEdge.csv')
data = pd.read_csv(path_file, sep=',', comment='#', header=0, index_col=None)
data

In [ ]:
# 画图
%matplotlib inline
fig = plt.figure(figsize=(7, 2.5))
gs = gridspec.GridSpec(1, 2, width_ratios=[1, 1], height_ratios=None, wspace=0.0, hspace=0.0, figure=fig)

xas_colors_Mn = [colors[0], colors[4]]
xas_cmap_Mn = ListedColormap(mpl.colormaps['coolwarm'](np.linspace(0.5, 1.0, data.shape[1]//4-6)), name='xas_cmap_Mn', )
labels_Mn = [r'${\alpha}$-MnO$\mathrm{_2}$', r'0.2M Mn$\mathrm{^{2\!+}}$']

# 图 B
subfig = fig.add_subfigure(gs[0, 0], zorder=0)
ax = subfig.add_subplot()
ax.set_position([0, 0, 1.0, 1.0])
ax.set_box_aspect(0.8)

ax.plot(data.iloc[:, 4], data.iloc[:, 7]+0.06, ls='-', color=colors[4], label=r'ref.MnO')
ax.plot(data.iloc[:, 0], data.iloc[:, 3]+0.06, ls='-', color=colors[5], label=r'ref.Mn$\mathrm{_2O_3}$')
ax.plot(data.iloc[:, 8], data.iloc[:, 11]+0.06, ls='-', color=colors[3], label=r'ref.Mn$\mathrm{O_2}$')
ax.plot(data.iloc[:, 16], data.iloc[:, 19]+0.06, ls='-', color=colors[4], label=r'ref.Mn$\mathrm{^{2+}}$')
ax.plot(data.iloc[:, 12], data.iloc[:, 15]+0.06, ls='-', color=colors[3], label=r'ref.$\mathrm{\alpha}$Mn$\mathrm{O_2}$')

ax.legend(loc='upper left', bbox_to_anchor=(0.0, 1.02), ncols=1, frameon=False, fontsize=9, labelcolor='linecolor', columnspacing=0.0)

for i in range(data.shape[1]//4-6):
    ax.plot(data.iloc[:, i*4+24], data.iloc[:, i*4+27], c=xas_cmap_Mn.colors[i], ls='-', label=None)

ax.set_xlim(6530.0, 6550.0)
ax.set_xlabel(r'Energy (eV)', fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=5, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=2.5, offset=0))
ax.set_ylim(-0.01, 0.20)
ax.set_ylabel(r'Relative Intensity (arb.u.)', fontsize=11,)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.05))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.025))

# Add colorbar and adjust ticks afterwards
colorbar = Colorbar(ax=ax.inset_axes([0.5, 0.8, 0.4, 0.07]), location='bottom', orientation='horizontal', 
                      cmap=xas_cmap_Mn, ticklocation='top', spacing = 'proportional', drawedges=False)
colorbar.set_ticks([])
colorbar.outline.set_visible(False)
ax.text(0.5, 0.94, 'OCV', transform=ax.transAxes, fontsize=9,
        va='top', ha='left', fontfamily='Arial', )
ax.text(0.65, 0.94, r'Discharge', transform=ax.transAxes, fontsize=9,
        va='top', ha='left', fontfamily='Arial', )

# ax.text(-0.25, 1.0, r'b', transform=ax.transAxes, fontsize=14, va='center', ha='right', fontfamily='Arial', fontweight='bold')

ax.axvline(x=6540.3, ymax=0.75, ls='--')
ax.axvline(x=6543.0, ymax=0.75, ls='--')

plt.savefig(path.joinpath(path_out, r'PreEdge_2_300_V3_0.tif'), transparent=False, pad_inches=0.05, bbox_inches='tight', dpi=300, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'PreEdge_2_600_V3_0.tif'), transparent=False, pad_inches=0.05, bbox_inches='tight', dpi=600, pil_kwargs={"compression": "tiff_lzw"})
plt.show()

### Mn2+ Solution

#### 拟合峰形, pseudo-Voigt 峰 - 两个

In [ ]:
pv1 = hs.model.components1D.SplitVoigt(centre=6540.3, sigma1=0.83, sigma2=1.1, fraction=0.45)
pv1.name = r'Mn_II_1'
# pv1.set_parameters_not_free(parameter_name_list=['sigma1', 'sigma2', 'fraction'])
pv1.set_parameters_not_free(parameter_name_list=['fraction', 'sigma2',])
pv1.centre.bmin = 6537.5
pv1.centre.bmax = 6542.5
pv1.A.bmin = 0

pv2 = hs.model.components1D.SplitVoigt(centre=6540.0, sigma1=0.83, sigma2=1.1, fraction=0.45)
pv2.name = r'Mn_II_2'
pv2.centre.bmin = 6537.5
pv2.centre.bmax = 6542.5
pv2.A.bmin = 0
pv2.sigma1.twin = pv1.sigma1
pv2.sigma2.twin = pv1.sigma2
pv2.fraction.twin = pv1.fraction

In [ ]:
Mn_Mn_2 = data.iloc[:, [16, 19]].copy().dropna(axis=0, how='all')

In [ ]:
from hyperspy.axes import DataAxis
energy_Mn_2 = Mn_Mn_2.iloc[:, 0].to_numpy(dtype='float64')
energy_Mn_2 = DataAxis(axis=energy_Mn_2, index_in_array=None, name='Energy', units='eV',)
Mn_2 = hs.signals.Signal1D(Mn_Mn_2.iloc[:, 1], axes=[energy_Mn_2,])
Mn_2 = Mn_2.isig[6530.0:].deepcopy()
Mn_2.plot()

In [ ]:
Mn_2_model = Mn_2.create_model()
Mn_2_model.extend((pv1, pv2))
Mn_2_model.set_signal_range(6538.0, 6544.0)

In [ ]:
%matplotlib inline
plt.close('all')
Mn_2_model.fit(optimizer="lm", bounded=True, return_info=False) 
Mn_2_model.plot(plot_components=True, plot_residual=True)
plt.savefig(path.joinpath(path_out, r'PreEdge_3_300_V3_0.tif'), pad_inches=0.05, bbox_inches='tight', dpi=300, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'PreEdge_3_600_V3_0.tif'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
Mn_2_model.print_current_values()

In [ ]:
# 转化成 DataFrame
fit_Mn_II_1 = pd.DataFrame([Mn_2_model.axes_manager[0].axis, Mn_2_model.as_signal(component_list=['Mn_II_1']).data])
fit_Mn_II_1 = fit_Mn_II_1.T
fit_Mn_II_1.dropna(how='any', inplace=True)

fit_Mn_II_2 = pd.DataFrame([Mn_2_model.axes_manager[0].axis, Mn_2_model.as_signal(component_list=['Mn_II_2']).data])
fit_Mn_II_2 = fit_Mn_II_2.T
fit_Mn_II_2.dropna(how='any', inplace=True)

In [ ]:
Pre_Edge_Mn_2 = xr.Dataset(
    data_vars=dict(
        Mn_2 = (['Energy_Mn_2'], Mn_2.data),
        fit = (['Energy_fit'], Mn_2_model.as_signal().data),
        fit_Mn_II_1 = (['Energy_fit'], Mn_2_model.as_signal(component_list=['Mn_II_1']).data),
        fit_Mn_II_2 = (['Energy_fit'], Mn_2_model.as_signal(component_list=['Mn_II_2']).data),
        ),
    coords=dict(
        Energy_Mn_2 = Mn_2.axes_manager[0].axis,
        Energy_fit= Mn_2_model.axes_manager[0].axis,
    ),
)

Mn_2_model.save_parameters2file(path.joinpath(path_out, r'Mn_2_model_parameters_component_pv_2'))
Mn_2_model.save(path.joinpath(path_out, r'Mn_2_model_parameters_component_pv_2'), 'model', overwrite=True)
Pre_Edge_Mn_2.to_netcdf(path.joinpath(path_out, r'Mn_2_component_pv_2.NETCDF4'), engine="h5netcdf")

In [ ]:
# 画图
%matplotlib inline
fig = plt.figure(figsize=(7, 2.5))
gs = gridspec.GridSpec(1, 2, width_ratios=[1, 1], height_ratios=None, wspace=0.0, hspace=0.0, figure=fig)

xas_colors_Mn = [colors[0], colors[4]]
xas_cmap_Mn = ListedColormap(mpl.colormaps['coolwarm'](np.linspace(0.5, 1.0, data.shape[1]//4-5)), name='xas_cmap_Mn', )
labels_Mn = [r'${\alpha}$-MnO$\mathrm{_2}$', r'0.2M Mn$\mathrm{^{2\!+}}$']

# 图 B
subfig = fig.add_subfigure(gs[0, 0], zorder=0)
ax = subfig.add_subplot()
ax.set_position([0, 0, 1.0, 1.0])
ax.set_box_aspect(0.8)

ax.plot(data.iloc[:, 4], data.iloc[:, 7]+0.08, ls='-', color=colors[4], label=r'ref.MnO')
ax.plot(data.iloc[:, 0], data.iloc[:, 3]+0.08, ls='-', color=colors[5], label=r'ref.Mn$\mathrm{_2O_3}$')
ax.plot(data.iloc[:, 8], data.iloc[:, 11]+0.08, ls='-', color=colors[3], label=r'ref.Mn$\mathrm{O_2}$')
ax.legend(loc='upper left', bbox_to_anchor=(0.0, 1.02), ncols=1, frameon=False, fontsize=9, labelcolor='linecolor', columnspacing=0.0)

ax.plot(data.iloc[:, 16], data.iloc[:, 19], ls='-', color='k', label=None)
# ax.plot(fit_Mn_II_2.iloc[:, 0], fit_Mn_II_2.iloc[:, 1], ls='--', color=colors[4], label=None)
# ax.plot(fit_Mn_II_1.iloc[:, 0], fit_Mn_II_1.iloc[:, 1], ls='--', color=colors[4], label=None)
ax.plot(fit_Mn_II_1.iloc[:, 0], fit_Mn_II_1.iloc[:, 1] + fit_Mn_II_2.iloc[:, 1], ls='--', color=colors[4], label=None)
# ax.plot(Mn_2_model.axes_manager[0].axis, Mn_2_model.as_signal().data, ls='--', color=colors[3], label=None)

ax.set_xlim(6535.0, 6550.0)
ax.set_xlabel(r'Energy (eV)', fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=5, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=2.5, offset=0))
ax.set_ylim(-0.01, 0.20)
ax.set_ylabel(r'Relative Intensity (arb.u.)', fontsize=11,)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.05))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.025))

# ax.text(-0.25, 1.0, r'b', transform=ax.transAxes, fontsize=14, va='center', ha='right', fontfamily='Arial', fontweight='bold')

ax.axvline(x=6540.3, ymax=0.65, ls='--')
ax.axvline(x=6543.0, ymax=0.7, ls='--')

plt.savefig(path.joinpath(path_out, r'PreEdge_4_300_V3_0.tif'), transparent=False, pad_inches=0.05, bbox_inches='tight', dpi=300, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'PreEdge_4_600_V3_0.tif'), transparent=False, pad_inches=0.05, bbox_inches='tight', dpi=600, pil_kwargs={"compression": "tiff_lzw"})
plt.show()

### 拟合 Mn(III) Reference

#### 拟合峰形, pseudo-Voigt 峰 - 三个

In [ ]:
pv1 = hs.model.components1D.SplitVoigt(centre=6540.7, sigma1=0.83, sigma2=1.1, fraction=0.45)
pv1.name = r'Mn_III_1'
# pv1.set_parameters_not_free(parameter_name_list=['sigma1', 'sigma2', 'fraction'])
pv1.set_parameters_not_free(parameter_name_list=['sigma2', 'fraction'])
pv1.centre.bmin = 6538.0
pv1.centre.bmax = 6542.5
pv1.A.bmin = 0

pv2 = hs.model.components1D.SplitVoigt(centre=6543.5, sigma1=0.83, sigma2=1.1, fraction=0.45)
pv2.name = r'Mn_III_2'
pv2.centre.bmin = 6543.0
pv2.centre.bmax = 6545.0
pv2.A.bmin = 0
pv2.sigma1.twin = pv1.sigma1
pv2.sigma2.twin = pv1.sigma2
pv2.fraction.twin = pv1.fraction

pv3 = hs.model.components1D.SplitVoigt(centre=6542.3, sigma1=0.83, sigma2=1.1, fraction=0.45)
pv3.name = r'Mn_III_3'
# pv3.centre.bmin = pv1.centre.bmax - 0.5
# pv3.centre.bmax = pv2.centre.bmin + 0.5
pv3.A.bmin = 0
pv3.sigma1.twin = pv1.sigma1
pv3.sigma2.twin = pv1.sigma2
pv3.fraction.twin = pv1.fraction

In [ ]:
Mn_Mn_III = data.iloc[:, [0, 3]].copy().dropna(axis=0, how='all')

In [ ]:
from hyperspy.axes import DataAxis
energy_Mn_Mn_III = Mn_Mn_III.iloc[:, 0].to_numpy(dtype='float64')
energy_Mn_Mn_III = DataAxis(axis=energy_Mn_Mn_III, index_in_array=None, name='Energy', units='eV',)
Mn_III = hs.signals.Signal1D(Mn_Mn_III.iloc[:, 1], axes=[energy_Mn_Mn_III,])
Mn_III = Mn_III.isig[6535.0:].deepcopy()
Mn_III.plot()

In [ ]:
Mn_III_model = Mn_III.create_model()
Mn_III_model.extend((pv1, pv2, pv3))
Mn_III_model.set_signal_range(6537.0, 6548.0)

In [ ]:
%matplotlib inline
plt.close('all')
Mn_III_model.fit(optimizer="lm", bounded=True, return_info=False) 
Mn_III_model.plot(plot_components=True, plot_residual=True)
plt.savefig(path.joinpath(path_out, r'PreEdge_5_300_V3_0.tif'), pad_inches=0.05, bbox_inches='tight', dpi=300, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'PreEdge_5_600_V3_0.tif'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
Mn_III_model.print_current_values()

In [ ]:
# 转化成 DataFrame
fit_Mn_III_1 = pd.DataFrame([Mn_III_model.axes_manager[0].axis, Mn_III_model.as_signal(component_list=['Mn_III_1']).data])
fit_Mn_III_1 = fit_Mn_III_1.T
fit_Mn_III_1.dropna(how='any', inplace=True)

fit_Mn_III_2 = pd.DataFrame([Mn_III_model.axes_manager[0].axis, Mn_III_model.as_signal(component_list=['Mn_III_2']).data])
fit_Mn_III_2 = fit_Mn_III_2.T
fit_Mn_III_2.dropna(how='any', inplace=True)

fit_Mn_III_3 = pd.DataFrame([Mn_III_model.axes_manager[0].axis, Mn_III_model.as_signal(component_list=['Mn_III_3']).data])
fit_Mn_III_3 = fit_Mn_III_3.T
fit_Mn_III_3.dropna(how='any', inplace=True)

In [ ]:
Pre_Edge_Mn_III = xr.Dataset(
    data_vars=dict(
        Mn_III = (['Energy_Mn_III'], Mn_III.data),
        fit = (['Energy_fit'], Mn_III_model.as_signal().data),
        fit_Mn_III_1 = (['Energy_fit'], Mn_III_model.as_signal(component_list=['Mn_III_1']).data),
        fit_Mn_III_2 = (['Energy_fit'], Mn_III_model.as_signal(component_list=['Mn_III_2']).data),
        fit_Mn_III_3 = (['Energy_fit'], Mn_III_model.as_signal(component_list=['Mn_III_3']).data),
        ),
    coords=dict(
        Energy_Mn_III = Mn_III.axes_manager[0].axis,
        Energy_fit= Mn_III_model.axes_manager[0].axis,
    ),
)

Mn_III_model.save_parameters2file(path.joinpath(path_out, r'Ref_Mn_III_model_parameters_component_pv_3'))
Mn_III_model.save(path.joinpath(path_out, r'Ref_Mn_III_model_parameters_component_pv_3'), 'model', overwrite=True)
Pre_Edge_Mn_III.to_netcdf(path.joinpath(path_out, r'Ref_Mn_III_component_pv_3.NETCDF4'), engine="h5netcdf")

In [ ]:
# 画图
%matplotlib inline
fig = plt.figure(figsize=(7, 2.5))
gs = gridspec.GridSpec(1, 2, width_ratios=[1, 1], height_ratios=None, wspace=0.0, hspace=0.0, figure=fig)

xas_colors_Mn = [colors[0], colors[4]]
xas_cmap_Mn = ListedColormap(mpl.colormaps['coolwarm'](np.linspace(0.5, 1.0, data.shape[1]//4-5)), name='xas_cmap_Mn', )
labels_Mn = [r'${\alpha}$-MnO$\mathrm{_2}$', r'0.2M Mn$\mathrm{^{2\!+}}$']

# 图 B
subfig = fig.add_subfigure(gs[0, 0], zorder=0)
ax = subfig.add_subplot()
ax.set_position([0, 0, 1.0, 1.0])
ax.set_box_aspect(0.8)

ax.plot(data.iloc[:, 4], data.iloc[:, 7]+0.08, ls='-', color=colors[4], label=r'ref.MnO')
ax.plot(data.iloc[:, 0], data.iloc[:, 3]+0.08, ls='-', color=colors[5], label=r'ref.Mn$\mathrm{_2O_3}$')
ax.plot(data.iloc[:, 8], data.iloc[:, 11]+0.08, ls='-', color=colors[3], label=r'ref.Mn$\mathrm{O_2}$')
ax.legend(loc='upper left', bbox_to_anchor=(0.0, 1.02), ncols=1, frameon=False, fontsize=9, labelcolor='linecolor', columnspacing=0.0)

ax.plot(Mn_III.axes_manager[0].axis, Mn_III.data, ls='-', color='k', label=None)
# ax.plot(fit_Mn_III_3.iloc[:, 0], fit_Mn_III_3.iloc[:, 1], ls='--', color=colors[5], label=None)
# ax.plot(fit_Mn_III_2.iloc[:, 0], fit_Mn_III_2.iloc[:, 1], ls='--', color=colors[5], label=None)
# ax.plot(fit_Mn_III_1.iloc[:, 0], fit_Mn_III_1.iloc[:, 1], ls='--', color=colors[5], label=None)
ax.plot(fit_Mn_III_1.iloc[:, 0], fit_Mn_III_1.iloc[:, 1] + fit_Mn_III_2.iloc[:, 1] + fit_Mn_III_3.iloc[:, 1], ls='--', color=colors[5], label=None)
ax.plot(Mn_III_model.axes_manager[0].axis, Mn_III_model.as_signal().data, ls='--', color=colors[3], label=None)

ax.set_xlim(6535.0, 6550.0)
ax.set_xlabel(r'Energy (eV)', fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=5, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=2.5, offset=0))
ax.set_ylim(-0.01, 0.20)
ax.set_ylabel(r'Relative Intensity (arb.u.)', fontsize=11,)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.05))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.025))

# ax.text(-0.25, 1.0, r'b', transform=ax.transAxes, fontsize=14, va='center', ha='right', fontfamily='Arial', fontweight='bold')

ax.axvline(x=6540.3, ymax=0.65, ls='--')
ax.axvline(x=6543.0, ymax=0.7, ls='--')

plt.savefig(path.joinpath(path_out, r'PreEdge_6_300_V3_0.tif'), transparent=False, pad_inches=0.05, bbox_inches='tight', dpi=300, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'PreEdge_6_600_V3_0.tif'), transparent=False, pad_inches=0.05, bbox_inches='tight', dpi=600, pil_kwargs={"compression": "tiff_lzw"})
plt.show()

### 验证 Prisitine alpha MnO2

#### 1.直接做 References, II + III + IV

In [ ]:
# 数据清洗
Mn_IV = data.iloc[:, [8, 11]].copy().dropna(axis=0, how='all')
Mn_III = data.iloc[:, [0, 3]].copy().dropna(axis=0, how='all')
Mn_II = data.iloc[:, [4, 7]].copy().dropna(axis=0, how='all')

Mn_IV.iloc[115:, 1] = 0
Mn_III.iloc[97:, 1] = 0
Mn_II.iloc[58:, 1] = 0

# 将 refMn_I//III/IV 做成一个 scale model
from hyperspy.axes import DataAxis
energy_refMn_IV = Mn_IV.iloc[:, 0].to_numpy(dtype='float64')
energy_refMn_IV = DataAxis(axis=energy_refMn_IV, index_in_array=None, name='Energy', units='eV',)
refMn_IV = hs.signals.Signal1D(Mn_IV.iloc[:, 1], axes=[energy_refMn_IV,])
model_refMn_IV = hs.model.components1D.ScalableFixedPattern(refMn_IV) 

energy_refMn_III = Mn_III.iloc[:, 0].to_numpy(dtype='float64')
energy_refMn_III = DataAxis(axis=energy_refMn_III, index_in_array=None, name='Energy', units='eV',)
refMn_III = hs.signals.Signal1D(Mn_III.iloc[:, 1], axes=[energy_refMn_III,])
model_refMn_III = hs.model.components1D.ScalableFixedPattern(refMn_III) 

energy_refMn_II = Mn_II.iloc[:, 0].to_numpy(dtype='float64')
energy_refMn_II = DataAxis(axis=energy_refMn_II, index_in_array=None, name='Energy', units='eV',)
refMn_II = hs.signals.Signal1D(Mn_II.iloc[:, 1], axes=[energy_refMn_II,])
model_refMn_II = hs.model.components1D.ScalableFixedPattern(refMn_II) 

# 初始化上述的模型
model_refMn_IV.name= r'Mn_IV'
model_refMn_IV.xscale.value=1.0
model_refMn_IV.shift.value=0
# model_refMn_IV.set_parameters_not_free(parameter_name_list=['xscale', 'shift',])
model_refMn_IV.yscale.bmin=0.0

model_refMn_III.name= r'Mn_III'
model_refMn_III.xscale.value=1.0
model_refMn_III.shift.value=0
# model_refMn_III.set_parameters_not_free(parameter_name_list=['xscale', 'shift',])
model_refMn_III.yscale.bmin=0.0

model_refMn_II.name= r'Mn_II'
model_refMn_II.xscale.value=1.0
model_refMn_II.shift.value=0
# model_refMn_II.set_parameters_not_free(parameter_name_list=['xscale', 'shift',])
model_refMn_II.yscale.bmin=0.0

In [ ]:
Mn_aMnO2 = data.iloc[:, [12, 15]].copy().dropna(axis=0, how='all')

In [ ]:
from hyperspy.axes import DataAxis
energy_aMnO2 = Mn_aMnO2.iloc[:, 0].to_numpy(dtype='float64')
energy_aMnO2 = DataAxis(axis=energy_aMnO2, index_in_array=None, name='Energy', units='eV',)
aMnO2 = hs.signals.Signal1D(Mn_aMnO2.iloc[:, 1], axes=[energy_aMnO2,])
aMnO2 = aMnO2.isig[6535.0:].deepcopy()
aMnO2.plot()

In [ ]:
aMnO2_model = aMnO2.create_model()
aMnO2_model.extend((model_refMn_II, model_refMn_III, model_refMn_IV))
aMnO2_model.set_signal_range(6536.0, 6550.0)

In [ ]:
%matplotlib inline
plt.close('all')
aMnO2_model.fit(optimizer="lm", bounded=True, return_info=False) 
aMnO2_model.plot(plot_components=True, plot_residual=False)
plt.savefig(path.joinpath(path_out, r'PreEdge_7_300_V3_0.tif'), pad_inches=0.05, bbox_inches='tight', dpi=300, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'PreEdge_7_600_V3_0.tif'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
aMnO2_model.print_current_values()

In [ ]:
# 转化成 DataFrame
fit_Mn_IV = pd.DataFrame([aMnO2_model.axes_manager[0].axis, aMnO2_model.as_signal(component_list=['Mn_IV']).data])
fit_Mn_IV = fit_Mn_IV.T
fit_Mn_IV.dropna(how='any', inplace=True)

fit_Mn_III = pd.DataFrame([aMnO2_model.axes_manager[0].axis, aMnO2_model.as_signal(component_list=['Mn_III']).data])
fit_Mn_III = fit_Mn_III.T
fit_Mn_III.dropna(how='any', inplace=True)

fit_Mn_II = pd.DataFrame([aMnO2_model.axes_manager[0].axis, aMnO2_model.as_signal(component_list=['Mn_II']).data])
fit_Mn_II = fit_Mn_II.T
fit_Mn_II.dropna(how='any', inplace=True)

# 计算比例
pre_edge_ratio_Mn_IV = scipy.integrate.simpson(y=fit_Mn_IV.iloc[:, 1], x=fit_Mn_IV.iloc[:, 0])/(scipy.integrate.simpson(y=fit_Mn_IV.iloc[:, 1], x=fit_Mn_IV.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_III.iloc[:, 1], x=fit_Mn_III.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_II.iloc[:, 1], x=fit_Mn_II.iloc[:, 0]))
pre_edge_ratio_Mn_III = scipy.integrate.simpson(y=fit_Mn_III.iloc[:, 1], x=fit_Mn_III.iloc[:, 0])/(scipy.integrate.simpson(y=fit_Mn_IV.iloc[:, 1], x=fit_Mn_IV.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_III.iloc[:, 1], x=fit_Mn_III.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_II.iloc[:, 1], x=fit_Mn_II.iloc[:, 0]))
pre_edge_ratio_Mn_II = scipy.integrate.simpson(y=fit_Mn_II.iloc[:, 1], x=fit_Mn_II.iloc[:, 0])/(scipy.integrate.simpson(y=fit_Mn_IV.iloc[:, 1], x=fit_Mn_IV.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_III.iloc[:, 1], x=fit_Mn_III.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_II.iloc[:, 1], x=fit_Mn_II.iloc[:, 0]))
pre_edge_ratio_Mn_IV, pre_edge_ratio_Mn_III, pre_edge_ratio_Mn_II

In [ ]:
Pre_Edge_aMnO2 = xr.Dataset(
    data_vars=dict(
        aMnO2 = (['Energy_aMnO2'], aMnO2.data),
        fit = (['Energy_fit'], aMnO2_model.as_signal().data),
        fit_Mn_IV = (['Energy_fit'], aMnO2_model.as_signal(component_list=['Mn_IV']).data),
        fit_Mn_III = (['Energy_fit'], aMnO2_model.as_signal(component_list=['Mn_III']).data),
        fit_Mn_II = (['Energy_fit'], aMnO2_model.as_signal(component_list=['Mn_II']).data),
        ),
    coords=dict(
        Energy_aMnO2 = aMnO2.axes_manager[0].axis,
        Energy_fit= aMnO2_model.axes_manager[0].axis,
    ),
    attrs=dict(
        ratio_Mn_IV = pre_edge_ratio_Mn_IV,
        ratio_Mn_III = pre_edge_ratio_Mn_III,
        ratio_Mn_II = pre_edge_ratio_Mn_II,
    ),
)

aMnO2_model.save_parameters2file(path.joinpath(path_out, r'Prisitne_aMnO2_model_parameters_component_ref_3'))
aMnO2_model.save(path.joinpath(path_out, r'Prisitne_aMnO2_model_parameters_component_ref_3'), 'model', overwrite=True)
Pre_Edge_aMnO2.to_netcdf(path.joinpath(path_out, r'Prisitne_aMnO2_component_ref_3.NETCDF4'), engine="h5netcdf")

In [ ]:
# 画图
%matplotlib inline
fig = plt.figure(figsize=(7, 2.5))
gs = gridspec.GridSpec(1, 2, width_ratios=[1, 1], height_ratios=None, wspace=0.0, hspace=0.0, figure=fig)

xas_colors_Mn = [colors[0], colors[4]]
xas_cmap_Mn = ListedColormap(mpl.colormaps['coolwarm'](np.linspace(0.5, 1.0, data.shape[1]//4-5)), name='xas_cmap_Mn', )
labels_Mn = [r'${\alpha}$-MnO$\mathrm{_2}$', r'0.2M Mn$\mathrm{^{2\!+}}$']

# 图 B
subfig = fig.add_subfigure(gs[0, 0], zorder=0)
ax = subfig.add_subplot()
ax.set_position([0, 0, 1.0, 1.0])
ax.set_box_aspect(0.8)

ax.plot(data.iloc[:, 4], data.iloc[:, 7]+0.08, ls='-', color=colors[4], label=r'ref.MnO')
ax.plot(data.iloc[:, 0], data.iloc[:, 3]+0.08, ls='-', color=colors[5], label=r'ref.Mn$\mathrm{_2O_3}$')
ax.plot(data.iloc[:, 8], data.iloc[:, 11]+0.08, ls='-', color=colors[3], label=r'ref.Mn$\mathrm{O_2}$')
ax.legend(loc='upper left', bbox_to_anchor=(0.0, 1.02), ncols=1, frameon=False, fontsize=9, labelcolor='linecolor', columnspacing=0.0)

ax.plot(aMnO2.axes_manager[0].axis, aMnO2.data, ls='-', color='k', label=None)
ax.plot(fit_Mn_II.iloc[:, 0], fit_Mn_II.iloc[:, 1], ls='--', color=colors[4], label=None)
ax.plot(fit_Mn_III.iloc[:, 0], fit_Mn_III.iloc[:, 1], ls='--', color=colors[5], label=None)
ax.plot(fit_Mn_IV.iloc[:, 0], fit_Mn_IV.iloc[:, 1], ls='--', color=colors[3], label=None)
ax.plot(aMnO2_model.axes_manager[0].axis, aMnO2_model.as_signal().data, ls='--', color=colors[3], label=None)

ax.set_xlim(6535.0, 6550.0)
ax.set_xlabel(r'Energy (eV)', fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=5, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=2.5, offset=0))
ax.set_ylim(-0.01, 0.20)
ax.set_ylabel(r'Relative Intensity (arb.u.)', fontsize=11,)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.05))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.025))

# ax.text(-0.25, 1.0, r'b', transform=ax.transAxes, fontsize=14, va='center', ha='right', fontfamily='Arial', fontweight='bold')

ax.axvline(x=6540.3, ymax=0.65, ls='--')
ax.axvline(x=6543.0, ymax=0.7, ls='--')

plt.savefig(path.joinpath(path_out, r'PreEdge_8_300_V3_0.tif'), transparent=False, pad_inches=0.05, bbox_inches='tight', dpi=300, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'PreEdge_8_600_V3_0.tif'), transparent=False, pad_inches=0.05, bbox_inches='tight', dpi=600, pil_kwargs={"compression": "tiff_lzw"})
plt.show()

#### 1.直接做 References, III + IV

In [ ]:
# 数据清洗
Mn_IV = data.iloc[:, [8, 11]].copy().dropna(axis=0, how='all')
Mn_III = data.iloc[:, [0, 3]].copy().dropna(axis=0, how='all')

Mn_IV.iloc[115:, 1] = 0
Mn_III.iloc[97:, 1] = 0

# 将 refMn_I//III/IV 做成一个 scale model
from hyperspy.axes import DataAxis
energy_refMn_IV = Mn_IV.iloc[:, 0].to_numpy(dtype='float64')
energy_refMn_IV = DataAxis(axis=energy_refMn_IV, index_in_array=None, name='Energy', units='eV',)
refMn_IV = hs.signals.Signal1D(Mn_IV.iloc[:, 1], axes=[energy_refMn_IV,])
model_refMn_IV = hs.model.components1D.ScalableFixedPattern(refMn_IV) 

energy_refMn_III = Mn_III.iloc[:, 0].to_numpy(dtype='float64')
energy_refMn_III = DataAxis(axis=energy_refMn_III, index_in_array=None, name='Energy', units='eV',)
refMn_III = hs.signals.Signal1D(Mn_III.iloc[:, 1], axes=[energy_refMn_III,])
model_refMn_III = hs.model.components1D.ScalableFixedPattern(refMn_III) 

# 初始化上述的模型
model_refMn_IV.name= r'Mn_IV'
model_refMn_IV.xscale.value=1.0
model_refMn_IV.shift.value=0
# model_refMn_IV.set_parameters_not_free(parameter_name_list=['xscale', 'shift',])
model_refMn_IV.yscale.bmin=0.0

model_refMn_III.name= r'Mn_III'
model_refMn_III.xscale.value=1.0
model_refMn_III.shift.value=0
# model_refMn_III.set_parameters_not_free(parameter_name_list=['xscale', 'shift',])
model_refMn_III.yscale.bmin=0.0

In [ ]:
Mn_aMnO2 = data.iloc[:, [12, 15]].copy().dropna(axis=0, how='all')

In [ ]:
from hyperspy.axes import DataAxis
energy_aMnO2 = Mn_aMnO2.iloc[:, 0].to_numpy(dtype='float64')
energy_aMnO2 = DataAxis(axis=energy_aMnO2, index_in_array=None, name='Energy', units='eV',)
aMnO2 = hs.signals.Signal1D(Mn_aMnO2.iloc[:, 1], axes=[energy_aMnO2,])
aMnO2 = aMnO2.isig[6535.0:].deepcopy()
aMnO2.plot()

In [ ]:
aMnO2_model = aMnO2.create_model()
aMnO2_model.extend((model_refMn_III, model_refMn_IV))
aMnO2_model.set_signal_range(6536.0, 6550.0)

In [ ]:
%matplotlib inline
plt.close('all')
aMnO2_model.fit(optimizer="lm", bounded=True, return_info=True) 
aMnO2_model.plot(plot_components=True, plot_residual=False)
plt.savefig(path.joinpath(path_out, r'PreEdge_9_300_V3_0.tif'), pad_inches=0.05, bbox_inches='tight', dpi=300, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'PreEdge_9_600_V3_0.tif'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
aMnO2_model.print_current_values()

In [ ]:
# 转化成 DataFrame
fit_Mn_IV = pd.DataFrame([aMnO2_model.axes_manager[0].axis, aMnO2_model.as_signal(component_list=['Mn_IV']).data])
fit_Mn_IV = fit_Mn_IV.T
fit_Mn_IV.dropna(how='any', inplace=True)

fit_Mn_III = pd.DataFrame([aMnO2_model.axes_manager[0].axis, aMnO2_model.as_signal(component_list=['Mn_III']).data])
fit_Mn_III = fit_Mn_III.T
fit_Mn_III.dropna(how='any', inplace=True)

# 计算比例
pre_edge_ratio_Mn_IV = scipy.integrate.simpson(y=fit_Mn_IV.iloc[:, 1], x=fit_Mn_IV.iloc[:, 0])/(scipy.integrate.simpson(y=fit_Mn_IV.iloc[:, 1], x=fit_Mn_IV.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_III.iloc[:, 1], x=fit_Mn_III.iloc[:, 0]))
pre_edge_ratio_Mn_III = scipy.integrate.simpson(y=fit_Mn_III.iloc[:, 1], x=fit_Mn_III.iloc[:, 0])/(scipy.integrate.simpson(y=fit_Mn_IV.iloc[:, 1], x=fit_Mn_IV.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_III.iloc[:, 1], x=fit_Mn_III.iloc[:, 0]))
pre_edge_ratio_Mn_IV, pre_edge_ratio_Mn_III

In [ ]:
Pre_Edge_aMnO2 = xr.Dataset(
    data_vars=dict(
        aMnO2 = (['Energy_aMnO2'], aMnO2.data),
        fit = (['Energy_fit'], aMnO2_model.as_signal().data),
        fit_Mn_IV = (['Energy_fit'], aMnO2_model.as_signal(component_list=['Mn_IV']).data),
        fit_Mn_III = (['Energy_fit'], aMnO2_model.as_signal(component_list=['Mn_III']).data),
        ),
    coords=dict(
        Energy_aMnO2 = aMnO2.axes_manager[0].axis,
        Energy_fit= aMnO2_model.axes_manager[0].axis,
    ),
    attrs=dict(
        ratio_Mn_IV = pre_edge_ratio_Mn_IV,
        ratio_Mn_III = pre_edge_ratio_Mn_III,
    ),
)

aMnO2_model.save_parameters2file(path.joinpath(path_out, r'Prisitne_aMnO2_model_parameters_component_ref_2'))
aMnO2_model.save(path.joinpath(path_out, r'Prisitne_aMnO2_model_parameters_component_ref_2'), 'model', overwrite=True)
Pre_Edge_aMnO2.to_netcdf(path.joinpath(path_out, r'Prisitne_aMnO2_component_ref_2.NETCDF4'), engine="h5netcdf")

In [ ]:
# 画图
%matplotlib inline
fig = plt.figure(figsize=(7, 2.5))
gs = gridspec.GridSpec(1, 2, width_ratios=[1, 1], height_ratios=None, wspace=0.0, hspace=0.0, figure=fig)

xas_colors_Mn = [colors[0], colors[4]]
xas_cmap_Mn = ListedColormap(mpl.colormaps['coolwarm'](np.linspace(0.5, 1.0, data.shape[1]//4-5)), name='xas_cmap_Mn', )
labels_Mn = [r'${\alpha}$-MnO$\mathrm{_2}$', r'0.2M Mn$\mathrm{^{2\!+}}$']

# 图 B
subfig = fig.add_subfigure(gs[0, 0], zorder=0)
ax = subfig.add_subplot()
ax.set_position([0, 0, 1.0, 1.0])
ax.set_box_aspect(0.8)

ax.plot(data.iloc[:, 4], data.iloc[:, 7]+0.08, ls='-', color=colors[4], label=r'ref.MnO')
ax.plot(data.iloc[:, 0], data.iloc[:, 3]+0.08, ls='-', color=colors[5], label=r'ref.Mn$\mathrm{_2O_3}$')
ax.plot(data.iloc[:, 8], data.iloc[:, 11]+0.08, ls='-', color=colors[3], label=r'ref.Mn$\mathrm{O_2}$')
ax.legend(loc='upper left', bbox_to_anchor=(0.0, 1.02), ncols=1, frameon=False, fontsize=9, labelcolor='linecolor', columnspacing=0.0)

ax.plot(aMnO2.axes_manager[0].axis, aMnO2.data, ls='-', color='k', label=None)
ax.plot(fit_Mn_III.iloc[:, 0], fit_Mn_III.iloc[:, 1], ls='--', color=colors[5], label=None)
ax.plot(fit_Mn_IV.iloc[:, 0], fit_Mn_IV.iloc[:, 1], ls='--', color=colors[3], label=None)
ax.plot(aMnO2_model.axes_manager[0].axis, aMnO2_model.as_signal().data, ls='--', color=colors[3], label=None)

ax.set_xlim(6535.0, 6550.0)
ax.set_xlabel(r'Energy (eV)', fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=5, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=2.5, offset=0))
ax.set_ylim(-0.01, 0.20)
ax.set_ylabel(r'Relative Intensity (arb.u.)', fontsize=11,)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.05))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.025))

# ax.text(-0.25, 1.0, r'b', transform=ax.transAxes, fontsize=14, va='center', ha='right', fontfamily='Arial', fontweight='bold')

ax.axvline(x=6540.3, ymax=0.65, ls='--')
ax.axvline(x=6543.0, ymax=0.7, ls='--')

plt.savefig(path.joinpath(path_out, r'PreEdge_10_300_V3_0.tif'), transparent=False, pad_inches=0.05, bbox_inches='tight', dpi=300, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'PreEdge_10_600_V3_0.tif'), transparent=False, pad_inches=0.05, bbox_inches='tight', dpi=600, pil_kwargs={"compression": "tiff_lzw"})
plt.show()

#### 2.拟合峰形, IV + pseudo-Voigt 峰 - 三个

In [ ]:
# 数据清洗
Mn_IV = data.iloc[:, [8, 11]].copy().dropna(axis=0, how='all')
Mn_IV.iloc[115:, 1] = 0

# 将 refMn_I//III/IV 做成一个 scale model
from hyperspy.axes import DataAxis
energy_refMn_IV = Mn_IV.iloc[:, 0].to_numpy(dtype='float64')
energy_refMn_IV = DataAxis(axis=energy_refMn_IV, index_in_array=None, name='Energy', units='eV',)
refMn_IV = hs.signals.Signal1D(Mn_IV.iloc[:, 1], axes=[energy_refMn_IV,])
model_refMn_IV = hs.model.components1D.ScalableFixedPattern(refMn_IV) 

# 初始化上述的模型
model_refMn_IV.name= r'Mn_IV'
model_refMn_IV.xscale.value=1.0
model_refMn_IV.shift.value=0
model_refMn_IV.set_parameters_not_free(parameter_name_list=['xscale', 'shift',])
model_refMn_IV.yscale.bmin=0.0

# III 的模型
pv1 = hs.model.components1D.SplitVoigt(centre=6541.05, sigma1=0.893791, sigma2=1.1, fraction=0.45)
pv1.name = r'Mn_III_1'
pv1.set_parameters_not_free(parameter_name_list=['centre', 'sigma1', 'sigma2', 'fraction'])
pv1.A.bmin = 0

pv2 = hs.model.components1D.SplitVoigt(centre=6544.63, sigma1=0.893791, sigma2=1.1, fraction=0.45)
pv2.name = r'Mn_III_2'
pv2.set_parameters_not_free(parameter_name_list=['centre',])
pv2.A.twin = pv1.A
pv2.A.twin_function_expr = '0.67912*x'
pv2.A.bmin = 0
pv2.sigma1.twin = pv1.sigma1
pv2.sigma2.twin = pv1.sigma2
pv2.fraction.twin = pv1.fraction

pv3 = hs.model.components1D.SplitVoigt(centre=6543.07, sigma1=0.893791, sigma2=1.1, fraction=0.45)
pv3.name = r'Mn_III_3'
pv3.set_parameters_not_free(parameter_name_list=['centre'])
pv3.A.twin = pv1.A
pv3.A.twin_function_expr = '0.788393*x'
pv3.A.bmin = 0
pv3.sigma1.twin = pv1.sigma1
pv3.sigma2.twin = pv1.sigma2
pv3.fraction.twin = pv1.fraction

In [ ]:
Mn_aMnO2 = data.iloc[:, [12, 15]].copy().dropna(axis=0, how='all')

In [ ]:
from hyperspy.axes import DataAxis
energy_aMnO2 = Mn_aMnO2.iloc[:, 0].to_numpy(dtype='float64')
energy_aMnO2 = DataAxis(axis=energy_aMnO2, index_in_array=None, name='Energy', units='eV',)
aMnO2 = hs.signals.Signal1D(Mn_aMnO2.iloc[:, 1], axes=[energy_aMnO2,])
aMnO2 = aMnO2.isig[6535.0:].deepcopy()
aMnO2.plot()

In [ ]:
aMnO2_model = aMnO2.create_model()
aMnO2_model.extend((model_refMn_IV, pv1, pv2, pv3))
aMnO2_model.set_signal_range(6536.0, 6550.0)

In [ ]:
%matplotlib inline
plt.close('all')
aMnO2_model.fit(optimizer="lm", bounded=True, return_info=False) 
aMnO2_model.plot(plot_components=True, plot_residual=True)
plt.savefig(path.joinpath(path_out, r'PreEdge_11_300_V3_0.tif'), pad_inches=0.05, bbox_inches='tight', dpi=300, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'PreEdge_11_600_V3_0.tif'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
aMnO2_model.print_current_values()

In [ ]:
# 转化成 DataFrame
fit_Mn_IV = pd.DataFrame([aMnO2_model.axes_manager[0].axis, aMnO2_model.as_signal(component_list=['Mn_IV']).data])
fit_Mn_IV = fit_Mn_IV.T
fit_Mn_IV.dropna(how='any', inplace=True)

fit_Mn_III_1 = pd.DataFrame([aMnO2_model.axes_manager[0].axis, aMnO2_model.as_signal(component_list=['Mn_III_1']).data])
fit_Mn_III_1 = fit_Mn_III_1.T
fit_Mn_III_1.dropna(how='any', inplace=True)

fit_Mn_III_2 = pd.DataFrame([aMnO2_model.axes_manager[0].axis, aMnO2_model.as_signal(component_list=['Mn_III_2']).data])
fit_Mn_III_2 = fit_Mn_III_2.T
fit_Mn_III_2.dropna(how='any', inplace=True)

fit_Mn_III_3 = pd.DataFrame([aMnO2_model.axes_manager[0].axis, aMnO2_model.as_signal(component_list=['Mn_III_3']).data])
fit_Mn_III_3 = fit_Mn_III_3.T
fit_Mn_III_3.dropna(how='any', inplace=True)

# 计算比例
pre_edge_ratio_Mn_IV = scipy.integrate.simpson(y=fit_Mn_IV.iloc[:, 1], x=fit_Mn_IV.iloc[:, 0])/(scipy.integrate.simpson(y=fit_Mn_IV.iloc[:, 1], x=fit_Mn_IV.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_III_1.iloc[:, 1], x=fit_Mn_III_1.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_III_2.iloc[:, 1], x=fit_Mn_III_2.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_III_3.iloc[:, 1], x=fit_Mn_III_3.iloc[:, 0]))
pre_edge_ratio_Mn_III = (scipy.integrate.simpson(y=fit_Mn_III_1.iloc[:, 1], x=fit_Mn_III_1.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_III_2.iloc[:, 1], x=fit_Mn_III_2.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_III_3.iloc[:, 1], x=fit_Mn_III_3.iloc[:, 0]))/(scipy.integrate.simpson(y=fit_Mn_IV.iloc[:, 1], x=fit_Mn_IV.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_III_1.iloc[:, 1], x=fit_Mn_III_1.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_III_2.iloc[:, 1], x=fit_Mn_III_2.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_III_3.iloc[:, 1], x=fit_Mn_III_3.iloc[:, 0]))
pre_edge_ratio_Mn_IV, pre_edge_ratio_Mn_III

In [ ]:
Pre_Edge_aMnO2 = xr.Dataset(
    data_vars=dict(
        aMnO2 = (['Energy_aMnO2'], aMnO2.data),
        fit = (['Energy_fit'], aMnO2_model.as_signal().data),
        fit_Mn_IV = (['Energy_fit'], aMnO2_model.as_signal(component_list=['Mn_IV']).data),
        fit_Mn_III_1 = (['Energy_fit'], aMnO2_model.as_signal(component_list=['Mn_III_1']).data),
        fit_Mn_III_2 = (['Energy_fit'], aMnO2_model.as_signal(component_list=['Mn_III_2']).data),
        fit_Mn_III_3 = (['Energy_fit'], aMnO2_model.as_signal(component_list=['Mn_III_3']).data),
        ),
    coords=dict(
        Energy_aMnO2 = aMnO2.axes_manager[0].axis,
        Energy_fit= aMnO2_model.axes_manager[0].axis,
    ),
    attrs=dict(
        ratio_Mn_IV = pre_edge_ratio_Mn_IV,
        ratio_Mn_III = pre_edge_ratio_Mn_III,
    ),
)

aMnO2_model.save_parameters2file(path.joinpath(path_out, r'Prisitne_aMnO2_model_parameters_component_ref_1_pv_3'))
aMnO2_model.save(path.joinpath(path_out, r'Prisitne_aMnO2_model_parameters_component_ref_1_pv_3'), 'model', overwrite=True)
Pre_Edge_aMnO2.to_netcdf(path.joinpath(path_out, r'Prisitne_aMnO2_component_ref_1_pv_3.NETCDF4'), engine="h5netcdf")

In [ ]:
# 画图
%matplotlib inline
fig = plt.figure(figsize=(7, 2.5))
gs = gridspec.GridSpec(1, 2, width_ratios=[1, 1], height_ratios=None, wspace=0.0, hspace=0.0, figure=fig)

xas_colors_Mn = [colors[0], colors[4]]
xas_cmap_Mn = ListedColormap(mpl.colormaps['coolwarm'](np.linspace(0.5, 1.0, data.shape[1]//4-5)), name='xas_cmap_Mn', )
labels_Mn = [r'${\alpha}$-MnO$\mathrm{_2}$', r'0.2M Mn$\mathrm{^{2\!+}}$']

# 图 B
subfig = fig.add_subfigure(gs[0, 0], zorder=0)
ax = subfig.add_subplot()
ax.set_position([0, 0, 1.0, 1.0])
ax.set_box_aspect(0.8)

ax.plot(data.iloc[:, 4], data.iloc[:, 7]+0.08, ls='-', color=colors[4], label=r'ref.MnO')
ax.plot(data.iloc[:, 0], data.iloc[:, 3]+0.08, ls='-', color=colors[5], label=r'ref.Mn$\mathrm{_2O_3}$')
ax.plot(data.iloc[:, 8], data.iloc[:, 11]+0.08, ls='-', color=colors[3], label=r'ref.Mn$\mathrm{O_2}$')
ax.legend(loc='upper left', bbox_to_anchor=(0.0, 1.02), ncols=1, frameon=False, fontsize=9, labelcolor='linecolor', columnspacing=0.0)

ax.plot(aMnO2.axes_manager[0].axis, aMnO2.data, ls='-', color='k', label=None)
# ax.plot(fit_Mn_III_3.iloc[:, 0], fit_Mn_III_3.iloc[:, 1], ls='--', color=colors[5], label=None)
# ax.plot(fit_Mn_III_2.iloc[:, 0], fit_Mn_III_2.iloc[:, 1], ls='--', color=colors[5], label=None)
# ax.plot(fit_Mn_III_1.iloc[:, 0], fit_Mn_III_1.iloc[:, 1], ls='--', color=colors[5], label=None)
ax.plot(fit_Mn_III_1.iloc[:, 0], fit_Mn_III_1.iloc[:, 1] + fit_Mn_III_2.iloc[:, 1] + fit_Mn_III_3.iloc[:, 1], ls='--', color=colors[5], label=None)
ax.plot(fit_Mn_IV.iloc[:, 0], fit_Mn_IV.iloc[:, 1], ls='--', color=colors[3], label=None)
ax.plot(aMnO2_model.axes_manager[0].axis, aMnO2_model.as_signal().data, ls='--', color=colors[3], label=None)

ax.set_xlim(6535.0, 6550.0)
ax.set_xlabel(r'Energy (eV)', fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=5, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=2.5, offset=0))
ax.set_ylim(-0.01, 0.20)
ax.set_ylabel(r'Relative Intensity (arb.u.)', fontsize=11,)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.05))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.025))

# ax.text(-0.25, 1.0, r'b', transform=ax.transAxes, fontsize=14, va='center', ha='right', fontfamily='Arial', fontweight='bold')

ax.axvline(x=6540.3, ymax=0.65, ls='--')
ax.axvline(x=6543.0, ymax=0.7, ls='--')

plt.savefig(path.joinpath(path_out, r'PreEdge_12_300_V3_0.tif'), transparent=False, pad_inches=0.05, bbox_inches='tight', dpi=300, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'PreEdge_12_600_V3_0.tif'), transparent=False, pad_inches=0.05, bbox_inches='tight', dpi=600, pil_kwargs={"compression": "tiff_lzw"})
plt.show()

### OCV

#### 拟合峰形, IV + III + II (pseudo-Voigt 峰 - 一个)

In [ ]:
# IV/III 的模型
Mn_IV = data.iloc[:, [8, 11]].copy().dropna(axis=0, how='all')
Mn_III = data.iloc[:, [0, 3]].copy().dropna(axis=0, how='all')

Mn_IV.iloc[115:, 1] = 0
Mn_III.iloc[97:, 1] = 0

# 将 refMn_I//III/IV 做成一个 scale model
from hyperspy.axes import DataAxis
energy_refMn_IV = Mn_IV.iloc[:, 0].to_numpy(dtype='float64')
energy_refMn_IV = DataAxis(axis=energy_refMn_IV, index_in_array=None, name='Energy', units='eV',)
refMn_IV = hs.signals.Signal1D(Mn_IV.iloc[:, 1], axes=[energy_refMn_IV,])
model_refMn_IV = hs.model.components1D.ScalableFixedPattern(refMn_IV) 

energy_refMn_III = Mn_III.iloc[:, 0].to_numpy(dtype='float64')
energy_refMn_III = DataAxis(axis=energy_refMn_III, index_in_array=None, name='Energy', units='eV',)
refMn_III = hs.signals.Signal1D(Mn_III.iloc[:, 1], axes=[energy_refMn_III,])
model_refMn_III = hs.model.components1D.ScalableFixedPattern(refMn_III) 

# 初始化上述的模型
model_refMn_IV.name= r'Mn_IV'
model_refMn_IV.xscale.value=1.03564
model_refMn_IV.shift.value=232.968
model_refMn_IV.set_parameters_not_free(parameter_name_list=['xscale', 'shift',])
model_refMn_IV.yscale.bmin=0.0

model_refMn_III.name= r'Mn_III'
model_refMn_III.xscale.value=1.13677
model_refMn_III.shift.value=894.306
model_refMn_III.set_parameters_not_free(parameter_name_list=['xscale', 'shift',])
model_refMn_III.yscale.bmin=0.0

# II 的模型
pv4 = hs.model.components1D.SplitVoigt(centre=6540.74, sigma1=1.54073, sigma2=1.1, fraction=0.45)
pv4.name = r'Mn_II'
pv4.set_parameters_not_free(parameter_name_list=['centre', 'sigma1', 'sigma2', 'fraction'])
pv4.A.bmin = 0

In [ ]:
Mn_OCV = data.iloc[:, [20, 23]].copy().dropna(axis=0, how='all')

In [ ]:
from hyperspy.axes import DataAxis
energy_OCV = Mn_OCV.iloc[:, 0].to_numpy(dtype='float64')
energy_OCV = DataAxis(axis=energy_OCV, index_in_array=None, name='Energy', units='eV',)
OCV = hs.signals.Signal1D(Mn_OCV.iloc[:, 1], axes=[energy_OCV,])
OCV = OCV.isig[6532.0:].deepcopy()
OCV.plot()

In [ ]:
OCV_model = OCV.create_model()
OCV_model.extend((model_refMn_IV, model_refMn_III, pv4))
OCV_model.set_signal_range(6536.0, 6550.0)

In [ ]:
%matplotlib widget
plt.close('all')
OCV_model.fit(optimizer="lm", bounded=True, return_info=False) 
OCV_model.plot(plot_components=True, plot_residual=False)
plt.savefig(path.joinpath(path_out, r'PreEdge_13_300_V3_0.tif'), pad_inches=0.05, bbox_inches='tight', dpi=300, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'PreEdge_13_600_V3_0.tif'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
OCV_model.print_current_values()

In [ ]:
# 转化成 DataFrame
fit_Mn_IV = pd.DataFrame([OCV_model.axes_manager[0].axis, OCV_model.as_signal(component_list=['Mn_IV']).data])
fit_Mn_IV = fit_Mn_IV.T
fit_Mn_IV.dropna(how='any', inplace=True)

fit_Mn_III = pd.DataFrame([OCV_model.axes_manager[0].axis, OCV_model.as_signal(component_list=['Mn_III']).data])
fit_Mn_III = fit_Mn_III.T
fit_Mn_III.dropna(how='any', inplace=True)

fit_Mn_II = pd.DataFrame([OCV_model.axes_manager[0].axis, OCV_model.as_signal(component_list=['Mn_II']).data])
fit_Mn_II = fit_Mn_II.T
fit_Mn_II.dropna(how='any', inplace=True)

# 计算比例
pre_edge_ratio_Mn_IV = scipy.integrate.simpson(y=fit_Mn_IV.iloc[:, 1], x=fit_Mn_IV.iloc[:, 0])/(scipy.integrate.simpson(y=fit_Mn_IV.iloc[:, 1], x=fit_Mn_IV.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_III.iloc[:, 1], x=fit_Mn_III.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_II.iloc[:, 1], x=fit_Mn_II.iloc[:, 0]))
pre_edge_ratio_Mn_III = (scipy.integrate.simpson(y=fit_Mn_III.iloc[:, 1], x=fit_Mn_III.iloc[:, 0]))/(scipy.integrate.simpson(y=fit_Mn_IV.iloc[:, 1], x=fit_Mn_IV.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_III.iloc[:, 1], x=fit_Mn_III.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_II.iloc[:, 1], x=fit_Mn_II.iloc[:, 0]))
pre_edge_ratio_Mn_IV, pre_edge_ratio_Mn_III

In [ ]:
Pre_Edge_OCV = xr.Dataset(
    data_vars=dict(
        OCV = (['Energy_OCV'], OCV.data),
        fit = (['Energy_fit'], OCV_model.as_signal().data),
        fit_Mn_IV = (['Energy_fit'], OCV_model.as_signal(component_list=['Mn_IV']).data),
        fit_Mn_III = (['Energy_fit'], OCV_model.as_signal(component_list=['Mn_III']).data),
        fit_Mn_II = (['Energy_fit'], OCV_model.as_signal(component_list=['Mn_II']).data),
        ),
    coords=dict(
        Energy_OCV = OCV.axes_manager[0].axis,
        Energy_fit= OCV_model.axes_manager[0].axis,
    ),
    attrs=dict(
        ratio_Mn_IV = pre_edge_ratio_Mn_IV,
        ratio_Mn_III = pre_edge_ratio_Mn_III,
    ),
)

OCV_model.save_parameters2file(path.joinpath(path_out, r'OCV_model_parameters_component_ref_2_pv_1'))
OCV_model.save(path.joinpath(path_out, r'OCV_model_parameters_component_ref_2_pv_1'), 'model', overwrite=True)
Pre_Edge_OCV.to_netcdf(path.joinpath(path_out, r'OCV_component_ref_2_pv_1.NETCDF4'), engine="h5netcdf")

In [ ]:
# 画图
%matplotlib inline
plt.close('all')
fig = plt.figure(figsize=(7, 2.5))
gs = gridspec.GridSpec(1, 2, width_ratios=[1, 1], height_ratios=None, wspace=0.0, hspace=0.0, figure=fig)

xas_colors_Mn = [colors[0], colors[4]]
xas_cmap_Mn = ListedColormap(mpl.colormaps['coolwarm'](np.linspace(0.5, 1.0, data.shape[1]//4-5)), name='xas_cmap_Mn', )
labels_Mn = [r'${\alpha}$-MnO$\mathrm{_2}$', r'0.2M Mn$\mathrm{^{2\!+}}$']

# 图 B
subfig = fig.add_subfigure(gs[0, 0], zorder=0)
ax = subfig.add_subplot()
ax.set_position([0, 0, 1.0, 1.0])
ax.set_box_aspect(0.8)

ax.plot(data.iloc[:, 4], data.iloc[:, 7]+0.08, ls='-', color=colors[4], label=r'ref.MnO')
ax.plot(data.iloc[:, 0], data.iloc[:, 3]+0.08, ls='-', color=colors[5], label=r'ref.Mn$\mathrm{_2O_3}$')
ax.plot(data.iloc[:, 8], data.iloc[:, 11]+0.08, ls='-', color=colors[3], label=r'ref.Mn$\mathrm{O_2}$')
ax.legend(loc='upper left', bbox_to_anchor=(0.0, 1.02), ncols=1, frameon=False, fontsize=9, labelcolor='linecolor', columnspacing=0.0)

ax.plot(OCV.axes_manager[0].axis, OCV.data, ls='-', color='k', label=None)
ax.plot(fit_Mn_II.iloc[:, 0], fit_Mn_II.iloc[:, 1], ls='--', color=colors[4], label=None)
ax.plot(fit_Mn_IV.iloc[:, 0], fit_Mn_IV.iloc[:, 1], ls='--', color=colors[3], label=None)
ax.plot(fit_Mn_III.iloc[:, 0], fit_Mn_III.iloc[:, 1], ls='--', color=colors[5], label=None)
ax.plot(OCV_model.axes_manager[0].axis, OCV_model.as_signal().data, ls='--', color=colors[3], label=None)

ax.set_xlim(6535.0, 6550.0)
ax.set_xlabel(r'Energy (eV)', fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=5, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=2.5, offset=0))
ax.set_ylim(-0.01, 0.20)
ax.set_ylabel(r'Relative Intensity (arb.u.)', fontsize=11,)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.05))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.025))

# ax.text(-0.25, 1.0, r'b', transform=ax.transAxes, fontsize=14, va='center', ha='right', fontfamily='Arial', fontweight='bold')

ax.axvline(x=6540.3, ymax=0.65, ls='--')
ax.axvline(x=6543.0, ymax=0.7, ls='--')

plt.savefig(path.joinpath(path_out, r'PreEdge_14_300_V3_0.tif'), transparent=False, pad_inches=0.05, bbox_inches='tight', dpi=300, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'PreEdge_14_600_V3_0.tif'), transparent=False, pad_inches=0.05, bbox_inches='tight', dpi=600, pil_kwargs={"compression": "tiff_lzw"})
plt.show()

#### 拟合峰形, IV + III (pseudo-Voigt 峰 - 三个) + II (pseudo-Voigt 峰 - 一个)

In [ ]:
# IV 的模型
Mn_IV = data.iloc[:, [8, 11]].copy().dropna(axis=0, how='all')
Mn_IV.iloc[115:, 1] = 0

# 将 IV 做成一个 scale model
from hyperspy.axes import DataAxis
energy_refMn_IV = Mn_IV.iloc[:, 0].to_numpy(dtype='float64')
energy_refMn_IV = DataAxis(axis=energy_refMn_IV, index_in_array=None, name='Energy', units='eV',)
refMn_IV = hs.signals.Signal1D(Mn_IV.iloc[:, 1], axes=[energy_refMn_IV,])
model_refMn_IV = hs.model.components1D.ScalableFixedPattern(refMn_IV) 

# 初始化上述的模型
model_refMn_IV.name= r'Mn_IV'
model_refMn_IV.xscale.value=1.0
model_refMn_IV.shift.value=0
model_refMn_IV.set_parameters_not_free(parameter_name_list=['xscale', 'shift',])
model_refMn_IV.yscale.bmin=0.0

# III 的模型
pv1 = hs.model.components1D.SplitVoigt(centre=6541.05, sigma1=0.893791, sigma2=1.1, fraction=0.45)
pv1.name = r'Mn_III_1'
pv1.set_parameters_not_free(parameter_name_list=['centre', 'sigma1', 'sigma2', 'fraction'])
pv1.A.bmin = 0

pv2 = hs.model.components1D.SplitVoigt(centre=6544.63, sigma1=0.893791, sigma2=1.1, fraction=0.45)
pv2.name = r'Mn_III_2'
pv2.set_parameters_not_free(parameter_name_list=['centre',])
pv2.A.twin = pv1.A
pv2.A.twin_function_expr = '0.146*x'
# pv2.A.bmin = 0
pv2.sigma1.twin = pv1.sigma1
pv2.sigma2.twin = pv1.sigma2
pv2.fraction.twin = pv1.fraction

pv3 = hs.model.components1D.SplitVoigt(centre=6543.07, sigma1=0.893791, sigma2=1.1, fraction=0.45)
pv3.name = r'Mn_III_3'
pv3.set_parameters_not_free(parameter_name_list=['centre'])
pv3.A.twin = pv1.A
pv3.A.twin_function_expr = '0.93872*x'
# pv3.A.bmin = 0
pv3.sigma1.twin = pv1.sigma1
pv3.sigma2.twin = pv1.sigma2
pv3.fraction.twin = pv1.fraction

# II 的模型
pv4 = hs.model.components1D.SplitVoigt(centre=6540.74, sigma1=1.54073, sigma2=1.1, fraction=0.45)
pv4.name = r'Mn_II'
pv4.set_parameters_not_free(parameter_name_list=['centre', 'sigma1', 'sigma2', 'fraction'])
pv4.A.bmin = 0

In [ ]:
Mn_OCV = data.iloc[:, [20, 23]].copy().dropna(axis=0, how='all')

In [ ]:
from hyperspy.axes import DataAxis
energy_OCV = Mn_OCV.iloc[:, 0].to_numpy(dtype='float64')
energy_OCV = DataAxis(axis=energy_OCV, index_in_array=None, name='Energy', units='eV',)
OCV = hs.signals.Signal1D(Mn_OCV.iloc[:, 1], axes=[energy_OCV,])
OCV = OCV.isig[6532.0:].deepcopy()
OCV.plot()

In [ ]:
OCV_model = OCV.create_model()
OCV_model.extend((model_refMn_IV, pv1, pv2, pv3, pv4))
OCV_model.set_signal_range(6536.0, 6550.0)

In [ ]:
%matplotlib widget
plt.close('all')
OCV_model.fit(optimizer="lm", bounded=True, return_info=False) 
OCV_model.plot(plot_components=True, plot_residual=False)
plt.savefig(path.joinpath(path_out, r'PreEdge_15_300_V3_0.tif'), pad_inches=0.05, bbox_inches='tight', dpi=300, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'PreEdge_15_600_V3_0.tif'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
OCV_model.print_current_values()

In [ ]:
# 转化成 DataFrame
fit_Mn_IV = pd.DataFrame([OCV_model.axes_manager[0].axis, OCV_model.as_signal(component_list=['Mn_IV']).data])
fit_Mn_IV = fit_Mn_IV.T
fit_Mn_IV.dropna(how='any', inplace=True)

fit_Mn_III_1 = pd.DataFrame([OCV_model.axes_manager[0].axis, OCV_model.as_signal(component_list=['Mn_III_1']).data])
fit_Mn_III_1 = fit_Mn_III_1.T
fit_Mn_III_1.dropna(how='any', inplace=True)

fit_Mn_III_2 = pd.DataFrame([OCV_model.axes_manager[0].axis, OCV_model.as_signal(component_list=['Mn_III_2']).data])
fit_Mn_III_2 = fit_Mn_III_2.T
fit_Mn_III_2.dropna(how='any', inplace=True)

fit_Mn_III_3 = pd.DataFrame([OCV_model.axes_manager[0].axis, OCV_model.as_signal(component_list=['Mn_III_3']).data])
fit_Mn_III_3 = fit_Mn_III_3.T
fit_Mn_III_3.dropna(how='any', inplace=True)

fit_Mn_II = pd.DataFrame([OCV_model.axes_manager[0].axis, OCV_model.as_signal(component_list=['Mn_II']).data])
fit_Mn_II = fit_Mn_II.T
fit_Mn_II.dropna(how='any', inplace=True)

# 计算比例
pre_edge_ratio_Mn_IV = scipy.integrate.simpson(y=fit_Mn_IV.iloc[:, 1], x=fit_Mn_IV.iloc[:, 0])/(scipy.integrate.simpson(y=fit_Mn_IV.iloc[:, 1], x=fit_Mn_IV.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_III_1.iloc[:, 1], x=fit_Mn_III_1.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_III_2.iloc[:, 1], x=fit_Mn_III_2.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_III_3.iloc[:, 1], x=fit_Mn_III_3.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_II.iloc[:, 1], x=fit_Mn_II.iloc[:, 0]))
pre_edge_ratio_Mn_III = (scipy.integrate.simpson(y=fit_Mn_III_1.iloc[:, 1], x=fit_Mn_III_1.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_III_2.iloc[:, 1], x=fit_Mn_III_2.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_III_3.iloc[:, 1], x=fit_Mn_III_3.iloc[:, 0]))/(scipy.integrate.simpson(y=fit_Mn_IV.iloc[:, 1], x=fit_Mn_IV.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_III_1.iloc[:, 1], x=fit_Mn_III_1.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_III_2.iloc[:, 1], x=fit_Mn_III_2.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_III_3.iloc[:, 1], x=fit_Mn_III_3.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_II.iloc[:, 1], x=fit_Mn_II.iloc[:, 0]))
pre_edge_ratio_Mn_IV, pre_edge_ratio_Mn_III

In [ ]:
Pre_Edge_OCV = xr.Dataset(
    data_vars=dict(
        OCV = (['Energy_OCV'], OCV.data),
        fit = (['Energy_fit'], OCV_model.as_signal().data),
        fit_Mn_IV = (['Energy_fit'], OCV_model.as_signal(component_list=['Mn_IV']).data),
        fit_Mn_III_1 = (['Energy_fit'], OCV_model.as_signal(component_list=['Mn_III_1']).data),
        fit_Mn_III_2 = (['Energy_fit'], OCV_model.as_signal(component_list=['Mn_III_2']).data),
        fit_Mn_III_3 = (['Energy_fit'], OCV_model.as_signal(component_list=['Mn_III_3']).data),
        fit_Mn_II = (['Energy_fit'], OCV_model.as_signal(component_list=['Mn_II']).data),
        ),
    coords=dict(
        Energy_OCV = OCV.axes_manager[0].axis,
        Energy_fit= OCV_model.axes_manager[0].axis,
    ),
    attrs=dict(
        ratio_Mn_IV = pre_edge_ratio_Mn_IV,
        ratio_Mn_III = pre_edge_ratio_Mn_III,
    ),
)

OCV_model.save_parameters2file(path.joinpath(path_out, r'OCV_model_parameters_component_ref_1_pv_3_pv_1'))
OCV_model.save(path.joinpath(path_out, r'OCV_model_parameters_component_ref_1_pv_3_pv_1'), 'model', overwrite=True)
Pre_Edge_OCV.to_netcdf(path.joinpath(path_out, r'OCV_component_ref_1_pv_3_pv_1.NETCDF4'), engine="h5netcdf")

In [ ]:
# 画图
%matplotlib inline
plt.close('all')
fig = plt.figure(figsize=(7, 2.5))
gs = gridspec.GridSpec(1, 2, width_ratios=[1, 1], height_ratios=None, wspace=0.0, hspace=0.0, figure=fig)

xas_colors_Mn = [colors[0], colors[4]]
xas_cmap_Mn = ListedColormap(mpl.colormaps['coolwarm'](np.linspace(0.5, 1.0, data.shape[1]//4-5)), name='xas_cmap_Mn', )
labels_Mn = [r'${\alpha}$-MnO$\mathrm{_2}$', r'0.2M Mn$\mathrm{^{2\!+}}$']

# 图 B
subfig = fig.add_subfigure(gs[0, 0], zorder=0)
ax = subfig.add_subplot()
ax.set_position([0, 0, 1.0, 1.0])
ax.set_box_aspect(0.8)

ax.plot(data.iloc[:, 4], data.iloc[:, 7]+0.08, ls='-', color=colors[4], label=r'ref.MnO')
ax.plot(data.iloc[:, 0], data.iloc[:, 3]+0.08, ls='-', color=colors[5], label=r'ref.Mn$\mathrm{_2O_3}$')
ax.plot(data.iloc[:, 8], data.iloc[:, 11]+0.08, ls='-', color=colors[3], label=r'ref.Mn$\mathrm{O_2}$')
ax.legend(loc='upper left', bbox_to_anchor=(0.0, 1.02), ncols=1, frameon=False, fontsize=9, labelcolor='linecolor', columnspacing=0.0)

ax.plot(OCV.axes_manager[0].axis, OCV.data, ls='-', color='k', label=None)
ax.plot(fit_Mn_II.iloc[:, 0], fit_Mn_II.iloc[:, 1], ls='--', color=colors[4], label=None)
ax.plot(fit_Mn_IV.iloc[:, 0], fit_Mn_IV.iloc[:, 1], ls='--', color=colors[3], label=None)
ax.plot(fit_Mn_III_1.iloc[:, 0], fit_Mn_III_1.iloc[:, 1] + fit_Mn_III_2.iloc[:, 1] + fit_Mn_III_3.iloc[:, 1], ls='--', color=colors[5], label=None)
ax.plot(OCV_model.axes_manager[0].axis, OCV_model.as_signal().data, ls='--', color=colors[3], label=None)

ax.set_xlim(6535.0, 6550.0)
ax.set_xlabel(r'Energy (eV)', fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=5, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=2.5, offset=0))
ax.set_ylim(-0.01, 0.20)
ax.set_ylabel(r'Relative Intensity (arb.u.)', fontsize=11,)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.05))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.025))

# ax.text(-0.25, 1.0, r'b', transform=ax.transAxes, fontsize=14, va='center', ha='right', fontfamily='Arial', fontweight='bold')

ax.axvline(x=6540.3, ymax=0.65, ls='--')
ax.axvline(x=6543.0, ymax=0.7, ls='--')

plt.savefig(path.joinpath(path_out, r'PreEdge_16_300_V3_0.tif'), transparent=False, pad_inches=0.05, bbox_inches='tight', dpi=300, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'PreEdge_16_600_V3_0.tif'), transparent=False, pad_inches=0.05, bbox_inches='tight', dpi=600, pil_kwargs={"compression": "tiff_lzw"})
plt.show()

### 1stDischarge

#### 拟合峰形, IV + III (pseudo-Voigt 峰 - 三个) + II (pseudo-Voigt 峰 - 一个)

In [ ]:
# IV 的模型
Mn_IV = data.iloc[:, [8, 11]].copy().dropna(axis=0, how='all')
Mn_IV.iloc[115:, 1] = 0

# 将 IV 做成一个 scale model
from hyperspy.axes import DataAxis
energy_refMn_IV = Mn_IV.iloc[:, 0].to_numpy(dtype='float64')
energy_refMn_IV = DataAxis(axis=energy_refMn_IV, index_in_array=None, name='Energy', units='eV',)
refMn_IV = hs.signals.Signal1D(Mn_IV.iloc[:, 1], axes=[energy_refMn_IV,])
model_refMn_IV = hs.model.components1D.ScalableFixedPattern(refMn_IV) 

# 初始化上述的模型
model_refMn_IV.name= r'Mn_IV'
model_refMn_IV.xscale.value=1.0
model_refMn_IV.shift.value=0
model_refMn_IV.set_parameters_not_free(parameter_name_list=['xscale', 'shift',])
model_refMn_IV.yscale.bmin=0.0

# III 的模型
pv1 = hs.model.components1D.SplitVoigt(centre=6541.05, sigma1=0.893791, sigma2=1.1, fraction=0.45)
pv1.name = r'Mn_III_1'
pv1.set_parameters_not_free(parameter_name_list=['centre', 'sigma1', 'sigma2', 'fraction'])
pv1.A.bmin = 0

pv2 = hs.model.components1D.SplitVoigt(centre=6544.63, sigma1=0.893791, sigma2=1.1, fraction=0.45)
pv2.name = r'Mn_III_2'
pv2.set_parameters_not_free(parameter_name_list=['centre',])
pv2.A.twin = pv1.A
pv2.A.twin_function_expr = '0.146*x'
# pv2.A.bmin = 0
pv2.sigma1.twin = pv1.sigma1
pv2.sigma2.twin = pv1.sigma2
pv2.fraction.twin = pv1.fraction

pv3 = hs.model.components1D.SplitVoigt(centre=6543.07, sigma1=0.893791, sigma2=1.1, fraction=0.45)
pv3.name = r'Mn_III_3'
pv3.set_parameters_not_free(parameter_name_list=['centre'])
pv3.A.twin = pv1.A
pv3.A.twin_function_expr = '0.93872*x'
# pv3.A.bmin = 0
pv3.sigma1.twin = pv1.sigma1
pv3.sigma2.twin = pv1.sigma2
pv3.fraction.twin = pv1.fraction

# II 的模型
pv4 = hs.model.components1D.SplitVoigt(centre=6540.74, sigma1=1.54073, sigma2=1.1, fraction=0.45)
pv4.name = r'Mn_II'
pv4.set_parameters_not_free(parameter_name_list=['centre', 'sigma1', 'sigma2', 'fraction'])
pv4.A.bmin = 0

In [ ]:
Mn_FD1st = data.iloc[:, [-4, -1]].copy().dropna(axis=0, how='all')

In [ ]:
from hyperspy.axes import DataAxis
energy_Mn_FD1st = Mn_FD1st.iloc[:, 0].to_numpy(dtype='float64')
energy_Mn_FD1st = DataAxis(axis=energy_Mn_FD1st, index_in_array=None, name='Energy', units='eV',)
FD1st = hs.signals.Signal1D(Mn_FD1st.iloc[:, 1], axes=[energy_Mn_FD1st,])
FD1st = FD1st.isig[6532.0:].deepcopy()
FD1st.plot()

In [ ]:
FD1st_model = FD1st.create_model()
FD1st_model.extend((model_refMn_IV, pv1, pv2, pv3, pv4))
FD1st_model.set_signal_range(6536.0, 6550.0)

In [ ]:
%matplotlib widget
plt.close('all')
FD1st_model.fit(optimizer="lm", bounded=True, return_info=False) 
FD1st_model.plot(plot_components=True, plot_residual=False)
plt.savefig(path.joinpath(path_out, r'PreEdge_17_300_V3_0.tif'), pad_inches=0.05, bbox_inches='tight', dpi=300, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'PreEdge_17_600_V3_0.tif'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
FD1st_model.print_current_values()

In [ ]:
# 转化成 DataFrame
fit_Mn_IV = pd.DataFrame([FD1st_model.axes_manager[0].axis, FD1st_model.as_signal(component_list=['Mn_IV']).data])
fit_Mn_IV = fit_Mn_IV.T
fit_Mn_IV.dropna(how='any', inplace=True)

fit_Mn_III_1 = pd.DataFrame([FD1st_model.axes_manager[0].axis, FD1st_model.as_signal(component_list=['Mn_III_1']).data])
fit_Mn_III_1 = fit_Mn_III_1.T
fit_Mn_III_1.dropna(how='any', inplace=True)

fit_Mn_III_2 = pd.DataFrame([FD1st_model.axes_manager[0].axis, FD1st_model.as_signal(component_list=['Mn_III_2']).data])
fit_Mn_III_2 = fit_Mn_III_2.T
fit_Mn_III_2.dropna(how='any', inplace=True)

fit_Mn_III_3 = pd.DataFrame([FD1st_model.axes_manager[0].axis, FD1st_model.as_signal(component_list=['Mn_III_3']).data])
fit_Mn_III_3 = fit_Mn_III_3.T
fit_Mn_III_3.dropna(how='any', inplace=True)

fit_Mn_II = pd.DataFrame([FD1st_model.axes_manager[0].axis, FD1st_model.as_signal(component_list=['Mn_II']).data])
fit_Mn_II = fit_Mn_II.T
fit_Mn_II.dropna(how='any', inplace=True)

# 计算比例
pre_edge_ratio_Mn_IV = scipy.integrate.simpson(y=fit_Mn_IV.iloc[:, 1], x=fit_Mn_IV.iloc[:, 0])/(scipy.integrate.simpson(y=fit_Mn_IV.iloc[:, 1], x=fit_Mn_IV.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_III_1.iloc[:, 1], x=fit_Mn_III_1.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_III_2.iloc[:, 1], x=fit_Mn_III_2.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_III_3.iloc[:, 1], x=fit_Mn_III_3.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_II.iloc[:, 1], x=fit_Mn_II.iloc[:, 0]))
pre_edge_ratio_Mn_III = (scipy.integrate.simpson(y=fit_Mn_III_1.iloc[:, 1], x=fit_Mn_III_1.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_III_2.iloc[:, 1], x=fit_Mn_III_2.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_III_3.iloc[:, 1], x=fit_Mn_III_3.iloc[:, 0]))/(scipy.integrate.simpson(y=fit_Mn_IV.iloc[:, 1], x=fit_Mn_IV.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_III_1.iloc[:, 1], x=fit_Mn_III_1.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_III_2.iloc[:, 1], x=fit_Mn_III_2.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_III_3.iloc[:, 1], x=fit_Mn_III_3.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_II.iloc[:, 1], x=fit_Mn_II.iloc[:, 0]))
pre_edge_ratio_Mn_IV, pre_edge_ratio_Mn_III

In [ ]:
Pre_Edge_FD1st = xr.Dataset(
    data_vars=dict(
        FD1st = (['Energy_FD1st'], FD1st.data),
        fit = (['Energy_fit'], FD1st_model.as_signal().data),
        fit_Mn_IV = (['Energy_fit'], FD1st_model.as_signal(component_list=['Mn_IV']).data),
        fit_Mn_III_1 = (['Energy_fit'], FD1st_model.as_signal(component_list=['Mn_III_1']).data),
        fit_Mn_III_2 = (['Energy_fit'], FD1st_model.as_signal(component_list=['Mn_III_2']).data),
        fit_Mn_III_3 = (['Energy_fit'], FD1st_model.as_signal(component_list=['Mn_III_3']).data),
        fit_Mn_II = (['Energy_fit'], FD1st_model.as_signal(component_list=['Mn_II']).data),
        ),
    coords=dict(
        Energy_FD1st = FD1st.axes_manager[0].axis,
        Energy_fit= FD1st_model.axes_manager[0].axis,
    ),
    attrs=dict(
        ratio_Mn_IV = pre_edge_ratio_Mn_IV,
        ratio_Mn_III = pre_edge_ratio_Mn_III,
    ),
)

FD1st_model.save_parameters2file(path.joinpath(path_out, r'FD1st_model_parameters_component_ref_1_pv_3_pv_1'))
FD1st_model.save(path.joinpath(path_out, r'FD1st_model_parameters_component_ref_1_pv_3_pv_1'), 'model', overwrite=True)
Pre_Edge_OCV.to_netcdf(path.joinpath(path_out, r'FD1st_component_ref_1_pv_3_pv_1.NETCDF4'), engine="h5netcdf")

In [ ]:
# 画图
%matplotlib inline
plt.close('all')
fig = plt.figure(figsize=(7, 2.5))
gs = gridspec.GridSpec(1, 2, width_ratios=[1, 1], height_ratios=None, wspace=0.0, hspace=0.0, figure=fig)

xas_colors_Mn = [colors[0], colors[4]]
xas_cmap_Mn = ListedColormap(mpl.colormaps['coolwarm'](np.linspace(0.5, 1.0, data.shape[1]//4-5)), name='xas_cmap_Mn', )
labels_Mn = [r'${\alpha}$-MnO$\mathrm{_2}$', r'0.2M Mn$\mathrm{^{2\!+}}$']

# 图 B
subfig = fig.add_subfigure(gs[0, 0], zorder=0)
ax = subfig.add_subplot()
ax.set_position([0, 0, 1.0, 1.0])
ax.set_box_aspect(0.8)

ax.plot(data.iloc[:, 4], data.iloc[:, 7]+0.08, ls='-', color=colors[4], label=r'ref.MnO')
ax.plot(data.iloc[:, 0], data.iloc[:, 3]+0.08, ls='-', color=colors[5], label=r'ref.Mn$\mathrm{_2O_3}$')
ax.plot(data.iloc[:, 8], data.iloc[:, 11]+0.08, ls='-', color=colors[3], label=r'ref.Mn$\mathrm{O_2}$')
ax.legend(loc='upper left', bbox_to_anchor=(0.0, 1.02), ncols=1, frameon=False, fontsize=9, labelcolor='linecolor', columnspacing=0.0)

ax.plot(FD1st.axes_manager[0].axis, FD1st.data, ls='-', color='k', label=None)
ax.plot(fit_Mn_II.iloc[:, 0], fit_Mn_II.iloc[:, 1], ls='--', color=colors[4], label=None)
ax.plot(fit_Mn_IV.iloc[:, 0], fit_Mn_IV.iloc[:, 1], ls='--', color=colors[3], label=None)
ax.plot(fit_Mn_III_1.iloc[:, 0], fit_Mn_III_1.iloc[:, 1] + fit_Mn_III_2.iloc[:, 1] + fit_Mn_III_3.iloc[:, 1], ls='--', color=colors[5], label=None)
ax.plot(FD1st_model.axes_manager[0].axis, FD1st_model.as_signal().data, ls='--', color=colors[3], label=None)

ax.set_xlim(6535.0, 6550.0)
ax.set_xlabel(r'Energy (eV)', fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=5, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=2.5, offset=0))
ax.set_ylim(-0.01, 0.20)
ax.set_ylabel(r'Relative Intensity (arb.u.)', fontsize=11,)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.05))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.025))

# ax.text(-0.25, 1.0, r'b', transform=ax.transAxes, fontsize=14, va='center', ha='right', fontfamily='Arial', fontweight='bold')

ax.axvline(x=6540.3, ymax=0.65, ls='--')
ax.axvline(x=6543.0, ymax=0.7, ls='--')

plt.savefig(path.joinpath(path_out, r'PreEdge_18_300_V3_0.tif'), transparent=False, pad_inches=0.05, bbox_inches='tight', dpi=300, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'PreEdge_18_600_V3_0.tif'), transparent=False, pad_inches=0.05, bbox_inches='tight', dpi=600, pil_kwargs={"compression": "tiff_lzw"})
plt.show()

## FD1st ex situ sample

#### 直接做 References, III + IV

In [ ]:
# 数据清洗
Mn_IV = data.iloc[:, [8, 11]].copy().dropna(axis=0, how='all')
Mn_III = data.iloc[:, [0, 3]].copy().dropna(axis=0, how='all')

Mn_IV.iloc[115:, 1] = 0
Mn_III.iloc[97:, 1] = 0

# 将 refMn_I//III/IV 做成一个 scale model
from hyperspy.axes import DataAxis
energy_refMn_IV = Mn_IV.iloc[:, 0].to_numpy(dtype='float64')
energy_refMn_IV = DataAxis(axis=energy_refMn_IV, index_in_array=None, name='Energy', units='eV',)
refMn_IV = hs.signals.Signal1D(Mn_IV.iloc[:, 1], axes=[energy_refMn_IV,])
model_refMn_IV = hs.model.components1D.ScalableFixedPattern(refMn_IV) 

energy_refMn_III = Mn_III.iloc[:, 0].to_numpy(dtype='float64')
energy_refMn_III = DataAxis(axis=energy_refMn_III, index_in_array=None, name='Energy', units='eV',)
refMn_III = hs.signals.Signal1D(Mn_III.iloc[:, 1], axes=[energy_refMn_III,])
model_refMn_III = hs.model.components1D.ScalableFixedPattern(refMn_III) 

# 初始化上述的模型
model_refMn_IV.name= r'Mn_IV'
model_refMn_IV.xscale.value=1.0
model_refMn_IV.shift.value=0
# model_refMn_IV.set_parameters_not_free(parameter_name_list=['xscale', 'shift',])
model_refMn_IV.yscale.bmin=0.0

model_refMn_III.name= r'Mn_III'
model_refMn_III.xscale.value=1.0
model_refMn_III.shift.value=0
# model_refMn_III.set_parameters_not_free(parameter_name_list=['xscale', 'shift',])
model_refMn_III.yscale.bmin=0.0

In [ ]:
Mn_aMnO2 = data.iloc[:, [20, 23]].copy().dropna(axis=0, how='all')

In [ ]:
from hyperspy.axes import DataAxis
energy_aMnO2 = Mn_aMnO2.iloc[:, 0].to_numpy(dtype='float64')
energy_aMnO2 = DataAxis(axis=energy_aMnO2, index_in_array=None, name='Energy', units='eV',)
aMnO2 = hs.signals.Signal1D(Mn_aMnO2.iloc[:, 1], axes=[energy_aMnO2,])
aMnO2 = aMnO2.isig[6533.0:].deepcopy()
aMnO2.plot()

In [ ]:
aMnO2_model = aMnO2.create_model()
aMnO2_model.extend((model_refMn_III, model_refMn_IV))
aMnO2_model.set_signal_range(6537.0, 6550.0)

In [ ]:
%matplotlib inline
plt.close('all')
aMnO2_model.fit(optimizer="lm", bounded=True, return_info=True) 
aMnO2_model.plot(plot_components=True, plot_residual=False)
plt.savefig(path.joinpath(path_out, r'PreEdge_19_300_V3_0.tif'), pad_inches=0.05, bbox_inches='tight', dpi=300, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'PreEdge_19_600_V3_0.tif'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
aMnO2_model.print_current_values()

In [ ]:
# 转化成 DataFrame
fit_Mn_IV = pd.DataFrame([aMnO2_model.axes_manager[0].axis, aMnO2_model.as_signal(component_list=['Mn_IV']).data])
fit_Mn_IV = fit_Mn_IV.T
fit_Mn_IV.dropna(how='any', inplace=True)

fit_Mn_III = pd.DataFrame([aMnO2_model.axes_manager[0].axis, aMnO2_model.as_signal(component_list=['Mn_III']).data])
fit_Mn_III = fit_Mn_III.T
fit_Mn_III.dropna(how='any', inplace=True)

# 计算比例
pre_edge_ratio_Mn_IV = scipy.integrate.simpson(y=fit_Mn_IV.iloc[:, 1], x=fit_Mn_IV.iloc[:, 0])/(scipy.integrate.simpson(y=fit_Mn_IV.iloc[:, 1], x=fit_Mn_IV.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_III.iloc[:, 1], x=fit_Mn_III.iloc[:, 0]))
pre_edge_ratio_Mn_III = scipy.integrate.simpson(y=fit_Mn_III.iloc[:, 1], x=fit_Mn_III.iloc[:, 0])/(scipy.integrate.simpson(y=fit_Mn_IV.iloc[:, 1], x=fit_Mn_IV.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_III.iloc[:, 1], x=fit_Mn_III.iloc[:, 0]))
pre_edge_ratio_Mn_IV, pre_edge_ratio_Mn_III

In [ ]:
Pre_Edge_aMnO2 = xr.Dataset(
    data_vars=dict(
        aMnO2 = (['Energy_aMnO2'], aMnO2.data),
        fit = (['Energy_fit'], aMnO2_model.as_signal().data),
        fit_Mn_IV = (['Energy_fit'], aMnO2_model.as_signal(component_list=['Mn_IV']).data),
        fit_Mn_III = (['Energy_fit'], aMnO2_model.as_signal(component_list=['Mn_III']).data),
        ),
    coords=dict(
        Energy_aMnO2 = aMnO2.axes_manager[0].axis,
        Energy_fit= aMnO2_model.axes_manager[0].axis,
    ),
    attrs=dict(
        ratio_Mn_IV = pre_edge_ratio_Mn_IV,
        ratio_Mn_III = pre_edge_ratio_Mn_III,
    ),
)

aMnO2_model.save_parameters2file(path.joinpath(path_out, r'FD1st_exsitu_model_parameters_component_ref_2'))
aMnO2_model.save(path.joinpath(path_out, r'FD1st_exsitu_model_parameters_component_ref_2'), 'model', overwrite=True)
Pre_Edge_aMnO2.to_netcdf(path.joinpath(path_out, r'FD1st_exsitu_component_ref_2.NETCDF4'), engine="h5netcdf")

In [ ]:
# 画图
%matplotlib inline
fig = plt.figure(figsize=(7, 2.5))
gs = gridspec.GridSpec(1, 2, width_ratios=[1, 1], height_ratios=None, wspace=0.0, hspace=0.0, figure=fig)

xas_colors_Mn = [colors[0], colors[4]]
xas_cmap_Mn = ListedColormap(mpl.colormaps['coolwarm'](np.linspace(0.5, 1.0, data.shape[1]//4-5)), name='xas_cmap_Mn', )
labels_Mn = [r'${\alpha}$-MnO$\mathrm{_2}$', r'0.2M Mn$\mathrm{^{2\!+}}$']

# 图 B
subfig = fig.add_subfigure(gs[0, 0], zorder=0)
ax = subfig.add_subplot()
ax.set_position([0, 0, 1.0, 1.0])
ax.set_box_aspect(0.8)

ax.plot(data.iloc[:, 4], data.iloc[:, 7]+0.08, ls='-', color=colors[4], label=r'ref.MnO')
ax.plot(data.iloc[:, 0], data.iloc[:, 3]+0.08, ls='-', color=colors[5], label=r'ref.Mn$\mathrm{_2O_3}$')
ax.plot(data.iloc[:, 8], data.iloc[:, 11]+0.08, ls='-', color=colors[3], label=r'ref.Mn$\mathrm{O_2}$')
ax.legend(loc='upper left', bbox_to_anchor=(0.0, 1.02), ncols=1, frameon=False, fontsize=9, labelcolor='linecolor', columnspacing=0.0)

ax.plot(aMnO2.axes_manager[0].axis, aMnO2.data, ls='-', color='k', label=None)
ax.plot(fit_Mn_III.iloc[:, 0], fit_Mn_III.iloc[:, 1], ls='--', color=colors[5], label=None)
ax.plot(fit_Mn_IV.iloc[:, 0], fit_Mn_IV.iloc[:, 1], ls='--', color=colors[3], label=None)
ax.plot(aMnO2_model.axes_manager[0].axis, aMnO2_model.as_signal().data, ls='--', color=colors[3], label=None)

ax.set_xlim(6535.0, 6550.0)
ax.set_xlabel(r'Energy (eV)', fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=5, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=2.5, offset=0))
ax.set_ylim(-0.01, 0.20)
ax.set_ylabel(r'Relative Intensity (arb.u.)', fontsize=11,)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.05))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.025))

# ax.text(-0.25, 1.0, r'b', transform=ax.transAxes, fontsize=14, va='center', ha='right', fontfamily='Arial', fontweight='bold')

ax.axvline(x=6540.3, ymax=0.65, ls='--')
ax.axvline(x=6543.0, ymax=0.7, ls='--')

plt.savefig(path.joinpath(path_out, r'PreEdge_20_300_V3_0.tif'), transparent=False, pad_inches=0.05, bbox_inches='tight', dpi=300, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'PreEdge_20_600_V3_0.tif'), transparent=False, pad_inches=0.05, bbox_inches='tight', dpi=600, pil_kwargs={"compression": "tiff_lzw"})
plt.show()

# V2

#### 找到所有的 data, 并除去背景

In [ ]:
def index(energy, value):
    return np.argmin(abs(energy-value))

In [ ]:
root_dir = path(r'C:\Users\chengliu\OneDrive - UAB\Short-Term\PreEdge\RawData\MnO')
dat_files = list(root_dir.rglob('*.dat'))
print(dat_files)
df=[]
for file in dat_files:
    data = pd.read_csv(file, sep='\s+', skiprows=0, comment='#', header=0, index_col=None, usecols=[0, 1, 2, 3])
    data_baseline = data.iloc[:, 2:4].copy().dropna(axis=0, how='any', ignore_index=True, inplace=False)
    data = data.iloc[:, 0:2].copy().dropna(axis=0, how='any', ignore_index=True, inplace=False)
    data = data.truncate(before=index(data.iloc[:, 0], data_baseline.iloc[0, 0]), after=index(data.iloc[:, 0], data_baseline.iloc[-1, 0]), axis=0, copy=True).reset_index(drop=True)
    data2 = data.iloc[:,1] - data_baseline.iloc[:,1]
    df.append(data)
    df.append(data_baseline.iloc[:,1])
    df.append(data2)  
data = pd.concat(df, ignore_index=True, axis=1)
data.to_csv(path.joinpath(path_out, r'PreEdge.csv'), index=None, header=True)

#### 读取数据

In [ ]:
path_file = path(r'C:\Users\chengliu\OneDrive - UAB\Short-Term\PreEdge\PreEdge1.csv')
data = pd.read_csv(path_file, sep=',', comment='#', header=0, index_col=None)
data

In [ ]:
# 画图
%matplotlib inline
fig = plt.figure(figsize=(7, 2.5))
gs = gridspec.GridSpec(1, 2, width_ratios=[1, 1], height_ratios=None, wspace=0.0, hspace=0.0, figure=fig)

xas_colors_Mn = [colors[0], colors[4]]
xas_cmap_Mn = ListedColormap(mpl.colormaps['coolwarm'](np.linspace(0.5, 1.0, data.shape[1]//4-6)), name='xas_cmap_Mn', )
labels_Mn = [r'${\alpha}$-MnO$\mathrm{_2}$', r'0.2M Mn$\mathrm{^{2\!+}}$']

# 图 B
subfig = fig.add_subfigure(gs[0, 0], zorder=0)
ax = subfig.add_subplot()
ax.set_position([0, 0, 1.0, 1.0])
ax.set_box_aspect(0.8)

ax.plot(data.iloc[:, 4], data.iloc[:, 7]+0.06, ls='-', color=colors[4], label=r'ref.MnO')
ax.plot(data.iloc[:, 0], data.iloc[:, 3]+0.06, ls='-', color=colors[5], label=r'ref.Mn$\mathrm{_2O_3}$')
ax.plot(data.iloc[:, 8], data.iloc[:, 11]+0.06, ls='-', color=colors[3], label=r'ref.Mn$\mathrm{O_2}$')
ax.plot(data.iloc[:, 16], data.iloc[:, 19]+0.06, ls='-', color=colors[4], label=r'ref.Mn$\mathrm{^{2+}}$')
ax.plot(data.iloc[:, 12], data.iloc[:, 15]+0.06, ls='-', color=colors[3], label=r'ref.$\mathrm{\alpha}$Mn$\mathrm{O_2}$')

ax.legend(loc='upper left', bbox_to_anchor=(0.0, 1.02), ncols=1, frameon=False, fontsize=9, labelcolor='linecolor', columnspacing=0.0)

for i in range(data.shape[1]//4-6):
    ax.plot(data.iloc[:, i*4+20], data.iloc[:, i*4+23], c=xas_cmap_Mn.colors[i], ls='-', label=None)

ax.set_xlim(6530.0, 6550.0)
ax.set_xlabel(r'Energy (eV)', fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=5, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=2.5, offset=0))
ax.set_ylim(-0.01, 0.20)
ax.set_ylabel(r'Relative Intensity (arb.u.)', fontsize=11,)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.05))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.025))

# Add colorbar and adjust ticks afterwards
colorbar = Colorbar(ax=ax.inset_axes([0.5, 0.8, 0.4, 0.07]), location='bottom', orientation='horizontal', 
                      cmap=xas_cmap_Mn, ticklocation='top', spacing = 'proportional', drawedges=False)
colorbar.set_ticks([])
colorbar.outline.set_visible(False)
ax.text(0.5, 0.94, 'OCV', transform=ax.transAxes, fontsize=9,
        va='top', ha='left', fontfamily='Arial', )
ax.text(0.65, 0.94, r'Discharge', transform=ax.transAxes, fontsize=9,
        va='top', ha='left', fontfamily='Arial', )

# ax.text(-0.25, 1.0, r'b', transform=ax.transAxes, fontsize=14, va='center', ha='right', fontfamily='Arial', fontweight='bold')

ax.axvline(x=6540.3, ymax=0.75, ls='--')
ax.axvline(x=6543.0, ymax=0.75, ls='--')

plt.savefig(path.joinpath(path_out, r'PreEdge_1_300_V1_0.tif'), transparent=False, pad_inches=0.05, bbox_inches='tight', dpi=300, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'PreEdge_1_600_V1_0.tif'), transparent=False, pad_inches=0.05, bbox_inches='tight', dpi=600, pil_kwargs={"compression": "tiff_lzw"})
plt.show()

## Mn2+ Solution

#### 拟合峰形, pseudo-Voigt 峰 - 两个

In [ ]:
pv1 = hs.model.components1D.SplitVoigt(centre=6540.3, sigma1=0.83, sigma2=1.1, fraction=0.45)
pv1.name = r'Mn_II_1'
# pv1.set_parameters_not_free(parameter_name_list=['sigma1', 'sigma2', 'fraction'])
pv1.set_parameters_not_free(parameter_name_list=['fraction', 'sigma2',])
pv1.centre.bmin = 6537.5
pv1.centre.bmax = 6542.5
pv1.A.bmin = 0

pv2 = hs.model.components1D.SplitVoigt(centre=6540.0, sigma1=0.83, sigma2=1.1, fraction=0.45)
pv2.name = r'Mn_II_2'
pv2.centre.bmin = 6537.5
pv2.centre.bmax = 6542.5
pv2.A.bmin = 0
pv2.sigma1.twin = pv1.sigma1
pv2.sigma2.twin = pv1.sigma2
pv2.fraction.twin = pv1.fraction

In [ ]:
Mn_Mn_2 = data.iloc[:, [16, 19]].copy().dropna(axis=0, how='all')

In [ ]:
from hyperspy.axes import DataAxis
energy_Mn_2 = Mn_Mn_2.iloc[:, 0].to_numpy(dtype='float64')
energy_Mn_2 = DataAxis(axis=energy_Mn_2, index_in_array=None, name='Energy', units='eV',)
Mn_2 = hs.signals.Signal1D(Mn_Mn_2.iloc[:, 1], axes=[energy_Mn_2,])
Mn_2 = Mn_2.isig[6530.0:].deepcopy()
Mn_2.plot()

In [ ]:
Mn_2_model = Mn_2.create_model()
Mn_2_model.extend((pv1, pv2))
Mn_2_model.set_signal_range(6538.0, 6544.0)

In [ ]:
%matplotlib inline
plt.close('all')
Mn_2_model.fit(optimizer="lm", bounded=True, return_info=False) 
Mn_2_model.plot(plot_components=True, plot_residual=True)
plt.savefig(path.joinpath(path_out, r'PreEdge_2_300_V1_0.tif'), pad_inches=0.05, bbox_inches='tight', dpi=300, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'PreEdge_2_600_V1_0.tif'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
Mn_2_model.print_current_values()

In [ ]:
# 转化成 DataFrame
fit_Mn_II_1 = pd.DataFrame([Mn_2_model.axes_manager[0].axis, Mn_2_model.as_signal(component_list=['Mn_II_1']).data])
fit_Mn_II_1 = fit_Mn_II_1.T
fit_Mn_II_1.dropna(how='any', inplace=True)

fit_Mn_II_2 = pd.DataFrame([Mn_2_model.axes_manager[0].axis, Mn_2_model.as_signal(component_list=['Mn_II_2']).data])
fit_Mn_II_2 = fit_Mn_II_2.T
fit_Mn_II_2.dropna(how='any', inplace=True)

In [ ]:
Pre_Edge_Mn_2 = xr.Dataset(
    data_vars=dict(
        Mn_2 = (['Energy_Mn_2'], Mn_2.data),
        fit = (['Energy_fit'], Mn_2_model.as_signal().data),
        fit_Mn_II_1 = (['Energy_fit'], Mn_2_model.as_signal(component_list=['Mn_II_1']).data),
        fit_Mn_II_2 = (['Energy_fit'], Mn_2_model.as_signal(component_list=['Mn_II_2']).data),
        ),
    coords=dict(
        Energy_Mn_2 = Mn_2.axes_manager[0].axis,
        Energy_fit= Mn_2_model.axes_manager[0].axis,
    ),
)

Mn_2_model.save_parameters2file(path.joinpath(path_out, r'Mn_2_model_parameters_component_pv_2'))
Mn_2_model.save(path.joinpath(path_out, r'Mn_2_model_parameters_component_pv_2'), 'model', overwrite=True)
Pre_Edge_Mn_2.to_netcdf(path.joinpath(path_out, r'Mn_2_component_pv_2.NETCDF4'), engine="h5netcdf")

In [ ]:
# 画图
%matplotlib inline
fig = plt.figure(figsize=(7, 2.5))
gs = gridspec.GridSpec(1, 2, width_ratios=[1, 1], height_ratios=None, wspace=0.0, hspace=0.0, figure=fig)

xas_colors_Mn = [colors[0], colors[4]]
xas_cmap_Mn = ListedColormap(mpl.colormaps['coolwarm'](np.linspace(0.5, 1.0, data.shape[1]//4-5)), name='xas_cmap_Mn', )
labels_Mn = [r'${\alpha}$-MnO$\mathrm{_2}$', r'0.2M Mn$\mathrm{^{2\!+}}$']

# 图 B
subfig = fig.add_subfigure(gs[0, 0], zorder=0)
ax = subfig.add_subplot()
ax.set_position([0, 0, 1.0, 1.0])
ax.set_box_aspect(0.8)

ax.plot(data.iloc[:, 4], data.iloc[:, 7]+0.08, ls='-', color=colors[4], label=r'ref.MnO')
ax.plot(data.iloc[:, 0], data.iloc[:, 3]+0.08, ls='-', color=colors[5], label=r'ref.Mn$\mathrm{_2O_3}$')
ax.plot(data.iloc[:, 8], data.iloc[:, 11]+0.08, ls='-', color=colors[3], label=r'ref.Mn$\mathrm{O_2}$')
ax.legend(loc='upper left', bbox_to_anchor=(0.0, 1.02), ncols=1, frameon=False, fontsize=9, labelcolor='linecolor', columnspacing=0.0)

ax.plot(data.iloc[:, 16], data.iloc[:, 19], ls='-', color='k', label=None)
# ax.plot(fit_Mn_II_2.iloc[:, 0], fit_Mn_II_2.iloc[:, 1], ls='--', color=colors[4], label=None)
# ax.plot(fit_Mn_II_1.iloc[:, 0], fit_Mn_II_1.iloc[:, 1], ls='--', color=colors[4], label=None)
ax.plot(fit_Mn_II_1.iloc[:, 0], fit_Mn_II_1.iloc[:, 1] + fit_Mn_II_2.iloc[:, 1], ls='--', color=colors[4], label=None)
# ax.plot(Mn_2_model.axes_manager[0].axis, Mn_2_model.as_signal().data, ls='--', color=colors[3], label=None)

ax.set_xlim(6535.0, 6550.0)
ax.set_xlabel(r'Energy (eV)', fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=5, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=2.5, offset=0))
ax.set_ylim(-0.01, 0.24)
ax.set_ylabel(r'Relative Intensity (arb.u.)', fontsize=11,)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.06))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.03))

# ax.text(-0.25, 1.0, r'b', transform=ax.transAxes, fontsize=14, va='center', ha='right', fontfamily='Arial', fontweight='bold')

ax.axvline(x=6540.3, ymax=0.65, ls='--')
ax.axvline(x=6543.0, ymax=0.7, ls='--')

plt.savefig(path.joinpath(path_out, r'PreEdge_3_300_V1_0.tif'), transparent=False, pad_inches=0.05, bbox_inches='tight', dpi=300, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'PreEdge_3_600_V1_0.tif'), transparent=False, pad_inches=0.05, bbox_inches='tight', dpi=600, pil_kwargs={"compression": "tiff_lzw"})
plt.show()

### 拟合 Mn(III) Reference

#### 拟合峰形, pseudo-Voigt 峰 - 三个

In [ ]:
pv1 = hs.model.components1D.SplitVoigt(centre=6540.7, sigma1=0.83, sigma2=1.1, fraction=0.45)
pv1.name = r'Mn_III_1'
# pv1.set_parameters_not_free(parameter_name_list=['sigma1', 'sigma2', 'fraction'])
pv1.set_parameters_not_free(parameter_name_list=['sigma2', 'fraction'])
pv1.centre.bmin = 6538.0
pv1.centre.bmax = 6542.5
pv1.A.bmin = 0

pv2 = hs.model.components1D.SplitVoigt(centre=6543.5, sigma1=0.83, sigma2=1.1, fraction=0.45)
pv2.name = r'Mn_III_2'
pv2.centre.bmin = 6543.0
pv2.centre.bmax = 6545.0
pv2.A.bmin = 0
pv2.sigma1.twin = pv1.sigma1
pv2.sigma2.twin = pv1.sigma2
pv2.fraction.twin = pv1.fraction

pv3 = hs.model.components1D.SplitVoigt(centre=6542.3, sigma1=0.83, sigma2=1.1, fraction=0.45)
pv3.name = r'Mn_III_3'
# pv3.centre.bmin = pv1.centre.bmax - 0.5
# pv3.centre.bmax = pv2.centre.bmin + 0.5
pv3.A.bmin = 0
pv3.sigma1.twin = pv1.sigma1
pv3.sigma2.twin = pv1.sigma2
pv3.fraction.twin = pv1.fraction

In [ ]:
Mn_Mn_III = data.iloc[:, [0, 3]].copy().dropna(axis=0, how='all')

In [ ]:
from hyperspy.axes import DataAxis
energy_Mn_Mn_III = Mn_Mn_III.iloc[:, 0].to_numpy(dtype='float64')
energy_Mn_Mn_III = DataAxis(axis=energy_Mn_Mn_III, index_in_array=None, name='Energy', units='eV',)
Mn_III = hs.signals.Signal1D(Mn_Mn_III.iloc[:, 1], axes=[energy_Mn_Mn_III,])
Mn_III = Mn_III.isig[6535.0:].deepcopy()
Mn_III.plot()

In [ ]:
Mn_III_model = Mn_III.create_model()
Mn_III_model.extend((pv1, pv2, pv3))
Mn_III_model.set_signal_range(6537.0, 6548.0)

In [ ]:
%matplotlib inline
plt.close('all')
Mn_III_model.fit(optimizer="lm", bounded=True, return_info=False) 
Mn_III_model.plot(plot_components=True, plot_residual=True)
plt.savefig(path.joinpath(path_out, r'PreEdge_4_300_V1_0.tif'), pad_inches=0.05, bbox_inches='tight', dpi=300, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'PreEdge_4_600_V1_0.tif'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
Mn_III_model.print_current_values()

In [ ]:
# 转化成 DataFrame
fit_Mn_III_1 = pd.DataFrame([Mn_III_model.axes_manager[0].axis, Mn_III_model.as_signal(component_list=['Mn_III_1']).data])
fit_Mn_III_1 = fit_Mn_III_1.T
fit_Mn_III_1.dropna(how='any', inplace=True)

fit_Mn_III_2 = pd.DataFrame([Mn_III_model.axes_manager[0].axis, Mn_III_model.as_signal(component_list=['Mn_III_2']).data])
fit_Mn_III_2 = fit_Mn_III_2.T
fit_Mn_III_2.dropna(how='any', inplace=True)

fit_Mn_III_3 = pd.DataFrame([Mn_III_model.axes_manager[0].axis, Mn_III_model.as_signal(component_list=['Mn_III_3']).data])
fit_Mn_III_3 = fit_Mn_III_3.T
fit_Mn_III_3.dropna(how='any', inplace=True)

In [ ]:
Pre_Edge_Mn_III = xr.Dataset(
    data_vars=dict(
        Mn_III = (['Energy_Mn_III'], Mn_III.data),
        fit = (['Energy_fit'], Mn_III_model.as_signal().data),
        fit_Mn_III_1 = (['Energy_fit'], Mn_III_model.as_signal(component_list=['Mn_III_1']).data),
        fit_Mn_III_2 = (['Energy_fit'], Mn_III_model.as_signal(component_list=['Mn_III_2']).data),
        fit_Mn_III_3 = (['Energy_fit'], Mn_III_model.as_signal(component_list=['Mn_III_3']).data),
        ),
    coords=dict(
        Energy_Mn_III = Mn_III.axes_manager[0].axis,
        Energy_fit= Mn_III_model.axes_manager[0].axis,
    ),
)

Mn_III_model.save_parameters2file(path.joinpath(path_out, r'Ref_Mn_III_model_parameters_component_pv_3'))
Mn_III_model.save(path.joinpath(path_out, r'Ref_Mn_III_model_parameters_component_pv_3'), 'model', overwrite=True)
Pre_Edge_Mn_III.to_netcdf(path.joinpath(path_out, r'Ref_Mn_III_component_pv_3.NETCDF4'), engine="h5netcdf")

In [ ]:
# 画图
%matplotlib inline
fig = plt.figure(figsize=(7, 2.5))
gs = gridspec.GridSpec(1, 2, width_ratios=[1, 1], height_ratios=None, wspace=0.0, hspace=0.0, figure=fig)

xas_colors_Mn = [colors[0], colors[4]]
xas_cmap_Mn = ListedColormap(mpl.colormaps['coolwarm'](np.linspace(0.5, 1.0, data.shape[1]//4-5)), name='xas_cmap_Mn', )
labels_Mn = [r'${\alpha}$-MnO$\mathrm{_2}$', r'0.2M Mn$\mathrm{^{2\!+}}$']

# 图 B
subfig = fig.add_subfigure(gs[0, 0], zorder=0)
ax = subfig.add_subplot()
ax.set_position([0, 0, 1.0, 1.0])
ax.set_box_aspect(0.8)

ax.plot(data.iloc[:, 4], data.iloc[:, 7]+0.08, ls='-', color=colors[4], label=r'ref.MnO')
ax.plot(data.iloc[:, 0], data.iloc[:, 3]+0.08, ls='-', color=colors[5], label=r'ref.Mn$\mathrm{_2O_3}$')
ax.plot(data.iloc[:, 8], data.iloc[:, 11]+0.08, ls='-', color=colors[3], label=r'ref.Mn$\mathrm{O_2}$')
ax.legend(loc='upper left', bbox_to_anchor=(0.0, 1.02), ncols=1, frameon=False, fontsize=9, labelcolor='linecolor', columnspacing=0.0)

ax.plot(Mn_III.axes_manager[0].axis, Mn_III.data, ls='-', color='k', label=None)
# ax.plot(fit_Mn_III_3.iloc[:, 0], fit_Mn_III_3.iloc[:, 1], ls='--', color=colors[5], label=None)
# ax.plot(fit_Mn_III_2.iloc[:, 0], fit_Mn_III_2.iloc[:, 1], ls='--', color=colors[5], label=None)
# ax.plot(fit_Mn_III_1.iloc[:, 0], fit_Mn_III_1.iloc[:, 1], ls='--', color=colors[5], label=None)
ax.plot(fit_Mn_III_1.iloc[:, 0], fit_Mn_III_1.iloc[:, 1] + fit_Mn_III_2.iloc[:, 1] + fit_Mn_III_3.iloc[:, 1], ls='--', color=colors[5], label=None)
ax.plot(Mn_III_model.axes_manager[0].axis, Mn_III_model.as_signal().data, ls='--', color=colors[3], label=None)

ax.set_xlim(6535.0, 6550.0)
ax.set_xlabel(r'Energy (eV)', fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=5, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=2.5, offset=0))
ax.set_ylim(-0.01, 0.24)
ax.set_ylabel(r'Relative Intensity (arb.u.)', fontsize=11,)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.06))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.03))

# ax.text(-0.25, 1.0, r'b', transform=ax.transAxes, fontsize=14, va='center', ha='right', fontfamily='Arial', fontweight='bold')

ax.axvline(x=6540.3, ymax=0.65, ls='--')
ax.axvline(x=6543.0, ymax=0.7, ls='--')

plt.savefig(path.joinpath(path_out, r'PreEdge_5_300_V1_0.tif'), transparent=False, pad_inches=0.05, bbox_inches='tight', dpi=300, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'PreEdge_5_600_V1_0.tif'), transparent=False, pad_inches=0.05, bbox_inches='tight', dpi=600, pil_kwargs={"compression": "tiff_lzw"})
plt.show()

### 验证 Prisitine alpha MnO2

#### 1.直接做 References, II + III + IV

In [ ]:
# 数据清洗
Mn_IV = data.iloc[:, [8, 11]].copy().dropna(axis=0, how='all')
Mn_III = data.iloc[:, [0, 3]].copy().dropna(axis=0, how='all')
Mn_II = data.iloc[:, [4, 7]].copy().dropna(axis=0, how='all')

Mn_IV.iloc[115:, 1] = 0
Mn_III.iloc[97:, 1] = 0
Mn_II.iloc[58:, 1] = 0

# 将 refMn_I//III/IV 做成一个 scale model
from hyperspy.axes import DataAxis
energy_refMn_IV = Mn_IV.iloc[:, 0].to_numpy(dtype='float64')
energy_refMn_IV = DataAxis(axis=energy_refMn_IV, index_in_array=None, name='Energy', units='eV',)
refMn_IV = hs.signals.Signal1D(Mn_IV.iloc[:, 1], axes=[energy_refMn_IV,])
model_refMn_IV = hs.model.components1D.ScalableFixedPattern(refMn_IV) 

energy_refMn_III = Mn_III.iloc[:, 0].to_numpy(dtype='float64')
energy_refMn_III = DataAxis(axis=energy_refMn_III, index_in_array=None, name='Energy', units='eV',)
refMn_III = hs.signals.Signal1D(Mn_III.iloc[:, 1], axes=[energy_refMn_III,])
model_refMn_III = hs.model.components1D.ScalableFixedPattern(refMn_III) 

energy_refMn_II = Mn_II.iloc[:, 0].to_numpy(dtype='float64')
energy_refMn_II = DataAxis(axis=energy_refMn_II, index_in_array=None, name='Energy', units='eV',)
refMn_II = hs.signals.Signal1D(Mn_II.iloc[:, 1], axes=[energy_refMn_II,])
model_refMn_II = hs.model.components1D.ScalableFixedPattern(refMn_II) 

# 初始化上述的模型
model_refMn_IV.name= r'Mn_IV'
model_refMn_IV.xscale.value=1.0
model_refMn_IV.shift.value=0
model_refMn_IV.set_parameters_not_free(parameter_name_list=['xscale', 'shift',])
# model_refMn_IV.set_parameters_not_free(parameter_name_list=['xscale',])
model_refMn_IV.yscale.bmin=0.0

model_refMn_III.name= r'Mn_III'
model_refMn_III.xscale.value=1.0
model_refMn_III.shift.value=0
model_refMn_III.set_parameters_not_free(parameter_name_list=['xscale', 'shift',])
# model_refMn_III.set_parameters_not_free(parameter_name_list=['xscale',])
model_refMn_III.yscale.bmin=0.0

model_refMn_II.name= r'Mn_II'
model_refMn_II.xscale.value=1.0
model_refMn_II.shift.value=0
model_refMn_II.set_parameters_not_free(parameter_name_list=['xscale', 'shift',])
# model_refMn_II.set_parameters_not_free(parameter_name_list=['xscale',])
model_refMn_II.yscale.bmin=0.0

In [ ]:
Mn_aMnO2 = data.iloc[:, [12, 15]].copy().dropna(axis=0, how='all')

In [ ]:
from hyperspy.axes import DataAxis
energy_aMnO2 = Mn_aMnO2.iloc[:, 0].to_numpy(dtype='float64')
energy_aMnO2 = DataAxis(axis=energy_aMnO2, index_in_array=None, name='Energy', units='eV',)
aMnO2 = hs.signals.Signal1D(Mn_aMnO2.iloc[:, 1], axes=[energy_aMnO2,])
aMnO2 = aMnO2.isig[6535.0:].deepcopy()
aMnO2.plot()

In [ ]:
aMnO2_model = aMnO2.create_model()
aMnO2_model.extend((model_refMn_II, model_refMn_III, model_refMn_IV))
aMnO2_model.set_signal_range(6536.0, 6550.0)

In [ ]:
%matplotlib inline
plt.close('all')
aMnO2_model.fit(optimizer="lm", bounded=True, return_info=False) 
aMnO2_model.plot(plot_components=True, plot_residual=False)
plt.savefig(path.joinpath(path_out, r'PreEdge_6_300_V1_0.tif'), pad_inches=0.05, bbox_inches='tight', dpi=300, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'PreEdge_6_600_V1_0.tif'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
aMnO2_model.print_current_values()

In [ ]:
# 转化成 DataFrame
fit_Mn_IV = pd.DataFrame([aMnO2_model.axes_manager[0].axis, aMnO2_model.as_signal(component_list=['Mn_IV']).data])
fit_Mn_IV = fit_Mn_IV.T
fit_Mn_IV.dropna(how='any', inplace=True)

fit_Mn_III = pd.DataFrame([aMnO2_model.axes_manager[0].axis, aMnO2_model.as_signal(component_list=['Mn_III']).data])
fit_Mn_III = fit_Mn_III.T
fit_Mn_III.dropna(how='any', inplace=True)

fit_Mn_II = pd.DataFrame([aMnO2_model.axes_manager[0].axis, aMnO2_model.as_signal(component_list=['Mn_II']).data])
fit_Mn_II = fit_Mn_II.T
fit_Mn_II.dropna(how='any', inplace=True)

# 计算比例
pre_edge_ratio_Mn_IV = scipy.integrate.simpson(y=fit_Mn_IV.iloc[:, 1], x=fit_Mn_IV.iloc[:, 0])/(scipy.integrate.simpson(y=fit_Mn_IV.iloc[:, 1], x=fit_Mn_IV.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_III.iloc[:, 1], x=fit_Mn_III.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_II.iloc[:, 1], x=fit_Mn_II.iloc[:, 0]))
pre_edge_ratio_Mn_III = scipy.integrate.simpson(y=fit_Mn_III.iloc[:, 1], x=fit_Mn_III.iloc[:, 0])/(scipy.integrate.simpson(y=fit_Mn_IV.iloc[:, 1], x=fit_Mn_IV.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_III.iloc[:, 1], x=fit_Mn_III.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_II.iloc[:, 1], x=fit_Mn_II.iloc[:, 0]))
pre_edge_ratio_Mn_II = scipy.integrate.simpson(y=fit_Mn_II.iloc[:, 1], x=fit_Mn_II.iloc[:, 0])/(scipy.integrate.simpson(y=fit_Mn_IV.iloc[:, 1], x=fit_Mn_IV.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_III.iloc[:, 1], x=fit_Mn_III.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_II.iloc[:, 1], x=fit_Mn_II.iloc[:, 0]))
pre_edge_ratio_Mn_IV, pre_edge_ratio_Mn_III, pre_edge_ratio_Mn_II

In [ ]:
Pre_Edge_aMnO2 = xr.Dataset(
    data_vars=dict(
        aMnO2 = (['Energy_aMnO2'], aMnO2.data),
        fit = (['Energy_fit'], aMnO2_model.as_signal().data),
        fit_Mn_IV = (['Energy_fit'], aMnO2_model.as_signal(component_list=['Mn_IV']).data),
        fit_Mn_III = (['Energy_fit'], aMnO2_model.as_signal(component_list=['Mn_III']).data),
        fit_Mn_II = (['Energy_fit'], aMnO2_model.as_signal(component_list=['Mn_II']).data),
        ),
    coords=dict(
        Energy_aMnO2 = aMnO2.axes_manager[0].axis,
        Energy_fit= aMnO2_model.axes_manager[0].axis,
    ),
    attrs=dict(
        ratio_Mn_IV = pre_edge_ratio_Mn_IV,
        ratio_Mn_III = pre_edge_ratio_Mn_III,
        ratio_Mn_II = pre_edge_ratio_Mn_II,
    ),
)

aMnO2_model.save_parameters2file(path.joinpath(path_out, r'Prisitne_aMnO2_model_parameters_component_ref_3'))
aMnO2_model.save(path.joinpath(path_out, r'Prisitne_aMnO2_model_parameters_component_ref_3'), 'model', overwrite=True)
Pre_Edge_aMnO2.to_netcdf(path.joinpath(path_out, r'Prisitne_aMnO2_component_ref_3.NETCDF4'), engine="h5netcdf")

In [ ]:
# 画图
%matplotlib inline
fig = plt.figure(figsize=(7, 2.5))
gs = gridspec.GridSpec(1, 2, width_ratios=[1, 1], height_ratios=None, wspace=0.0, hspace=0.0, figure=fig)

xas_colors_Mn = [colors[0], colors[4]]
xas_cmap_Mn = ListedColormap(mpl.colormaps['coolwarm'](np.linspace(0.5, 1.0, data.shape[1]//4-5)), name='xas_cmap_Mn', )
labels_Mn = [r'${\alpha}$-MnO$\mathrm{_2}$', r'0.2M Mn$\mathrm{^{2\!+}}$']

# 图 B
subfig = fig.add_subfigure(gs[0, 0], zorder=0)
ax = subfig.add_subplot()
ax.set_position([0, 0, 1.0, 1.0])
ax.set_box_aspect(0.8)

ax.plot(data.iloc[:, 4], data.iloc[:, 7]+0.08, ls='-', color=colors[4], label=r'ref.MnO')
ax.plot(data.iloc[:, 0], data.iloc[:, 3]+0.08, ls='-', color=colors[5], label=r'ref.Mn$\mathrm{_2O_3}$')
ax.plot(data.iloc[:, 8], data.iloc[:, 11]+0.08, ls='-', color=colors[3], label=r'ref.Mn$\mathrm{O_2}$')
ax.legend(loc='upper left', bbox_to_anchor=(0.0, 1.02), ncols=1, frameon=False, fontsize=9, labelcolor='linecolor', columnspacing=0.0)

ax.plot(aMnO2.axes_manager[0].axis, aMnO2.data, ls='-', color='k', label=None)
ax.plot(fit_Mn_II.iloc[:, 0], fit_Mn_II.iloc[:, 1], ls='--', color=colors[4], label=None)
ax.plot(fit_Mn_III.iloc[:, 0], fit_Mn_III.iloc[:, 1], ls='--', color=colors[5], label=None)
ax.plot(fit_Mn_IV.iloc[:, 0], fit_Mn_IV.iloc[:, 1], ls='--', color=colors[3], label=None)
ax.plot(aMnO2_model.axes_manager[0].axis, aMnO2_model.as_signal().data, ls='--', color=colors[3], label=None)

ax.set_xlim(6535.0, 6550.0)
ax.set_xlabel(r'Energy (eV)', fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=5, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=2.5, offset=0))
ax.set_ylim(-0.01, 0.24)
ax.set_ylabel(r'Relative Intensity (arb.u.)', fontsize=11,)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.06))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.03))

# ax.text(-0.25, 1.0, r'b', transform=ax.transAxes, fontsize=14, va='center', ha='right', fontfamily='Arial', fontweight='bold')

ax.axvline(x=6540.3, ymax=0.65, ls='--')
ax.axvline(x=6543.0, ymax=0.7, ls='--')

plt.savefig(path.joinpath(path_out, r'PreEdge_7_300_V1_0.tif'), transparent=False, pad_inches=0.05, bbox_inches='tight', dpi=300, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'PreEdge_7_600_V1_0.tif'), transparent=False, pad_inches=0.05, bbox_inches='tight', dpi=600, pil_kwargs={"compression": "tiff_lzw"})
plt.show()

#### 1.直接做 References, III + IV

In [ ]:
# 数据清洗
Mn_IV = data.iloc[:, [8, 11]].copy().dropna(axis=0, how='all')
Mn_III = data.iloc[:, [0, 3]].copy().dropna(axis=0, how='all')

Mn_IV.iloc[115:, 1] = 0
Mn_III.iloc[97:, 1] = 0

# 将 refMn_I//III/IV 做成一个 scale model
from hyperspy.axes import DataAxis
energy_refMn_IV = Mn_IV.iloc[:, 0].to_numpy(dtype='float64')
energy_refMn_IV = DataAxis(axis=energy_refMn_IV, index_in_array=None, name='Energy', units='eV',)
refMn_IV = hs.signals.Signal1D(Mn_IV.iloc[:, 1], axes=[energy_refMn_IV,])
model_refMn_IV = hs.model.components1D.ScalableFixedPattern(refMn_IV) 

energy_refMn_III = Mn_III.iloc[:, 0].to_numpy(dtype='float64')
energy_refMn_III = DataAxis(axis=energy_refMn_III, index_in_array=None, name='Energy', units='eV',)
refMn_III = hs.signals.Signal1D(Mn_III.iloc[:, 1], axes=[energy_refMn_III,])
model_refMn_III = hs.model.components1D.ScalableFixedPattern(refMn_III) 

# 初始化上述的模型
model_refMn_IV.name= r'Mn_IV'
model_refMn_IV.xscale.value=1.0
model_refMn_IV.shift.value=0
model_refMn_IV.set_parameters_not_free(parameter_name_list=['xscale', 'shift',])
# model_refMn_IV.set_parameters_not_free(parameter_name_list=['xscale',])
model_refMn_IV.yscale.bmin=0.0

model_refMn_III.name= r'Mn_III'
model_refMn_III.xscale.value=1.0
model_refMn_III.shift.value=0
model_refMn_III.set_parameters_not_free(parameter_name_list=['xscale', 'shift',])
# model_refMn_III.set_parameters_not_free(parameter_name_list=['xscale',])
model_refMn_III.yscale.bmin=0.0

In [ ]:
Mn_aMnO2 = data.iloc[:, [12, 15]].copy().dropna(axis=0, how='all')

In [ ]:
from hyperspy.axes import DataAxis
energy_aMnO2 = Mn_aMnO2.iloc[:, 0].to_numpy(dtype='float64')
energy_aMnO2 = DataAxis(axis=energy_aMnO2, index_in_array=None, name='Energy', units='eV',)
aMnO2 = hs.signals.Signal1D(Mn_aMnO2.iloc[:, 1], axes=[energy_aMnO2,])
aMnO2 = aMnO2.isig[6535.0:].deepcopy()
aMnO2.plot()

In [ ]:
aMnO2_model = aMnO2.create_model()
aMnO2_model.extend((model_refMn_III, model_refMn_IV))
aMnO2_model.set_signal_range(6536.0, 6550.0)

In [ ]:
%matplotlib inline
plt.close('all')
aMnO2_model.fit(optimizer="lm", bounded=True, return_info=False) 
aMnO2_model.plot(plot_components=True, plot_residual=False)
plt.savefig(path.joinpath(path_out, r'PreEdge_8_300_V1_0.tif'), pad_inches=0.05, bbox_inches='tight', dpi=300, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'PreEdge_8_600_V1_0.tif'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
aMnO2_model.print_current_values()

In [ ]:
# 转化成 DataFrame
fit_Mn_IV = pd.DataFrame([aMnO2_model.axes_manager[0].axis, aMnO2_model.as_signal(component_list=['Mn_IV']).data])
fit_Mn_IV = fit_Mn_IV.T
fit_Mn_IV.dropna(how='any', inplace=True)

fit_Mn_III = pd.DataFrame([aMnO2_model.axes_manager[0].axis, aMnO2_model.as_signal(component_list=['Mn_III']).data])
fit_Mn_III = fit_Mn_III.T
fit_Mn_III.dropna(how='any', inplace=True)

# 计算比例
pre_edge_ratio_Mn_IV = scipy.integrate.simpson(y=fit_Mn_IV.iloc[:, 1], x=fit_Mn_IV.iloc[:, 0])/(scipy.integrate.simpson(y=fit_Mn_IV.iloc[:, 1], x=fit_Mn_IV.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_III.iloc[:, 1], x=fit_Mn_III.iloc[:, 0]))
pre_edge_ratio_Mn_III = scipy.integrate.simpson(y=fit_Mn_III.iloc[:, 1], x=fit_Mn_III.iloc[:, 0])/(scipy.integrate.simpson(y=fit_Mn_IV.iloc[:, 1], x=fit_Mn_IV.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_III.iloc[:, 1], x=fit_Mn_III.iloc[:, 0]))
pre_edge_ratio_Mn_IV, pre_edge_ratio_Mn_III

In [ ]:
Pre_Edge_aMnO2 = xr.Dataset(
    data_vars=dict(
        aMnO2 = (['Energy_aMnO2'], aMnO2.data),
        fit = (['Energy_fit'], aMnO2_model.as_signal().data),
        fit_Mn_IV = (['Energy_fit'], aMnO2_model.as_signal(component_list=['Mn_IV']).data),
        fit_Mn_III = (['Energy_fit'], aMnO2_model.as_signal(component_list=['Mn_III']).data),
        ),
    coords=dict(
        Energy_aMnO2 = aMnO2.axes_manager[0].axis,
        Energy_fit= aMnO2_model.axes_manager[0].axis,
    ),
    attrs=dict(
        ratio_Mn_IV = pre_edge_ratio_Mn_IV,
        ratio_Mn_III = pre_edge_ratio_Mn_III,
    ),
)

aMnO2_model.save_parameters2file(path.joinpath(path_out, r'Prisitne_aMnO2_model_parameters_component_ref_2'))
aMnO2_model.save(path.joinpath(path_out, r'Prisitne_aMnO2_model_parameters_component_ref_2'), 'model', overwrite=True)
Pre_Edge_aMnO2.to_netcdf(path.joinpath(path_out, r'Prisitne_aMnO2_component_ref_2.NETCDF4'), engine="h5netcdf")

In [ ]:
# 画图
%matplotlib inline
fig = plt.figure(figsize=(7, 2.5))
gs = gridspec.GridSpec(1, 2, width_ratios=[1, 1], height_ratios=None, wspace=0.0, hspace=0.0, figure=fig)

xas_colors_Mn = [colors[0], colors[4]]
xas_cmap_Mn = ListedColormap(mpl.colormaps['coolwarm'](np.linspace(0.5, 1.0, data.shape[1]//4-5)), name='xas_cmap_Mn', )
labels_Mn = [r'${\alpha}$-MnO$\mathrm{_2}$', r'0.2M Mn$\mathrm{^{2\!+}}$']

# 图 B
subfig = fig.add_subfigure(gs[0, 0], zorder=0)
ax = subfig.add_subplot()
ax.set_position([0, 0, 1.0, 1.0])
ax.set_box_aspect(0.8)

ax.plot(data.iloc[:, 4], data.iloc[:, 7]+0.08, ls='-', color=colors[4], label=r'ref.MnO')
ax.plot(data.iloc[:, 0], data.iloc[:, 3]+0.08, ls='-', color=colors[5], label=r'ref.Mn$\mathrm{_2O_3}$')
ax.plot(data.iloc[:, 8], data.iloc[:, 11]+0.08, ls='-', color=colors[3], label=r'ref.Mn$\mathrm{O_2}$')
ax.legend(loc='upper left', bbox_to_anchor=(0.0, 1.02), ncols=1, frameon=False, fontsize=9, labelcolor='linecolor', columnspacing=0.0)

ax.plot(aMnO2.axes_manager[0].axis, aMnO2.data, ls='-', color='k', label=None)
ax.plot(fit_Mn_III.iloc[:, 0], fit_Mn_III.iloc[:, 1], ls='--', color=colors[5], label=None)
ax.plot(fit_Mn_IV.iloc[:, 0], fit_Mn_IV.iloc[:, 1], ls='--', color=colors[3], label=None)
ax.plot(aMnO2_model.axes_manager[0].axis, aMnO2_model.as_signal().data, ls='--', color=colors[3], label=None)

ax.set_xlim(6535.0, 6550.0)
ax.set_xlabel(r'Energy (eV)', fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=5, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=2.5, offset=0))
ax.set_ylim(-0.01, 0.24)
ax.set_ylabel(r'Relative Intensity (arb.u.)', fontsize=11,)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.06))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.03))

# ax.text(-0.25, 1.0, r'b', transform=ax.transAxes, fontsize=14, va='center', ha='right', fontfamily='Arial', fontweight='bold')

ax.axvline(x=6540.3, ymax=0.65, ls='--')
ax.axvline(x=6543.0, ymax=0.7, ls='--')

plt.savefig(path.joinpath(path_out, r'PreEdge_9_300_V1_0.tif'), transparent=False, pad_inches=0.05, bbox_inches='tight', dpi=300, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'PreEdge_9_600_V1_0.tif'), transparent=False, pad_inches=0.05, bbox_inches='tight', dpi=600, pil_kwargs={"compression": "tiff_lzw"})
plt.show()

#### 2.拟合峰形, IV + pseudo-Voigt 峰 - 三个

In [ ]:
# 数据清洗
Mn_IV = data.iloc[:, [8, 11]].copy().dropna(axis=0, how='all')
Mn_IV.iloc[115:, 1] = 0

# 将 refMn_I//III/IV 做成一个 scale model
from hyperspy.axes import DataAxis
energy_refMn_IV = Mn_IV.iloc[:, 0].to_numpy(dtype='float64')
energy_refMn_IV = DataAxis(axis=energy_refMn_IV, index_in_array=None, name='Energy', units='eV',)
refMn_IV = hs.signals.Signal1D(Mn_IV.iloc[:, 1], axes=[energy_refMn_IV,])
model_refMn_IV = hs.model.components1D.ScalableFixedPattern(refMn_IV) 

# 初始化上述的模型
model_refMn_IV.name= r'Mn_IV'
model_refMn_IV.xscale.value=1.0
model_refMn_IV.shift.value=0
# model_refMn_IV.set_parameters_not_free(parameter_name_list=['xscale', 'shift',])
model_refMn_IV.yscale.bmin=0.0

# III 的模型
pv1 = hs.model.components1D.SplitVoigt(centre=6541.05, sigma1=0.893791, sigma2=1.1, fraction=0.45)
pv1.name = r'Mn_III_1'
pv1.set_parameters_not_free(parameter_name_list=['centre', 'sigma1', 'sigma2', 'fraction'])
# pv1.centre.bmin = 6540.3
# pv1.centre.bmax = 6542.5
pv1.A.bmin = 0

pv2 = hs.model.components1D.SplitVoigt(centre=6544.63, sigma1=0.893791, sigma2=1.1, fraction=0.45)
pv2.name = r'Mn_III_2'
pv2.set_parameters_not_free(parameter_name_list=['centre',])
# pv2.centre.bmin = 6543.0
# pv2.centre.bmax = 6545.0
# pv2.A.twin = pv1.A
# pv2.A.twin_function_expr = '0.67912*x'
pv2.A.bmin = 0
pv2.sigma1.twin = pv1.sigma1
pv2.sigma2.twin = pv1.sigma2
pv2.fraction.twin = pv1.fraction

pv3 = hs.model.components1D.SplitVoigt(centre=6543.0, sigma1=0.83, sigma2=1.1, fraction=0.45)
pv3.name = r'Mn_III_3'
pv3.set_parameters_not_free(parameter_name_list=['centre'])
# pv3.centre.bmin = pv1.centre.bmax - 0.5
# pv3.centre.bmax = pv2.centre.bmin + 0.5
# pv3.A.twin = pv1.A
# pv3.A.twin_function_expr = '0.788393*x'
pv3.A.bmin = 0
pv3.sigma1.twin = pv1.sigma1
pv3.sigma2.twin = pv1.sigma2
pv3.fraction.twin = pv1.fraction

In [ ]:
Mn_aMnO2 = data.iloc[:, [12, 15]].copy().dropna(axis=0, how='all')

In [ ]:
from hyperspy.axes import DataAxis
energy_aMnO2 = Mn_aMnO2.iloc[:, 0].to_numpy(dtype='float64')
energy_aMnO2 = DataAxis(axis=energy_aMnO2, index_in_array=None, name='Energy', units='eV',)
aMnO2 = hs.signals.Signal1D(Mn_aMnO2.iloc[:, 1], axes=[energy_aMnO2,])
aMnO2 = aMnO2.isig[6535.0:].deepcopy()
aMnO2.plot()

In [ ]:
aMnO2_model = aMnO2.create_model()
aMnO2_model.extend((model_refMn_IV, pv1, pv2, pv3))
aMnO2_model.set_signal_range(6536.0, 6550.0)

In [ ]:
%matplotlib inline
plt.close('all')
aMnO2_model.fit(optimizer="lm", bounded=True, return_info=False) 
aMnO2_model.plot(plot_components=True, plot_residual=True)
plt.savefig(path.joinpath(path_out, r'PreEdge_10_300_V1_0.tif'), pad_inches=0.05, bbox_inches='tight', dpi=300, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'PreEdge_10_600_V1_0.tif'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
aMnO2_model.print_current_values()

In [ ]:
# 转化成 DataFrame
fit_Mn_IV = pd.DataFrame([aMnO2_model.axes_manager[0].axis, aMnO2_model.as_signal(component_list=['Mn_IV']).data])
fit_Mn_IV = fit_Mn_IV.T
fit_Mn_IV.dropna(how='any', inplace=True)

fit_Mn_III_1 = pd.DataFrame([aMnO2_model.axes_manager[0].axis, aMnO2_model.as_signal(component_list=['Mn_III_1']).data])
fit_Mn_III_1 = fit_Mn_III_1.T
fit_Mn_III_1.dropna(how='any', inplace=True)

fit_Mn_III_2 = pd.DataFrame([aMnO2_model.axes_manager[0].axis, aMnO2_model.as_signal(component_list=['Mn_III_2']).data])
fit_Mn_III_2 = fit_Mn_III_2.T
fit_Mn_III_2.dropna(how='any', inplace=True)

fit_Mn_III_3 = pd.DataFrame([aMnO2_model.axes_manager[0].axis, aMnO2_model.as_signal(component_list=['Mn_III_3']).data])
fit_Mn_III_3 = fit_Mn_III_3.T
fit_Mn_III_3.dropna(how='any', inplace=True)

# 计算比例
pre_edge_ratio_Mn_IV = scipy.integrate.simpson(y=fit_Mn_IV.iloc[:, 1], x=fit_Mn_IV.iloc[:, 0])/(scipy.integrate.simpson(y=fit_Mn_IV.iloc[:, 1], x=fit_Mn_IV.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_III_1.iloc[:, 1], x=fit_Mn_III_1.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_III_2.iloc[:, 1], x=fit_Mn_III_2.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_III_3.iloc[:, 1], x=fit_Mn_III_3.iloc[:, 0]))
pre_edge_ratio_Mn_III = (scipy.integrate.simpson(y=fit_Mn_III_1.iloc[:, 1], x=fit_Mn_III_1.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_III_2.iloc[:, 1], x=fit_Mn_III_2.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_III_3.iloc[:, 1], x=fit_Mn_III_3.iloc[:, 0]))/(scipy.integrate.simpson(y=fit_Mn_IV.iloc[:, 1], x=fit_Mn_IV.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_III_1.iloc[:, 1], x=fit_Mn_III_1.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_III_2.iloc[:, 1], x=fit_Mn_III_2.iloc[:, 0]) + scipy.integrate.simpson(y=fit_Mn_III_3.iloc[:, 1], x=fit_Mn_III_3.iloc[:, 0]))
pre_edge_ratio_Mn_IV, pre_edge_ratio_Mn_III

In [ ]:
Pre_Edge_aMnO2 = xr.Dataset(
    data_vars=dict(
        aMnO2 = (['Energy_aMnO2'], aMnO2.data),
        fit = (['Energy_fit'], aMnO2_model.as_signal().data),
        fit_Mn_IV = (['Energy_fit'], aMnO2_model.as_signal(component_list=['Mn_IV']).data),
        fit_Mn_III_1 = (['Energy_fit'], aMnO2_model.as_signal(component_list=['Mn_III_1']).data),
        fit_Mn_III_2 = (['Energy_fit'], aMnO2_model.as_signal(component_list=['Mn_III_2']).data),
        fit_Mn_III_3 = (['Energy_fit'], aMnO2_model.as_signal(component_list=['Mn_III_3']).data),
        ),
    coords=dict(
        Energy_aMnO2 = aMnO2.axes_manager[0].axis,
        Energy_fit= aMnO2_model.axes_manager[0].axis,
    ),
    attrs=dict(
        ratio_Mn_IV = pre_edge_ratio_Mn_IV,
        ratio_Mn_III = pre_edge_ratio_Mn_III,
    ),
)

aMnO2_model.save_parameters2file(path.joinpath(path_out, r'Prisitne_aMnO2_model_parameters_component_ref_1_pv_3'))
aMnO2_model.save(path.joinpath(path_out, r'Prisitne_aMnO2_model_parameters_component_ref_1_pv_3'), 'model', overwrite=True)
Pre_Edge_aMnO2.to_netcdf(path.joinpath(path_out, r'Prisitne_aMnO2_component_ref_1_pv_3.NETCDF4'), engine="h5netcdf")

In [ ]:
# 画图
%matplotlib inline
fig = plt.figure(figsize=(7, 2.5))
gs = gridspec.GridSpec(1, 2, width_ratios=[1, 1], height_ratios=None, wspace=0.0, hspace=0.0, figure=fig)

xas_colors_Mn = [colors[0], colors[4]]
xas_cmap_Mn = ListedColormap(mpl.colormaps['coolwarm'](np.linspace(0.5, 1.0, data.shape[1]//4-5)), name='xas_cmap_Mn', )
labels_Mn = [r'${\alpha}$-MnO$\mathrm{_2}$', r'0.2M Mn$\mathrm{^{2\!+}}$']

# 图 B
subfig = fig.add_subfigure(gs[0, 0], zorder=0)
ax = subfig.add_subplot()
ax.set_position([0, 0, 1.0, 1.0])
ax.set_box_aspect(0.8)

ax.plot(data.iloc[:, 4], data.iloc[:, 7]+0.08, ls='-', color=colors[4], label=r'ref.MnO')
ax.plot(data.iloc[:, 0], data.iloc[:, 3]+0.08, ls='-', color=colors[5], label=r'ref.Mn$\mathrm{_2O_3}$')
ax.plot(data.iloc[:, 8], data.iloc[:, 11]+0.08, ls='-', color=colors[3], label=r'ref.Mn$\mathrm{O_2}$')
ax.legend(loc='upper left', bbox_to_anchor=(0.0, 1.02), ncols=1, frameon=False, fontsize=9, labelcolor='linecolor', columnspacing=0.0)

ax.plot(aMnO2.axes_manager[0].axis, aMnO2.data, ls='-', color='k', label=None)
# ax.plot(fit_Mn_III_3.iloc[:, 0], fit_Mn_III_3.iloc[:, 1], ls='--', color=colors[5], label=None)
# ax.plot(fit_Mn_III_2.iloc[:, 0], fit_Mn_III_2.iloc[:, 1], ls='--', color=colors[5], label=None)
# ax.plot(fit_Mn_III_1.iloc[:, 0], fit_Mn_III_1.iloc[:, 1], ls='--', color=colors[5], label=None)
ax.plot(fit_Mn_III_1.iloc[:, 0], fit_Mn_III_1.iloc[:, 1] + fit_Mn_III_2.iloc[:, 1] + fit_Mn_III_3.iloc[:, 1], ls='--', color=colors[5], label=None)
ax.plot(fit_Mn_IV.iloc[:, 0], fit_Mn_IV.iloc[:, 1], ls='--', color=colors[3], label=None)
ax.plot(aMnO2_model.axes_manager[0].axis, aMnO2_model.as_signal().data, ls='--', color=colors[3], label=None)

ax.set_xlim(6535.0, 6550.0)
ax.set_xlabel(r'Energy (eV)', fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=5, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=2.5, offset=0))
ax.set_ylim(-0.01, 0.24)
ax.set_ylabel(r'Relative Intensity (arb.u.)', fontsize=11,)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.06))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.03))

# ax.text(-0.25, 1.0, r'b', transform=ax.transAxes, fontsize=14, va='center', ha='right', fontfamily='Arial', fontweight='bold')

ax.axvline(x=6540.3, ymax=0.65, ls='--')
ax.axvline(x=6543.0, ymax=0.7, ls='--')

plt.savefig(path.joinpath(path_out, r'PreEdge_11_300_V1_0.tif'), transparent=False, pad_inches=0.05, bbox_inches='tight', dpi=300, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'PreEdge_11_600_V1_0.tif'), transparent=False, pad_inches=0.05, bbox_inches='tight', dpi=600, pil_kwargs={"compression": "tiff_lzw"})
plt.show()

## OCV

#### 拟合峰形, IV + III (pseudo-Voigt 峰 - 三个) + II (pseudo-Voigt 峰 - 一个)

In [ ]:
# IV 的模型
Mn_IV = data.iloc[:, [8, 11]].copy().dropna(axis=0, how='all')
Mn_IV.iloc[115:, 1] = 0

# 将 IV 做成一个 scale model
from hyperspy.axes import DataAxis
energy_refMn_IV = Mn_IV.iloc[:, 0].to_numpy(dtype='float64')
energy_refMn_IV = DataAxis(axis=energy_refMn_IV, index_in_array=None, name='Energy', units='eV',)
refMn_IV = hs.signals.Signal1D(Mn_IV.iloc[:, 1], axes=[energy_refMn_IV,])
model_refMn_IV = hs.model.components1D.ScalableFixedPattern(refMn_IV) 

# 初始化上述的模型
model_refMn_IV.name= r'Mn_IV'
model_refMn_IV.xscale.value=1.0
model_refMn_IV.shift.value=0
# model_refMn_IV.set_parameters_not_free(parameter_name_list=['xscale', 'shift',])
# model_refMn_IV.set_parameters_not_free(parameter_name_list=['xscale',])
model_refMn_IV.yscale.bmin=0.0

# III 的模型
pv1 = hs.model.components1D.SplitVoigt(centre=6541.05, sigma1=0.893791, sigma2=1.1, fraction=0.45)
pv1.name = r'Mn_III_1'
pv1.set_parameters_not_free(parameter_name_list=['centre', 'sigma1', 'sigma2', 'fraction'])
pv1.A.bmin = 0

pv2 = hs.model.components1D.SplitVoigt(centre=6544.63, sigma1=0.893791, sigma2=1.1, fraction=0.45)
pv2.name = r'Mn_III_2'
pv2.set_parameters_not_free(parameter_name_list=['centre',])
pv2.A.twin = pv1.A
pv2.A.twin_function_expr = '0.2491491*x'
pv2.A.bmin = 0
pv2.sigma1.twin = pv1.sigma1
pv2.sigma2.twin = pv1.sigma2
pv2.fraction.twin = pv1.fraction

pv3 = hs.model.components1D.SplitVoigt(centre=6543.0, sigma1=0.83, sigma2=1.1, fraction=0.45)
pv3.name = r'Mn_III_3'
pv3.set_parameters_not_free(parameter_name_list=['centre'])
pv3.A.twin = pv1.A
pv3.A.twin_function_expr = '0.924138604*x'
pv3.A.bmin = 0
pv3.sigma1.twin = pv1.sigma1
pv3.sigma2.twin = pv1.sigma2
pv3.fraction.twin = pv1.fraction

# II 的模型
pv4 = hs.model.components1D.SplitVoigt(centre=6540.74, sigma1=1.54073, sigma2=1.1, fraction=0.45)
pv4.name = r'Mn_II'
pv4.set_parameters_not_free(parameter_name_list=['centre', 'sigma1', 'sigma2', 'fraction'])
pv4.A.bmin = 0

In [ ]:
root_dir = path(r'C:\Users\chengliu\Desktop\Figure')
dat_files = list(root_dir.rglob('*.dat'))
print(dat_files)
df=[]
for file in dat_files:
    data = pd.read_csv(file, sep='\s+', skiprows=0, comment='#', header=0, index_col=None, usecols=[0, 1, 2, 3])
    data_baseline = data.iloc[:, 2:4].copy().dropna(axis=0, how='any', ignore_index=True, inplace=False)
    data = data.iloc[:, 0:2].copy().dropna(axis=0, how='any', ignore_index=True, inplace=False)
    data = data.truncate(before=index(data.iloc[:, 0], data_baseline.iloc[0, 0]), after=index(data.iloc[:, 0], data_baseline.iloc[-1, 0]), axis=0, copy=True).reset_index(drop=True)
    data2 = data.iloc[:,1] - data_baseline.iloc[:,1]
    df.append(data)
    df.append(data_baseline.iloc[:,1])
    df.append(data2)  
data = pd.concat(df, ignore_index=True, axis=1)
data.to_csv(path.joinpath(path_out, r'PreEdge.csv'), index=None, header=True)

In [ ]:
path_file1 = path(r'C:\Users\chengliu\Desktop\Figure\PreEdge.csv')
data1 = pd.read_csv(path_file1, sep=',', comment='#', header=0, index_col=None)
Mn_OCV = data1.iloc[:, [0, 3]].copy().dropna(axis=0, how='all')

In [ ]:
# Mn_OCV = data.iloc[:, [20, 23]].copy().dropna(axis=0, how='all')

In [ ]:
from hyperspy.axes import DataAxis
energy_OCV = Mn_OCV.iloc[:, 0].to_numpy(dtype='float64')
energy_OCV = DataAxis(axis=energy_OCV, index_in_array=None, name='Energy', units='eV',)
OCV = hs.signals.Signal1D(Mn_OCV.iloc[:, 1], axes=[energy_OCV,])
OCV = OCV.isig[6532.0:].deepcopy()
OCV.plot()

In [ ]:
OCV_model = OCV.create_model()
OCV_model.extend((model_refMn_IV, pv1, pv2, pv3, pv4))
OCV_model.set_signal_range(6538.0, 6546.0)

In [ ]:
%matplotlib widget
plt.close('all')
OCV_model.fit(optimizer="lm", bounded=True, return_info=False) 
OCV_model.plot(plot_components=True, plot_residual=False)
plt.savefig(path.joinpath(path_out, r'PreEdge_12_300_V1_0.tif'), pad_inches=0.05, bbox_inches='tight', dpi=300, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'PreEdge_12_600_V1_0.tif'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
OCV_model.print_current_values()

In [ ]:
# # 转化成 DataFrame
# fit_Mn_II_1 = pd.DataFrame([OCV_model.axes_manager[0].axis, OCV_model.as_signal(component_list=['Mn_II_1']).data])
# fit_Mn_II_1 = fit_Mn_II_1.T
# fit_Mn_II_1.dropna(how='any', inplace=True)

# fit_Mn_II_2 = pd.DataFrame([OCV_model.axes_manager[0].axis, OCV_model.as_signal(component_list=['Mn_II_2']).data])
# fit_Mn_II_2 = fit_Mn_II_2.T
# fit_Mn_II_2.dropna(how='any', inplace=True)

In [ ]:
Pre_Edge_Mn_2 = xr.Dataset(
    data_vars=dict(
        Mn_2 = (['Energy_Mn_2'], Mn_2.data),
        fit = (['Energy_fit'], OCV_model.as_signal().data),
        fit_Mn_II_1 = (['Energy_fit'], OCV_model.as_signal(component_list=['Mn_II_1']).data),
        fit_Mn_II_2 = (['Energy_fit'], OCV_model.as_signal(component_list=['Mn_II_2']).data),
        ),
    coords=dict(
        Energy_Mn_2 = Mn_2.axes_manager[0].axis,
        Energy_fit= OCV_model.axes_manager[0].axis,
    ),
)

OCV_model.save_parameters2file(path.joinpath(path_out, r'OCV_model_parameters_component_pv_2'))
OCV_model.save(path.joinpath(path_out, r'OCV_model_parameters_component_pv_2'), 'model', overwrite=True)
Pre_Edge_Mn_2.to_netcdf(path.joinpath(path_out, r'Mn_2_component_pv_2.NETCDF4'), engine="h5netcdf")

In [ ]:
# 画图
%matplotlib inline
fig = plt.figure(figsize=(7, 2.5))
gs = gridspec.GridSpec(1, 2, width_ratios=[1, 1], height_ratios=None, wspace=0.0, hspace=0.0, figure=fig)

xas_colors_Mn = [colors[0], colors[4]]
xas_cmap_Mn = ListedColormap(mpl.colormaps['coolwarm'](np.linspace(0.5, 1.0, data.shape[1]//4-5)), name='xas_cmap_Mn', )
labels_Mn = [r'${\alpha}$-MnO$\mathrm{_2}$', r'0.2M Mn$\mathrm{^{2\!+}}$']

# 图 B
subfig = fig.add_subfigure(gs[0, 0], zorder=0)
ax = subfig.add_subplot()
ax.set_position([0, 0, 1.0, 1.0])
ax.set_box_aspect(0.8)

ax.plot(data.iloc[:, 4], data.iloc[:, 7]+0.08, ls='-', color=colors[4], label=r'ref.MnO')
ax.plot(data.iloc[:, 0], data.iloc[:, 3]+0.08, ls='-', color=colors[5], label=r'ref.Mn$\mathrm{_2O_3}$')
ax.plot(data.iloc[:, 8], data.iloc[:, 11]+0.08, ls='-', color=colors[3], label=r'ref.Mn$\mathrm{O_2}$')
ax.legend(loc='upper left', bbox_to_anchor=(0.0, 1.02), ncols=1, frameon=False, fontsize=9, labelcolor='linecolor', columnspacing=0.0)

ax.plot(data.iloc[:, 16], data.iloc[:, 19], ls='-', color='k', label=None)
# ax.plot(fit_Mn_II_2.iloc[:, 0], fit_Mn_II_2.iloc[:, 1], ls='--', color=colors[5], label=None)
# ax.plot(fit_Mn_II_1.iloc[:, 0], fit_Mn_II_1.iloc[:, 1], ls='--', color=colors[5], label=None)
ax.plot(fit_Mn_II_1.iloc[:, 0], fit_Mn_II_1.iloc[:, 1] + fit_Mn_II_2.iloc[:, 1], ls='--', color=colors[5], label=None)
ax.plot(OCV_model.axes_manager[0].axis, OCV_model.as_signal().data, ls='--', color=colors[3], label=None)

ax.set_xlim(6535.0, 6550.0)
ax.set_xlabel(r'Energy (eV)', fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=5, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=2.5, offset=0))
ax.set_ylim(-0.01, 0.24)
ax.set_ylabel(r'Relative Intensity (arb.u.)', fontsize=11,)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.06))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.03))

# Add colorbar and adjust ticks afterwards
colorbar = Colorbar(ax=ax.inset_axes([0.5, 0.8, 0.4, 0.07]), location='bottom', orientation='horizontal', 
                      cmap=xas_cmap_Mn, ticklocation='top', spacing = 'proportional', drawedges=False)
colorbar.set_ticks([])
colorbar.outline.set_visible(False)
ax.text(0.5, 0.94, 'OCV', transform=ax.transAxes, fontsize=9,
        va='top', ha='left', fontfamily='Arial', )
ax.text(0.65, 0.94, r'Discharge', transform=ax.transAxes, fontsize=9,
        va='top', ha='left', fontfamily='Arial', )

# ax.text(-0.25, 1.0, r'b', transform=ax.transAxes, fontsize=14, va='center', ha='right', fontfamily='Arial', fontweight='bold')

ax.axvline(x=6540.3, ymax=0.65, ls='--')
ax.axvline(x=6543.0, ymax=0.7, ls='--')

plt.savefig(path.joinpath(path_out, r'PreEdge_3_300_V1_0.tif'), transparent=False, pad_inches=0.05, bbox_inches='tight', dpi=300, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'PreEdge_3_600_V1_0.tif'), transparent=False, pad_inches=0.05, bbox_inches='tight', dpi=600, pil_kwargs={"compression": "tiff_lzw"})
plt.show()

# V1

### Ex situ Data

In [ ]:
def index(energy, value):
    return np.argmin(abs(energy-value))

#### Fitting of Pre-Edge of Pristne

In [ ]:
# 读取数据
path_file = path(r'C:\Users\chengliu\OneDrive - UAB\ICMAB-Data\Zn-Mn\Uno\Result\XAS\ExSitu\αMnO2\XAS\Pristine\2023-CLAESS-2\Results\Pre-Edge\refMnO2_referenece-6538.0-6552.0-6539.2-6545.2')
refMnO2 = pd.read_csv(path.joinpath(path_file, r'pre_edge_baseline_MnO2.dat'), sep='\s+', comment='#', index_col=None, header=0).iloc[:, 0:4]
refMnO2_baseline = refMnO2.iloc[:, 2:4].copy().dropna(axis=0, how='any', ignore_index=True, inplace=False)
refMnO2 = refMnO2.iloc[:, 0:2].copy().dropna(axis=0, how='any', ignore_index=True, inplace=False)
refMnO2 = refMnO2.truncate(before=index(refMnO2.iloc[:, 0], refMnO2_baseline.iloc[0, 0]), after=index(refMnO2.iloc[:, 0], refMnO2_baseline.iloc[-1, 0]), axis=0, copy=True).reset_index(drop=True)

aMnO2 = pd.read_csv(path.joinpath(path_file, r'pre_edge_baseline_Merge_alpha_MnO2_powder.dat'), sep='\s+', comment='#', index_col=None, header=0).iloc[:, 0:4]
aMnO2_baseline = aMnO2.iloc[:, 2:4].copy().dropna(axis=0, how='any', ignore_index=True, inplace=False)
aMnO2 = aMnO2.iloc[:, 0:2].copy().dropna(axis=0, how='any', ignore_index=True, inplace=False)
aMnO2 = aMnO2.truncate(before=index(aMnO2.iloc[:, 0], aMnO2_baseline.iloc[0, 0]), after=index(aMnO2.iloc[:, 0], aMnO2_baseline.iloc[-1, 0]), axis=0, copy=True).reset_index(drop=True)

In [ ]:
from hyperspy.axes import DataAxis
energy_aMnO2_a = aMnO2.iloc[:, 0].to_numpy(dtype='float64')
energy_aMnO2 = DataAxis(axis=energy_aMnO2_a, index_in_array=None, name='Energy', units='eV',)
s_aMnO2 = hs.signals.Signal1D(aMnO2.iloc[:, 1]-aMnO2_baseline.iloc[:, 1], axes=[energy_aMnO2,])
model_pre_edge = s_aMnO2.create_model()

##### Mn(VI) model

In [ ]:
# 将 refMnO2 做成一个 scale model
energy_refMnO2_a = refMnO2.iloc[:, 0].to_numpy(dtype='float64')
energy_refMnO2 = DataAxis(axis=energy_refMnO2_a, index_in_array=None, name='Energy', units='eV',)

s_refMnO2 = hs.signals.Signal1D(refMnO2.iloc[:, 1]- refMnO2_baseline.iloc[:, 1], axes=[energy_refMnO2,])
model_refMnO2 = hs.model.components1D.ScalableFixedPattern(s_refMnO2) 
model_refMnO2.name= r'Mn_IV'

model_refMnO2.xscale.value=1.0
model_refMnO2.set_parameters_not_free(parameter_name_list=['xscale'])

model_refMnO2.yscale.bmin=0.0
model_pre_edge.append(model_refMnO2)
# model_pre_edge.components

model_pre_edge.fit_component(model_refMnO2,  signal_range=(6539.0, 6546.0), bounded=True, only_current=True) # get good values
model_pre_edge.assign_current_values_to_all(components_list = [model_refMnO2]) # apply those good values to all pixels
model_pre_edge.fit_component(model_refMnO2, signal_range=(6539.0, 6546.0), bounded=True, only_current=False)

##### Mn(III) 只用一个高斯峰

In [ ]:
# 一个高斯曲线 model
model_III_1 = hs.model.components1D.GaussianHF()
model_III_1.name = "Mn_III"
model_III_1.centre.bmin = 6539.5
model_III_1.centre.bmax = 6542.0
model_III_1.centre.value = (model_III_1.centre.bmin + model_III_1.centre.bmax)/2
model_III_1.height.bmin = 0
model_III_1.fwhm.bmax=1.5

model_pre_edge.append(model_III_1)
# model_pre_edge.components

model_pre_edge.fit_component(model_III_1,  signal_range=(6539.0, 6542.0), bounded=True, only_current=True) # get good values
model_pre_edge.assign_current_values_to_all(components_list = [model_III_1]) # apply those good values to all pixels
model_pre_edge.fit_component(model_III_1, signal_range=(6539.0, 6542.0), bounded=True, only_current=False)

In [ ]:
model_pre_edge.set_signal_range(6538.0, 6548.0)
model_pre_edge.multifit(bounded=True) # optional final pass may improve fit by letting all components adjust to each other. 
model_pre_edge.plot(plot_components=True)

In [ ]:
model_pre_edge.print_current_values()

In [ ]:
pre_edge_ratio = model_pre_edge.as_signal(component_list=['Mn_IV']).isig[6539.0:6545.0].integrate1D(axis='Energy')/(model_pre_edge.as_signal(component_list=['Mn_III']).isig[6539.0:6545.0].integrate1D(axis='Energy') + model_pre_edge.as_signal(component_list=['Mn_IV']).isig[6539.0:6545.0].integrate1D(axis='Energy'))
pre_edge_ratio.data

In [ ]:
Pre_Edge_aMnO2 = xr.Dataset(
    data_vars=dict(
            aMnO2 = (['Energy_aMnO2'], s_aMnO2.data),
            refMnO2 = (['Energy_refMnO2'], s_refMnO2.data),
            fit = (['Energy_fit'], model_pre_edge.as_signal().data),
            fit_Mn_IV = (['Energy_fit'], model_pre_edge.as_signal(component_list=['Mn_IV']).data),
            fit_Mn_III = (['Energy_fit'], model_pre_edge.as_signal(component_list=['Mn_III']).data),
        ),
    coords=dict(
        Energy_aMnO2 = s_aMnO2.axes_manager[0].axis,
        Energy_refMnO2 = s_refMnO2.axes_manager[0].axis,
        Energy_fit= model_pre_edge.axes_manager[0].axis,
    ),
    attrs=dict(
        ratio_pre_edge_Mn_IV=pre_edge_ratio.data,
    ),
)

model_pre_edge.save_parameters2file(path.joinpath(path_file, r'Prisitne_aMnO2_Model_2_Parameters'))
model_pre_edge.save(path.joinpath(path_file, r'Prisitne_aMnO2_Model_2_Model'), 'model', overwrite=True)
Pre_Edge_aMnO2.to_netcdf(path.joinpath(path_file, r'Prisitne_aMnO2_Model_2.NETCDF4'), engine="h5netcdf")

In [ ]:
# 画图
%matplotlib inline    

plt.close('all')
fig = plt.figure(figsize=(3.3, 2.5))
gs = gridspec.GridSpec(1, 1, width_ratios=None, height_ratios=None,
                        wspace=0, hspace=0, figure=fig)

subfig = fig.add_subfigure(gs[0, 0], zorder=0)
ax = subfig.add_subplot()
ax.set_position([0, 0, 1.0, 1.0])

padding=0.05
ax.plot(refMnO2.iloc[:, 0], refMnO2.iloc[:, 1]+padding, c=colors[0], ls='-', label=r'refMnO$_2$')
ax.plot(refMnO2_baseline.iloc[:, 0], refMnO2_baseline.iloc[:, 1]+padding, ls='--', c=colors[0], label=None)
ax.plot(aMnO2.iloc[:, 0], aMnO2.iloc[:, 1]+padding, c=colors[1], ls='-', label=r'$\mathrm{\alpha MnO_2}$')
ax.plot(aMnO2_baseline.iloc[:, 0], aMnO2_baseline.iloc[:, 1]+padding, ls='--', c=colors[1], label=None)

ax.plot(s_aMnO2.axes_manager[0].axis, s_aMnO2.data, c=colors[1], label=None, zorder=5)
ax.plot(model_pre_edge.axes_manager[0].axis, model_pre_edge.as_signal().data, ls='--', c=colors[2], label='fit', zorder=6)
ax.plot(model_pre_edge.axes_manager[0].axis, model_pre_edge.as_signal(component_list=['Mn_IV']).data, c=colors[0], label=None, alpha=0.5)
ax.fill_between(model_pre_edge.axes_manager[0].axis, y1=0, y2=model_pre_edge.as_signal(component_list=['Mn_IV']).data, facecolor=colors[0], alpha=0.1)
ax.plot(model_pre_edge.axes_manager[0].axis, model_pre_edge.as_signal(component_list=['Mn_III']).data, c=colors[3], label=r'Mn_III', alpha=0.5)
ax.fill_between(model_pre_edge.axes_manager[0].axis, y1=0, y2=model_pre_edge.as_signal(component_list=['Mn_III']).data, facecolor=colors[3], alpha=0.1)

ax.set_xlabel(r'Energy (eV)', fontsize=11,)
ax.set_xlim(6538.0, 6548.0)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=2, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=1, offset=0))

ax.set_ylabel(r'Intensity (a.u.)', fontsize=11,)
ax.set_ylim(-0.005, 0.20)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.05, offset=0))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.025, offset=0))

ax.legend(loc='upper left', bbox_to_anchor=(0.0, 1.02), ncols=1, frameon=False, fontsize=10, labelcolor='linecolor', columnspacing=0.4)
ax.text(0.70, 0.30, f'Mn(IV): {pre_edge_ratio.data[0]:.3f}', horizontalalignment='left', verticalalignment='top', transform=ax.transAxes, fontsize=9)

plt.savefig(path.joinpath(path_out, r'600_Prisitne_aMnO2_Model_2.tif'),  pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.show()

##### Mn(III) 用两个高斯峰

In [ ]:
# 一到两个高斯曲线 model
model_III_1 = hs.model.components1D.GaussianHF()
model_III_1.name = "Mn_III_1"
model_III_1.centre.bmin = 6539.5
model_III_1.centre.bmax = 6541.5
model_III_1.centre.value = (model_III_1.centre.bmin + model_III_1.centre.bmax)/2

model_III_1.height.bmin = 0
model_III_1.fwhm.bmax=1.5

model_III_2 = hs.model.components1D.GaussianHF()
model_III_2.name = "Mn_III_2"
model_III_2.centre.value = 6545.0
model_III_2.centre.bmin = 6539.5
model_III_2.height.bmin = 0
model_III_2.fwhm.bmax=1.5

model_pre_edge.extend([model_III_1, model_III_2])
# model_pre_edge.components

model_pre_edge.fit_component(model_III_1,  signal_range=(6539.0, 6542.0), bounded=True, only_current=True) # get good values
model_pre_edge.assign_current_values_to_all(components_list = [model_III_1]) # apply those good values to all pixels
model_pre_edge.fit_component(model_III_1, signal_range=(6539.0, 6542.0), bounded=True, only_current=False)

model_pre_edge.fit_component(model_III_2,  signal_range=(6539.0, 6542.0), bounded=True, only_current=True) # get good values
model_pre_edge.assign_current_values_to_all(components_list = [model_III_2]) # apply those good values to all pixels
model_pre_edge.fit_component(model_III_2, signal_range=(6539.0, 6542.0), bounded=True, only_current=False)

In [ ]:
pre_edge_ratio = model_pre_edge.as_signal(component_list=['Mn_IV']).isig[6539.0:6545.0].integrate1D(axis='Energy')/(model_pre_edge.as_signal(component_list=['Mn_III_1']).isig[6539.0:6545.0].integrate1D(axis='Energy') + model_pre_edge.as_signal(component_list=['Mn_III_2']).isig[6539.0:6545.0].integrate1D(axis='Energy') + model_pre_edge.as_signal(component_list=['Mn_IV']).isig[6539.0:6545.0].integrate1D(axis='Energy'))
pre_edge_ratio.data

In [ ]:
Pre_Edge_aMnO2_3 = xr.Dataset(
    data_vars=dict(
            aMnO2 = (['Energy_aMnO2'], s_aMnO2.data),
            refMnO2 = (['Energy_refMnO2'], s_refMnO2.data),
            fit = (['Energy_fit'], model_pre_edge.as_signal().data),
            fit_Mn_IV = (['Energy_fit'], model_pre_edge.as_signal(component_list=['Mn_IV']).data),
            fit_Mn_III1 = (['Energy_fit'], model_pre_edge.as_signal(component_list=['Mn_III_1']).data),
            fit_Mn_III2 = (['Energy_fit'], model_pre_edge.as_signal(component_list=['Mn_III_2']).data),        
        ),
    coords=dict(
        Energy_aMnO2 = s_aMnO2.axes_manager[0].axis,
        Energy_refMnO2 = s_refMnO2.axes_manager[0].axis,
        Energy_fit= model_pre_edge.axes_manager[0].axis,
    ),
    attrs=dict(
        ratio_pre_edge_Mn_IV=pre_edge_ratio.data,
    ),
)

model_pre_edge.save_parameters2file(path.joinpath(path_file, r'Prisitne_aMnO2_Model_3_Parameters'))
model_pre_edge.save(path.joinpath(path_file, r'Prisitne_aMnO2_Model_3_Model'), 'model', overwrite=True)
Pre_Edge_aMnO2_3.to_netcdf(path.joinpath(path_file, r'Prisitne_aMnO2_Model_3.NETCDF4'), engine="h5netcdf")

In [ ]:
# 画图
%matplotlib inline    

plt.close('all')
fig = plt.figure(figsize=(3.3, 2.5))
gs = gridspec.GridSpec(1, 1, width_ratios=None, height_ratios=None,
                        wspace=0, hspace=0, figure=fig)

subfig = fig.add_subfigure(gs[0, 0], zorder=0)
ax = subfig.add_subplot()
ax.set_position([0, 0, 1.0, 1.0])

padding=0.04
ax.plot(refMnO2.iloc[:, 0], refMnO2.iloc[:, 1]+padding, c=colors[0], ls='-', label=r'refMnO$_2$')
ax.plot(refMnO2_baseline.iloc[:, 0], refMnO2_baseline.iloc[:, 1]+padding, ls='--', c=colors[0], label=None)
ax.plot(aMnO2.iloc[:, 0], aMnO2.iloc[:, 1]+padding, c=colors[1], ls='-', label=r'$\mathrm{\alpha MnO_2}$')
ax.plot(aMnO2_baseline.iloc[:, 0], aMnO2_baseline.iloc[:, 1]+padding, ls='--', c=colors[1], label=None)

ax.plot(s_aMnO2.axes_manager[0].axis, s_aMnO2.data, c=colors[1], label=None, zorder=5)
ax.plot(model_pre_edge.axes_manager[0].axis, model_pre_edge.as_signal().data, ls='--', c=colors[2], label='fit', zorder=6)
ax.plot(model_pre_edge.axes_manager[0].axis, model_pre_edge.as_signal(component_list=['Mn_IV']).data, c=colors[0], label=None, alpha=0.5)
ax.fill_between(model_pre_edge.axes_manager[0].axis, y1=0, y2=model_pre_edge.as_signal(component_list=['Mn_IV']).data, facecolor=colors[0], alpha=0.1)
ax.plot(model_pre_edge.axes_manager[0].axis, model_pre_edge.as_signal(component_list=['Mn_III_1']).data, c=colors[3], label=r'Mn_III', alpha=0.5)
ax.fill_between(model_pre_edge.axes_manager[0].axis, y1=0, y2=model_pre_edge.as_signal(component_list=['Mn_III_1']).data, facecolor=colors[3], alpha=0.1)

ax.plot(model_pre_edge.axes_manager[0].axis, model_pre_edge.as_signal(component_list=['Mn_III_2']).data, c=colors[3], label=None, alpha=0.5)
ax.fill_between(model_pre_edge.axes_manager[0].axis, y1=0, y2=model_pre_edge.as_signal(component_list=['Mn_III_2']).data, facecolor=colors[3], alpha=0.1)


ax.set_xlabel(r'Energy (eV)', fontsize=11,)
ax.set_xlim(6538.0, 6548.0)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=2, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=1, offset=0))

ax.set_ylabel(r'Intensity (a.u.)', fontsize=11,)
ax.set_ylim(-0.005, 0.18)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.06, offset=0))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.03, offset=0))

ax.legend(loc='upper left', bbox_to_anchor=(0.0, 1.02), ncols=1, frameon=False, fontsize=10, labelcolor='linecolor', columnspacing=0.4)
ax.text(0.7, 0.25, f'Mn(IV): {pre_edge_ratio.data[0]:.3f}', horizontalalignment='left', verticalalignment='top', transform=ax.transAxes, fontsize=10)

plt.savefig(path.joinpath(path_out, r'600_Prisitne_aMnO2_Model_3.tif'), 
            pad_inches=0.05, bbox_inches='tight', dpi=300, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.show()

### Operando Data

#### pre-edge 数据获取

In [ ]:
# 读取数据
path_file = path(r'C:\Users\chengliu\Desktop\AA')
Mn_raw = xr.open_dataset(path.joinpath(path_file, r'Raw_data.NETCDF4'),)
Mn_mcr_2_hm = xr.open_dataset(path.joinpath(path_file, r'PcaMcr_N=2.NETCDF4'))

In [ ]:
MnO2_rest = Mn_raw['Raw_spectrum'].sel(Energy=slice(6530.0, 6630.0)).data - (Mn_mcr_2_hm['MCR_HM_C'][:,1]*Mn_mcr_2_hm['MCR_HM_St'][1,:]).data.T

In [ ]:
MnO2_Rest = xr.DataArray(
    data=MnO2_rest,
    coords={
        'components_number': range(MnO2_rest.shape[1]),
        'Energy': Mn_raw['Raw_spectrum'].sel(Energy=slice(6530.0, 6630.0))['Energy'],
    },
    dims=['Energy', 'components_number'],
    name = r'MnO2_Rest',
)

##### PCA 降噪

In [ ]:
data = scp.NDDataset(data=MnO2_Rest.data.T,
                     name="opPre_Edge_Mn_Oct2022",
                     author="Cheng Liu, and Dino Tonti",
                     description="XANES obtained from CLAESS, alba, July2023",
                     history="creation",
                     title='absorption',
                     units=ur.absorbance)

data.y = scp.Coord.arange(MnO2_Rest['components_number'].shape[0], title='sperctum number')
data.x = scp.Coord(MnO2_Rest.coords['Energy'], title='Energy', units=ur.eV)
# data

In [ ]:
data_denoise = scp.denoise(data)
MnO2_Rest_Denoise = data_denoise.to_xarray().rename('MnO2_Rest_Denoise')
MnO2_Rest_Denoise.attrs['units'] = r'a.u.'
# data_denoise.plot()

In [ ]:
%matplotlib inline
plt.style.use('default')
plt.close('all')

fig = plt.figure(figsize=(9.0, 2.5))
gs = gridspec.GridSpec(1, 3, width_ratios=[1,1,1], height_ratios=None,
                        wspace=0, hspace=0, figure=fig)

# 图 A
subfig = fig.add_subfigure(gs[0, 0], zorder=0)
ax = subfig.add_axes((0, 0, 1, 1),zorder=0)

xascolors = [colors[0], colors[4]]
labels = [r'0.2M Mn$\mathrm{^{2\!+}}$', r'${\alpha}$MnO$\mathrm{_2}$']
Mn_cmap = ListedColormap(mpl.colormaps['sunset'](np.linspace(0.5, 1.0, MnO2_Rest.shape[1])), name='Mn_cmap', )

for i in range(2):
    ax.plot(Mn_raw['Energy'].sel(Energy=slice(6530.0, 6630.0)), Mn_raw['Raw_ref'].sel(Energy=slice(6530.0, 6630.0))[:, i], c=xascolors[i], ls='--', lw=1.0, label=labels[i], zorder=5)
for i in range(MnO2_Rest.shape[1]):
    ax.plot(MnO2_Rest['Energy'], MnO2_Rest[:, i], c=Mn_cmap.colors[i], ls='-', lw=1.0)
    
ax.set_xlim(6530, 6630)
ax.set_xlabel(r'Energy (eV)', fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=20, offset=-10))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=10, offset=-10))
ax.set_ylim(-0.05, 2.1)
ax.set_ylabel(r"Absorption (a.u.)", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.4))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.2))
ax.tick_params(axis='x', labelsize=9) 
ax.tick_params(axis='y', labelsize=9)

colorbar = Colorbar(ax=ax.inset_axes([0.65, 0.01, 0.3, 0.05]), location='bottom', orientation='horizontal', 
                      cmap=Mn_cmap, ticklocation='top', spacing = 'proportional', drawedges=False)
colorbar.set_ticks([])

ax.legend(loc='upper left', bbox_to_anchor=(0.50, 1.03), ncols=1, frameon=False,
              labelcolor='linecolor', fontsize=11)
ax.text(0.65, 0.11, 'OCV', transform=ax.transAxes, fontsize=9,
        va='top', ha='left', fontfamily='Arial', )
ax.text(0.86, 0.11, '0.9 V', transform=ax.transAxes, fontsize=9,
        va='top', ha='left', fontfamily='Arial', )

# 图 B
subfig = fig.add_subfigure(gs[0, 1], zorder=0)
ax = subfig.add_axes((0.30, 0, 1, 1),zorder=0)

xascolors = [colors[0], colors[4]]
labels = [r'0.2M Mn$\mathrm{^{2\!+}}$', r'${\alpha}$MnO$\mathrm{_2}$']
Mn_cmap = ListedColormap(mpl.colormaps['sunset'](np.linspace(0.5, 1.0, MnO2_Rest.shape[1])), name='Mn_cmap', )

for i in range(2):
    ax.plot(Mn_raw['Energy'].sel(Energy=slice(6537.5, 6550.5)), Mn_raw['Raw_ref'].sel(Energy=slice(6537.5, 6550.5))[:, i], c=xascolors[i], ls='--', lw=1.0, label=labels[i], zorder=5)
ax.legend(loc='upper left', bbox_to_anchor=(0.0, 1.03), ncols=1, frameon=False, labelcolor='linecolor', fontsize=11)

for i in range(MnO2_Rest.shape[1]):
    ax.plot(MnO2_Rest['Energy'].sel(Energy=slice(6537.5, 6550.5)), MnO2_Rest.sel(Energy=slice(6537.5, 6550.5))[:, i], c=Mn_cmap.colors[i], ls='-', lw=1.0)
    
ax.set_xlim(6537.5, 6550.5)
ax.set_xlabel(r'Energy (eV)', fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=2, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=1, offset=0))
ax.set_ylim(-0.01, 0.105)
ax.set_ylabel(r"Absorption (a.u.)", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.02))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.01))
ax.tick_params(axis='x', labelsize=9) 
ax.tick_params(axis='y', labelsize=9)

colorbar = Colorbar(ax=ax.inset_axes([0.65, 0.01, 0.3, 0.05]), location='bottom', orientation='horizontal', 
                      cmap=Mn_cmap, ticklocation='top', spacing = 'proportional', drawedges=False)
colorbar.set_ticks([])


ax.text(0.65, 0.11, 'OCV', transform=ax.transAxes, fontsize=9,
        va='top', ha='left', fontfamily='Arial', )
ax.text(0.86, 0.11, '0.9 V', transform=ax.transAxes, fontsize=9,
        va='top', ha='left', fontfamily='Arial', )

# 图 C
subfig = fig.add_subfigure(gs[0, 2], zorder=0)
ax = subfig.add_axes((0.6, 0, 1, 1),zorder=0)

xascolors = [colors[0], colors[4]]
labels = [r'0.2M Mn$\mathrm{^{2\!+}}$', r'${\alpha}$MnO$\mathrm{_2}$']
Mn_cmap = ListedColormap(mpl.colormaps['sunset'](np.linspace(0.5, 1.0, MnO2_Rest_Denoise.shape[0])), name='Mn_cmap', )

for i in range(2):
    ax.plot(Mn_raw['Energy'].sel(Energy=slice(6537.5, 6550.5)), Mn_raw['Raw_ref'].sel(Energy=slice(6537.5, 6550.5))[:, i], c=xascolors[i], ls='--', lw=1.0, label=labels[i], zorder=5)
ax.legend(loc='upper left', bbox_to_anchor=(0.0, 1.03), ncols=1, frameon=False, labelcolor='linecolor', fontsize=11)

for i in range(MnO2_Rest_Denoise.shape[0]):
    ax.plot(MnO2_Rest_Denoise['Energy'].sel(Energy=slice(6537.5, 6550.5)), MnO2_Rest_Denoise.sel(Energy=slice(6537.5, 6550.5))[i, :], c=Mn_cmap.colors[i], ls='-', lw=1.0)

ax.set_xlim(6537.5, 6550.5)
ax.set_xlabel(r'Energy (eV)', fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=2, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=1, offset=0))
ax.set_ylim(-0.01, 0.105)
ax.set_ylabel(r"Absorption (a.u.)", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.02))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.01))
ax.tick_params(axis='x', labelsize=9) 
ax.tick_params(axis='y', labelsize=9)

colorbar = Colorbar(ax=ax.inset_axes([0.65, 0.01, 0.3, 0.05]), location='bottom', orientation='horizontal', 
                      cmap=Mn_cmap, ticklocation='top', spacing = 'proportional', drawedges=False)
colorbar.set_ticks([])


ax.text(0.65, 0.11, 'OCV', transform=ax.transAxes, fontsize=9,
        va='top', ha='left', fontfamily='Arial', )
ax.text(0.86, 0.11, '0.9 V', transform=ax.transAxes, fontsize=9,
        va='top', ha='left', fontfamily='Arial', )

plt.savefig(path.joinpath(path_out, r'1_opPreedge_Mn.tif'), transparent=False,
            pad_inches=0.05, bbox_inches='tight', dpi=300)
plt.show()

In [ ]:
# 保存数据
xr.merge([Mn_raw['Raw_spectrum'], Mn_raw['Raw_ref'], Mn_mcr_2_hm['MCR_HM_C'], Mn_mcr_2_hm['MCR_HM_St'], MnO2_Rest, MnO2_Rest_Denoise],).to_netcdf(path.joinpath(path_out, r'opPreedge_Mn.NETCDF4'), engine="h5netcdf")

MnO2_Rest_pd = MnO2_Rest.to_pandas()
Mn_ref_pd = Mn_raw['Raw_ref'].sel(Energy=slice(6530.0, 6630.0)).to_pandas()
MnO2_Rest_Denoise_pd = MnO2_Rest_Denoise.to_pandas()
pd.concat([Mn_ref_pd, MnO2_Rest_pd], axis=1, ignore_index=True).to_csv(path.joinpath(path_out, r'opPreedge_Mn_rest.csv'), header=True)
MnO2_Rest_Denoise_pd.T.to_csv(path.joinpath(path_out, r'opPreedge_Mn_denoise.csv'), header=True)

#### Charge

In [ ]:
# 读取数据
path_file = path(r'C:\Users\chengliu\OneDrive - UAB\ICMAB-Data\Zn-Mn\Uno\Result\XAS\Operando\αMnO2\XAS\CLAESS-2022-10\Results\cell3\Oct2022_cell3_P2a\Pre-Edge\1-1-Trials6-6538.0-6552.0-6539.2-6545.2')
refMnO2 = pd.read_csv(path.joinpath(path_file, r'pre_edge_baseline_MnO2.dat'), sep='\s+', comment='#', index_col=None, header=0).iloc[:, 0:4]
refMnO2_baseline = refMnO2.iloc[:, 2:4].copy().dropna(axis=0, how='any', ignore_index=True, inplace=False)
refMnO2 = refMnO2.iloc[:, 0:2].copy().dropna(axis=0, how='any', ignore_index=True, inplace=False)
refMnO2 = refMnO2.truncate(before=index(refMnO2.iloc[:, 0], refMnO2_baseline.iloc[0, 0]), after=index(refMnO2.iloc[:, 0], refMnO2_baseline.iloc[-1, 0]), axis=0, copy=True).reset_index(drop=True)

In [ ]:
filelist

In [ ]:
filelist = list(path_file.glob(r'*_denoise.dat'))
# filelist
df_data = pd.DataFrame()
for file  in filelist:
    df = pd.read_csv(file, sep='\s+', comment='#', index_col=None, header=0).iloc[:, 0:4]
    df_baseline = df.iloc[:, 2:4].copy().dropna(axis=0, how='any', ignore_index=True, inplace=False)    
    df = df.iloc[:, 0:2].copy().dropna(axis=0, how='any', ignore_index=True, inplace=False)
    df = df.truncate(before=index(df.iloc[:, 0], df_baseline.iloc[0, 0]), after=index(df.iloc[:, 0], df_baseline.iloc[-1, 0]), axis=0, copy=True).reset_index(drop=True)
    df_out = pd.concat([df, df_baseline.iloc[:, 1]], ignore_index=True, axis=1)
    df_out.columns = [file.stem[18:-8], 'Data', 'Baseline']
    df_out.to_csv(path.joinpath(path_out, f'{file.stem}.csv'), sep=',', index=None, header=True)
    df_data = pd.concat([df_data, df_out], ignore_index=True, axis=1)
df_data

In [ ]:
%matplotlib inline 

# 比较 Intensity 的变化
# 画图
plt.close('all')
fig = plt.figure(figsize=(3.3, 2.5))
gs = gridspec.GridSpec(1, 1, width_ratios=None, height_ratios=None,
                        wspace=0, hspace=0, figure=fig)

subfig = fig.add_subfigure(gs[0, 0], zorder=0)
ax = subfig.add_subplot()
ax.set_position([0, 0, 1.0, 1.0])

colors_cmap = ListedColormap(mpl.colormaps['sunset'](np.linspace(0.5, 1.0, df_data.shape[1]//3)), name='colors_cmap', )

for i in range(df_data.shape[1]//3):
    ax.plot(df_data.iloc[:, 3*i], df_data.iloc[:, 3*i+1]-df_data.iloc[:, 3*i+2], c=colors_cmap.colors[i], ls='-', label=None)
    
ax.set_xlabel(r'Energy (eV)', fontsize=11,)
ax.set_xlim(6538.0, 6547.0)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=2, offset=0.5))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=1, offset=0.5))

ax.set_ylabel(r'Intensity (a.u.)', fontsize=11,)
ax.set_ylim(-0.005, 0.06)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.02, offset=0))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.01, offset=0))

# Add colorbar and adjust ticks afterwards
colorbar = Colorbar(ax=ax.inset_axes([0.65, 0.8, 0.32, 0.07]), location='bottom', orientation='horizontal', 
                      cmap=colors_cmap, ticklocation='top', spacing = 'proportional', drawedges=False)
colorbar.set_ticks([])
ax.text(0.62, 0.94, 'OCV', transform=ax.transAxes, fontsize=9,
        va='top', ha='left', fontfamily='Arial', )
ax.text(0.75, 0.955, r'$\mathrm{1^{st}Discharge}$', transform=ax.transAxes, fontsize=9,
        va='top', ha='left', fontfamily='Arial', )


plt.savefig(path.joinpath(path_out, r'1_opPreEdge_all_data_1.tif'), 
            pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
plt.show()

In [ ]:
model_III_1.print_current_values()

In [ ]:
%matplotlib inline 
from hyperspy.axes import DataAxis
# 将 refMnO2 做成一个 scale model
energy_refMnO2_a = refMnO2.iloc[:, 0].to_numpy(dtype='float64')
energy_refMnO2 = DataAxis(axis=energy_refMnO2_a, index_in_array=None, name='Energy', units='eV',)

s_refMnO2 = hs.signals.Signal1D(refMnO2.iloc[:, 1]- refMnO2_baseline.iloc[:, 1], axes=[energy_refMnO2,])
model_refMnO2 = hs.model.components1D.ScalableFixedPattern(s_refMnO2) 
model_refMnO2.name= r'Mn_IV'
model_refMnO2.xscale.value=1.0
model_refMnO2.set_parameters_not_free(parameter_name_list=['xscale'])
model_refMnO2.yscale.bmin=0.0

# 一个高斯曲线
model_III_1 = hs.model.components1D.GaussianHF()
model_III_1.name = "Mn_III"
model_III_1.centre.bmin = 6540.85
model_III_1.centre.bmax = 6541.05
# model_III_1.centre.value = (model_III_1.centre.bmin + model_III_1.centre.bmax)/2
model_III_1.centre.value = 6540.95

model_III_1.height.bmin = 0
model_III_1.fwhm.bmax=0.6
model_III_1.fwhm.value=0.570037
model_III_1.set_parameters_not_free(parameter_name_list=['centre', 'fwhm'])

def PreEdge_fit(df, Gaussian=0, fig_plot=False, fig_save=True, padding=0.0):
    energy_df_a = df.iloc[:, 0].to_numpy(dtype='float64')
    energy_df = DataAxis(axis=energy_df_a, index_in_array=None, name='Energy', units='eV',)
    s_df = hs.signals.Signal1D(df.iloc[:, 1]-df.iloc[:, 2], axes=[energy_df,])
    model_df = s_df.create_model()
    
    if Gaussian == 0:
        model_df.append(model_refMnO2)
        model_df.fit_component(model_refMnO2,  signal_range=(6538.0, 6548.0), bounded=True, only_current=True) # 6538.0 6548.0
        model_df.assign_current_values_to_all(components_list = [model_refMnO2]) # apply those good values to all pixels
        model_df.fit_component(model_refMnO2, signal_range=(6538.0, 6548.0), bounded=True, only_current=False)
        
        Pre_Edge_Data = xr.Dataset(
            data_vars=dict(
            raw_df = (['Energy_aMnO2'], s_df.data),
            refMnO2 = (['Energy_refMnO2'], s_refMnO2.data),
            fit = (['Energy_fit'], model_df.as_signal().data),
            fit_Mn_IV = (['Energy_fit'], model_df.as_signal(component_list=['Mn_IV']).data),
            ),
            coords=dict(
                Energy_df = s_df.axes_manager[0].axis,
                Energy_refMnO2 = s_refMnO2.axes_manager[0].axis,
                Energy_fit= model_df.axes_manager[0].axis,
            ),
            attrs={'name': f'{filelist[i].stem[18:-8]}'},
        )
        
    if  Gaussian ==1:
        model_df.extend([model_refMnO2, model_III_1])
        model_df.fit_component(model_refMnO2,  signal_range=(6538.0, 6548.0), bounded=True, only_current=True) # 6539.0, 6547.0
        model_df.assign_current_values_to_all(components_list = [model_refMnO2]) # apply those good values to all pixels
        model_df.fit_component(model_refMnO2, signal_range=(6538.0, 6548.0), bounded=True, only_current=False)
        model_df.fit_component(model_III_1,  signal_range=(6538.0, 6548.0), bounded=True, only_current=True) # get good values 6539.0, 6542.0
        model_df.assign_current_values_to_all(components_list = [model_III_1]) # apply those good values to all pixels
        model_df.fit_component(model_III_1, signal_range=(6538.0, 6548.0), bounded=True, only_current=False)
        model_df.set_signal_range(6538.0, 6548.0)
        model_df.multifit(bounded=True) 
        
        pre_edge_ratio = model_df.as_signal(component_list=['Mn_IV']).isig[6538.0 :6548.0].integrate1D(axis='Energy')/(model_df.as_signal(component_list=['Mn_III']).isig[6538.0 :6548.0].integrate1D(axis='Energy') + model_df.as_signal(component_list=['Mn_IV']).isig[6538.0 :6548.0].integrate1D(axis='Energy'))
        
        Pre_Edge_Data = xr.Dataset(
            data_vars=dict(
                raw_df = (['Energy_aMnO2'], s_df.data),
                refMnO2 = (['Energy_refMnO2'], s_refMnO2.data),
                fit = (['Energy_fit'], model_df.as_signal().data),
                fit_Mn_IV = (['Energy_fit'], model_df.as_signal(component_list=['Mn_IV']).data),
                fit_Mn_III = (['Energy_fit'], model_df.as_signal(component_list=['Mn_III']).data),
            ),
            coords=dict(
                Energy_aMnO2 = s_df.axes_manager[0].axis,
                Energy_refMnO2 = s_refMnO2.axes_manager[0].axis,
                Energy_fit= model_df.axes_manager[0].axis,
            ),
            attrs={'name': f'{filelist[i].stem[18:-8]}',
                  'ratio_pre_edge_Mn_IV':pre_edge_ratio.data,
                  },  
        )

    # 画图      
    fig = plt.figure(figsize=(3.3, 2.5))
    gs = gridspec.GridSpec(1, 1, width_ratios=None, height_ratios=None,
                            wspace=0, hspace=0, figure=fig)

    subfig = fig.add_subfigure(gs[0, 0], zorder=0)
    ax = subfig.add_subplot()
    ax.set_position([0, 0, 1.0, 1.0])

    if Gaussian == 0:

        ax.plot(refMnO2.iloc[:, 0], refMnO2.iloc[:, 1]+padding, c=colors[0], ls='-', label=r'refMnO$_2$')
        ax.plot(refMnO2_baseline.iloc[:, 0], refMnO2_baseline.iloc[:, 1]+padding, ls='--', c=colors[0], label=None)
        ax.plot(df.iloc[:, 0], df.iloc[:, 1]+padding, c=colors[1], ls='-', label=f'{filelist[i].stem[18:-8]}')
        ax.plot(df.iloc[:, 0], df.iloc[:, 2]+padding, ls='--', c=colors[1], label=None)

        ax.plot(s_df.axes_manager[0].axis, s_df.data, c=colors[1], label=None, zorder=5)
        ax.plot(model_df.axes_manager[0].axis, model_df.as_signal().data, ls='--', c=colors[2], label='fit', zorder=6)
        ax.plot(model_df.axes_manager[0].axis, model_df.as_signal(component_list=['Mn_IV']).data, c=colors[0], label=None, alpha=0.5)
        ax.fill_between(model_df.axes_manager[0].axis, y1=0, y2=model_df.as_signal(component_list=['Mn_IV']).data, facecolor=colors[0], alpha=0.1)

        ax.set_xlabel(r'Energy (eV)', fontsize=11,)
        ax.set_xlim(6538.0, 6548.0)
        ax.xaxis.set_major_locator(ticker.MultipleLocator(base=2, offset=0))
        ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=1, offset=0))

        ax.set_ylabel(r'Intensity (a.u.)', fontsize=11,)
        ax.set_ylim(-0.005, 0.20)
        ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.05, offset=0))
        ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.025, offset=0))

        ax.legend(loc='upper left', bbox_to_anchor=(0.0, 1.02), ncols=1, frameon=False, fontsize=10, labelcolor='linecolor', columnspacing=0.4)
        ax.text(0.75, 0.25, f'Mn(IV): 1.0', horizontalalignment='left', verticalalignment='top', transform=ax.transAxes, fontsize=10)

    if Gaussian == 1:

        ax.plot(refMnO2.iloc[:, 0], refMnO2.iloc[:, 1]+padding, c=colors[0], ls='-', label=r'refMnO$_2$')
        ax.plot(refMnO2_baseline.iloc[:, 0], refMnO2_baseline.iloc[:, 1]+padding, ls='--', c=colors[0], label=None)
        ax.plot(df.iloc[:, 0], df.iloc[:, 1]+padding, c=colors[1], ls='-', label=f'{filelist[i].stem[18:-8]}')
        ax.plot(df.iloc[:, 0], df.iloc[:, 2]+padding, ls='--', c=colors[1], label=None)

        ax.plot(s_df.axes_manager[0].axis, s_df.data, c=colors[1], label=None, zorder=5)
        ax.plot(model_df.axes_manager[0].axis, model_df.as_signal().data, ls='--', c=colors[2], label='fit', zorder=6)
        ax.plot(model_df.axes_manager[0].axis, model_df.as_signal(component_list=['Mn_IV']).data, c=colors[0], label=None, alpha=0.5)
        ax.fill_between(model_df.axes_manager[0].axis, y1=0, y2=model_df.as_signal(component_list=['Mn_IV']).data, facecolor=colors[0], alpha=0.1)
        ax.plot(model_df.axes_manager[0].axis, model_df.as_signal(component_list=['Mn_III']).data, c=colors[3], label=r'Mn_III', alpha=0.5)
        ax.fill_between(model_df.axes_manager[0].axis, y1=0, y2=model_df.as_signal(component_list=['Mn_III']).data, facecolor=colors[3], alpha=0.1)

        ax.set_xlabel(r'Energy (eV)', fontsize=11,)
        ax.set_xlim(6538.0, 6548.0)
        ax.xaxis.set_major_locator(ticker.MultipleLocator(base=2, offset=0))
        ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=1, offset=0))

        ax.set_ylabel(r'Intensity (a.u.)', fontsize=11,)
        ax.set_ylim(-0.005, 0.20)
        ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.05, offset=0))
        ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.025, offset=0))

        ax.legend(loc='upper left', bbox_to_anchor=(0.0, 1.02), ncols=1, frameon=False, fontsize=10, labelcolor='linecolor', columnspacing=0.4)
        ax.text(0.7, 0.20, f'Mn(IV): {pre_edge_ratio.data[0]:.2f}', horizontalalignment='left', verticalalignment='top', transform=ax.transAxes, fontsize=10)
        
    if fig_save==True:
        plt.savefig(path.joinpath(path_out, f'1_{filelist[i].stem}_Gaussian_{Gaussian}.tif'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
        model_df.save_parameters2file(path.joinpath(path_out, f'{filelist[i].stem}_Gaussian_{Gaussian}_parameters'))
        model_df.save(path.joinpath(path_out, f'{filelist[i].stem}l_Gaussian_{Gaussian}_mode'), 'model', overwrite=True)
        Pre_Edge_Data.to_netcdf(path.joinpath(path_out, f'{filelist[i].stem}_Gaussian_{Gaussian}.NETCDF4'), engine="h5netcdf")
        
    if fig_plot==True:     
        plt.show()
    if fig_plot==False:     
        plt.close('all')  

    return None

In [ ]:
for i in range(df_data.shape[1]//3):
    df = df_data.iloc[:, 3*i:3*i+3]
    PreEdge_fit(df, Gaussian=0, fig_plot=False, fig_save=True, padding=0.08)
    PreEdge_fit(df, Gaussian=1, fig_plot=False, fig_save=True, padding=0.06)   